In [2]:
import json
import urllib
import zipfile
import io
import numpy as np

# Load dataset
NB_DATASET_URL = 'https://github.com/SeijiEmery/shape-net-data/raw/master/car-vertex-data/nobuyuki-data.json.zip'
with urllib.request.urlopen(NB_DATASET_URL) as f:
    with zipfile.ZipFile(io.BytesIO(f.read()), 'r') as zf:
        with zf.open('nobuyuki-data.json', 'r') as f:
            sw_dataset = json.loads(f.read())

sw_dimensions = set(map(len, sw_dataset.values()))
print("dataset members have the following lengths: %s (expected 18438)"%(sw_dimensions,))

sw_keys = [ key for key in sw_dataset.keys() if len(sw_dataset[key]) == 18438 ]
non_corrupt_values = [ x for x in sw_dataset.values() if len(x) == 18438 ]
corrupt_values = [ x for x in sw_dataset.values() if len(x) != 18438 ]
print("Has %s corrupt value(s), %s values, %s keys"%(
    len(corrupt_values), len(non_corrupt_values), len(sw_keys)))

sw_data = np.array(non_corrupt_values)
print("dataset size: %s"%(sw_data.shape,))

x_train, x_test = np.split(sw_data, [800], 0)
print("partitioning as train = %s, test = %s"%(x_train.shape, x_test.shape))

dataset members have the following lengths: {18438} (expected 18438)
Has 0 corrupt value(s), 1240 values, 1240 keys
dataset size: (1240, 18438)
partitioning as train = (800, 18438), test = (440, 18438)


In [23]:
from keras.models import Sequential
from keras.layers import Input, Dense, Activation
from keras.models import Model, load_model
import os

INPUT_DIM = 18438
HIDDEN_DIM = 128
LATENT_DIM = 10

model = Sequential([
    Dense(HIDDEN_DIM, input_shape=(INPUT_DIM,)),
    Activation('relu'),
    Dense(LATENT_DIM),
    Activation('relu'),
    Dense(HIDDEN_DIM),
    Activation('relu'),
    Dense(INPUT_DIM),
    Activation('linear')
])
trained_epochs = 0
model.compile(optimizer='adam', loss='mse')
model.summary()

def train (epochs, batch_size=50):
    global model, trained_epochs
    print("Training: epoch %s -> %s"%(trained_epochs, epochs + trained_epochs))
    model.fit(x_train, x_train,
             epochs=epochs,
             batch_size=batch_size)
    trained_epochs += epochs
    if not os.path.exists('altmodel'):
        os.makedirs('altmodel')
    path = os.path.join('altmodel', 'altmodel-epoch-%s.h5'%(trained_epochs,))
    print("Saving snapshot as %s"%path)
    model.save(path)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 128)               2360192   
_________________________________________________________________
activation_25 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                1290      
_________________________________________________________________
activation_26 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               1408      
_________________________________________________________________
activation_27 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 18438)             2378502   
__________

In [36]:
for i in range(10):
    train(100)
    export_model_objs(trained_epochs)

Training: epoch 500 -> 600
Epoch 1/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 2/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 3/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 4/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 5/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0028
Epoch 6/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 7/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 9/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0026
Epoch 10/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0028
Epoch 11/100
800/800 [==============================] - 3s 3ms/step - loss: 0.0032
Epoch 12/100
800/800 [==============================] - 3s 3ms/step -

800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 100/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Saving snapshot as altmodel/altmodel-epoch-600.h5
Loading altmodel/altmodel-epoch-600.h5
predicting for epoch 600
generating write-obj jobs for epoch 600
dispatching write-obj jobs for epoch 600
writing altencoder-objs/600/input/train-b8c8758f8ee058182a9eb0f146e94477-input.obj
writing altencoder-objs/600/output/train-b8c8758f8ee058182a9eb0f146e94477-output.obj
writing altencoder-objs/600/input/test-a52f704da0d3c947ca92a4cdad802b45-input.obj
writing altencoder-objs/600/output/test-a52f704da0d3c947ca92a4cdad802b45-output.obj
writing altencoder-objs/600/input/train-9b0d7aac4500546911718baa6d4afc0b-input.obj
writing altencoder-objs/600/output/train-9b0d7aac4500546911718baa6d4afc0b-output.obj
writing altencoder-objs/600/input/test-f61888bca4392106bc790478b4a84bf2-input.obj
writing altencoder-objs/600/output/test-f61888bca4392106bc790478b4a

800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 55/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 56/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0029
Epoch 57/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 58/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 59/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 60/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 61/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 62/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 63/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 64/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 65/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 66/100
800/

800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 10/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 11/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 12/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 13/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 14/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 15/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0026
Epoch 16/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0026
Epoch 17/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0025
Epoch 18/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 19/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 20/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0027
Epoch 21/100
800/

writing altencoder-objs/800/input/train-ba1b8956b1d84856369912a1ddf80ef7-input.obj
writing altencoder-objs/800/output/train-ba1b8956b1d84856369912a1ddf80ef7-output.obj
writing altencoder-objs/800/input/test-d2e1dc21db9b45df6436916a86a90ed7-input.obj
writing altencoder-objs/800/output/test-d2e1dc21db9b45df6436916a86a90ed7-output.obj
writing altencoder-objs/800/input/train-909422a9d1b42e1380360680c1602c7d-input.obj
writing altencoder-objs/800/output/train-909422a9d1b42e1380360680c1602c7d-output.obj
writing altencoder-objs/800/input/test-dd0b595b15a7203e185ce5d54f27f6b9-input.obj
writing altencoder-objs/800/output/test-dd0b595b15a7203e185ce5d54f27f6b9-output.obj
writing altencoder-objs/800/input/train-15cc37ac9b3b756acfd826dd468a5497-input.obj
writing altencoder-objs/800/output/train-15cc37ac9b3b756acfd826dd468a5497-output.obj
writing altencoder-objs/800/input/test-8fc3cde1054cc1aaceb4167db4d0e4de-input.obj
writing altencoder-objs/800/output/test-8fc3cde1054cc1aaceb4167db4d0e4de-output.ob

...

Training: epoch 1200 -> 1300
Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0024
Epoch 2/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0024
Epoch 3/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 4/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 5/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 6/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 7/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 9/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 10/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0024
Epoch 11/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 12/100
800/800 [==============================] - 2s 2ms/step

800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 100/100
800/800 [==============================] - 1s 1ms/step - loss: 0.0024
Saving snapshot as altmodel/altmodel-epoch-1300.h5
Loading altmodel/altmodel-epoch-1300.h5
predicting for epoch 1300
generating write-obj jobs for epoch 1300
dispatching write-obj jobs for epoch 1300
writing altencoder-objs/1300/input/train-b8c8758f8ee058182a9eb0f146e94477-input.obj
writing altencoder-objs/1300/output/train-b8c8758f8ee058182a9eb0f146e94477-output.obj
writing altencoder-objs/1300/input/test-a52f704da0d3c947ca92a4cdad802b45-input.obj
writing altencoder-objs/1300/output/test-a52f704da0d3c947ca92a4cdad802b45-output.obj
writing altencoder-objs/1300/input/train-9b0d7aac4500546911718baa6d4afc0b-input.obj
writing altencoder-objs/1300/output/train-9b0d7aac4500546911718baa6d4afc0b-output.obj
writing altencoder-objs/1300/input/test-f61888bca4392106bc790478b4a84bf2-input.obj
writing altencoder-objs/1300/output/test-f61888bca43921

800/800 [==============================] - 2s 2ms/step - loss: 0.0024
Epoch 55/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 56/100
800/800 [==============================] - 3s 3ms/step - loss: 0.0024
Epoch 57/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 58/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 59/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 60/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0024
Epoch 61/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0024
Epoch 62/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 63/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 64/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0025
Epoch 65/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 66/100
800/

800/800 [==============================] - 1s 1ms/step - loss: 0.0024
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.0024
Epoch 11/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 12/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 13/100
800/800 [==============================] - 1s 1ms/step - loss: 0.0024
Epoch 14/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 15/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 16/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0024
Epoch 17/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 18/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0024
Epoch 19/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 20/100
800/800 [==============================] - 1s 2ms/step - loss: 0.0025
Epoch 21/100
800/

writing altencoder-objs/1500/input/train-ba1b8956b1d84856369912a1ddf80ef7-input.obj
writing altencoder-objs/1500/output/train-ba1b8956b1d84856369912a1ddf80ef7-output.obj
writing altencoder-objs/1500/input/test-d2e1dc21db9b45df6436916a86a90ed7-input.obj
writing altencoder-objs/1500/output/test-d2e1dc21db9b45df6436916a86a90ed7-output.obj
writing altencoder-objs/1500/input/train-909422a9d1b42e1380360680c1602c7d-input.obj
writing altencoder-objs/1500/output/train-909422a9d1b42e1380360680c1602c7d-output.obj
writing altencoder-objs/1500/input/test-dd0b595b15a7203e185ce5d54f27f6b9-input.obj
writing altencoder-objs/1500/output/test-dd0b595b15a7203e185ce5d54f27f6b9-output.obj
writing altencoder-objs/1500/input/train-15cc37ac9b3b756acfd826dd468a5497-input.obj
writing altencoder-objs/1500/output/train-15cc37ac9b3b756acfd826dd468a5497-output.obj
writing altencoder-objs/1500/input/test-8fc3cde1054cc1aaceb4167db4d0e4de-input.obj
writing altencoder-objs/1500/output/test-8fc3cde1054cc1aaceb4167db4d0e4

In [26]:
# for i in range(10):
#     train(10)

Training: epoch 10 -> 20
Epoch 1/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0058
Epoch 2/10
800/800 [==============================] - 2s 2ms/step - loss: 0.0040
Epoch 3/10
800/800 [==============================] - 2s 2ms/step - loss: 0.0038
Epoch 4/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0037
Epoch 5/10
800/800 [==============================] - 2s 2ms/step - loss: 0.0037
Epoch 6/10
800/800 [==============================] - 3s 3ms/step - loss: 0.0045
Epoch 7/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0042
Epoch 8/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0038
Epoch 9/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0037
Epoch 10/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0051
Saving snapshot as altmodel/altmodel-epoch-20.h5
Training: epoch 20 -> 30
Epoch 1/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0041
Epoch 2/1

800/800 [==============================] - 1s 2ms/step - loss: 0.0036
Epoch 4/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0035
Epoch 5/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0033
Epoch 6/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0033
Epoch 7/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0035
Epoch 8/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0034
Epoch 9/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0034
Epoch 10/10
800/800 [==============================] - 1s 2ms/step - loss: 0.0033
Saving snapshot as altmodel/altmodel-epoch-110.h5


In [6]:
def load_model_from_epoch (epoch):
    path = os.path.join('altmodel', 'altmodel-epoch-%s.h5'%(epoch,))
    print("Loading %s"%path)
    return load_model(path)

def predict (epoch):
    model = load_model_from_epoch(epoch)
    print("Generating prediction")
    return map(model.predict, (x_train, x_test))
y_train, y_test = predict(2)
y_train.shape, y_test.shape

Loading altmodel/altmodel-epoch-2.h5
Generating prediction


((800, 18438), (440, 18438))

In [10]:
OBJ_TEMPLATE = '''
# {name}
{vertices}
f 1 1541 4611 1540
f 389 3079 4611 1541
f 1155 3077 4611 3079
f 388 1540 4611 3077
f 101 1933 4612 1931
f 775 3851 4612 1933
f 1155 3079 4612 3851
f 389 1931 4612 3079
f 291 2691 4613 2692
f 773 3847 4613 2691
f 1155 3851 4613 3847
f 775 2692 4613 3851
f 100 1927 4614 1929
f 388 3077 4614 1927
f 1155 3847 4614 3077
f 773 1929 4614 3847
f 29 1645 4615 1643
f 493 3287 4615 1645
f 1156 3283 4615 3287
f 491 1643 4615 3283
f 199 2325 4616 2323
f 971 4243 4616 2325
f 1156 3287 4616 4243
f 493 2323 4616 3287
f 291 2692 4617 2694
f 775 3852 4617 2692
f 1156 4243 4617 3852
f 971 2694 4617 4243
f 101 1932 4618 1933
f 491 3283 4618 1932
f 1156 3852 4618 3283
f 775 1933 4618 3852
f 75 1827 4619 1828
f 675 3651 4619 1827
f 1157 3653 4619 3651
f 676 1828 4619 3653
f 197 2317 4620 2316
f 967 4235 4620 2317
f 1157 3651 4620 4235
f 675 2316 4620 3651
f 291 2694 4621 2693
f 971 4244 4621 2694
f 1157 4235 4621 4244
f 967 2693 4621 4235
f 199 2324 4622 2325
f 676 3653 4622 2324
f 1157 4244 4622 3653
f 971 2325 4622 4244
f 28 1639 4623 1641
f 487 3275 4623 1639
f 1158 3279 4623 3275
f 489 1641 4623 3279
f 100 1929 4624 1928
f 773 3848 4624 1929
f 1158 3275 4624 3848
f 487 1928 4624 3275
f 291 2693 4625 2691
f 967 4236 4625 2693
f 1158 3848 4625 4236
f 773 2691 4625 3848
f 197 2315 4626 2317
f 489 3279 4626 2315
f 1158 4236 4626 3279
f 967 2317 4626 4236
f 11 1573 4627 1571
f 421 3143 4627 1573
f 1159 3139 4627 3143
f 419 1571 4627 3139
f 133 2061 4628 2059
f 839 3979 4628 2061
f 1159 3143 4628 3979
f 421 2059 4628 3143
f 292 2695 4629 2696
f 835 3971 4629 2695
f 1159 3979 4629 3971
f 839 2696 4629 3979
f 131 2051 4630 2053
f 419 3139 4630 2051
f 1159 3971 4630 3139
f 835 2053 4630 3971
f 55 1749 4631 1747
f 597 3495 4631 1749
f 1160 3491 4631 3495
f 595 1747 4631 3491
f 251 2533 4632 2531
f 1075 4451 4632 2533
f 1160 3495 4632 4451
f 597 2531 4632 3495
f 292 2696 4633 2698
f 839 3980 4633 2696
f 1160 4451 4633 3980
f 1075 2698 4633 4451
f 133 2060 4634 2061
f 595 3491 4634 2060
f 1160 3980 4634 3491
f 839 2061 4634 3980
f 75 1828 4635 1830
f 676 3654 4635 1828
f 1161 3657 4635 3654
f 678 1830 4635 3657
f 199 2326 4636 2324
f 972 4245 4636 2326
f 1161 3654 4636 4245
f 676 2324 4636 3654
f 292 2698 4637 2697
f 1075 4452 4637 2698
f 1161 4245 4637 4452
f 972 2697 4637 4245
f 251 2532 4638 2533
f 678 3657 4638 2532
f 1161 4452 4638 3657
f 1075 2533 4638 4452
f 29 1644 4639 1645
f 492 3285 4639 1644
f 1162 3288 4639 3285
f 493 1645 4639 3288
f 131 2053 4640 2052
f 835 3972 4640 2053
f 1162 3285 4640 3972
f 492 2052 4640 3285
f 292 2697 4641 2695
f 972 4246 4641 2697
f 1162 3972 4641 4246
f 835 2695 4641 3972
f 199 2323 4642 2326
f 493 3288 4642 2323
f 1162 4246 4642 3288
f 972 2326 4642 4246
f 21 1611 4643 1612
f 459 3219 4643 1611
f 1163 3221 4643 3219
f 460 1612 4643 3221
f 171 2213 4644 2211
f 915 4131 4644 2213
f 1163 3219 4644 4131
f 459 2211 4644 3219
f 293 2700 4645 2699
f 917 4135 4645 2700
f 1163 4131 4645 4135
f 915 2699 4645 4131
f 172 2215 4646 2217
f 460 3221 4646 2215
f 1163 4135 4646 3221
f 917 2217 4646 4135
f 53 1741 4647 1740
f 589 3479 4647 1741
f 1164 3477 4647 3479
f 588 1740 4647 3477
f 247 2517 4648 2515
f 1067 4435 4648 2517
f 1164 3479 4648 4435
f 589 2515 4648 3479
f 293 2699 4649 2701
f 915 4132 4649 2699
f 1164 4435 4649 4132
f 1067 2701 4649 4435
f 171 2212 4650 2213
f 588 3477 4650 2212
f 1164 4132 4650 3477
f 915 2213 4650 4132
f 75 1830 4651 1829
f 678 3658 4651 1830
f 1165 3655 4651 3658
f 677 1829 4651 3655
f 251 2534 4652 2532
f 1076 4453 4652 2534
f 1165 3658 4652 4453
f 678 2532 4652 3658
f 293 2701 4653 2702
f 1067 4436 4653 2701
f 1165 4453 4653 4436
f 1076 2702 4653 4453
f 247 2516 4654 2517
f 677 3655 4654 2516
f 1165 4436 4654 3655
f 1067 2517 4654 4436
f 55 1748 4655 1749
f 596 3493 4655 1748
f 1166 3496 4655 3493
f 597 1749 4655 3496
f 172 2217 4656 2216
f 917 4136 4656 2217
f 1166 3493 4656 4136
f 596 2216 4656 3493
f 293 2702 4657 2700
f 1076 4454 4657 2702
f 1166 4136 4657 4454
f 917 2700 4657 4136
f 251 2531 4658 2534
f 597 3496 4658 2531
f 1166 4454 4658 3496
f 1076 2534 4658 4454
f 10 1567 4659 1569
f 415 3131 4659 1567
f 1167 3135 4659 3131
f 417 1569 4659 3135
f 127 2037 4660 2035
f 827 3955 4660 2037
f 1167 3131 4660 3955
f 415 2035 4660 3131
f 294 2704 4661 2703
f 831 3963 4661 2704
f 1167 3955 4661 3963
f 827 2703 4661 3955
f 129 2043 4662 2045
f 417 3135 4662 2043
f 1167 3963 4662 3135
f 831 2045 4662 3963
f 28 1641 4663 1640
f 489 3280 4663 1641
f 1168 3277 4663 3280
f 488 1640 4663 3277
f 197 2318 4664 2315
f 968 4237 4664 2318
f 1168 3280 4664 4237
f 489 2315 4664 3280
f 294 2703 4665 2705
f 827 3956 4665 2703
f 1168 4237 4665 3956
f 968 2705 4665 4237
f 127 2036 4666 2037
f 488 3277 4666 2036
f 1168 3956 4666 3277
f 827 2037 4666 3956
f 75 1829 4667 1827
f 677 3656 4667 1829
f 1169 3652 4667 3656
f 675 1827 4667 3652
f 247 2518 4668 2516
f 1068 4437 4668 2518
f 1169 3656 4668 4437
f 677 2516 4668 3656
f 294 2705 4669 2706
f 968 4238 4669 2705
f 1169 4437 4669 4238
f 1068 2706 4669 4437
f 197 2316 4670 2318
f 675 3652 4670 2316
f 1169 4238 4670 3652
f 968 2318 4670 4238
f 53 1739 4671 1741
f 587 3475 4671 1739
f 1170 3480 4671 3475
f 589 1741 4671 3480
f 129 2045 4672 2044
f 831 3964 4672 2045
f 1170 3475 4672 3964
f 587 2044 4672 3475
f 294 2706 4673 2704
f 1068 4438 4673 2706
f 1170 3964 4673 4438
f 831 2704 4673 3964
f 247 2515 4674 2518
f 589 3480 4674 2515
f 1170 4438 4674 3480
f 1068 2518 4674 4438
f 5 1553 4675 1551
f 401 3103 4675 1553
f 1171 3099 4675 3103
f 399 1551 4675 3099
f 113 1981 4676 1979
f 799 3899 4676 1981
f 1171 3103 4676 3899
f 401 1979 4676 3103
f 295 2707 4677 2708
f 795 3891 4677 2707
f 1171 3899 4677 3891
f 799 2708 4677 3899
f 111 1971 4678 1973
f 399 3099 4678 1971
f 1171 3891 4678 3099
f 795 1973 4678 3891
f 41 1693 4679 1691
f 541 3383 4679 1693
f 1172 3379 4679 3383
f 539 1691 4679 3379
f 223 2421 4680 2419
f 1019 4339 4680 2421
f 1172 3383 4680 4339
f 541 2419 4680 3383
f 295 2708 4681 2710
f 799 3900 4681 2708
f 1172 4339 4681 3900
f 1019 2710 4681 4339
f 113 1980 4682 1981
f 539 3379 4682 1980
f 1172 3900 4682 3379
f 799 1981 4682 3900
f 76 1831 4683 1832
f 679 3659 4683 1831
f 1173 3661 4683 3659
f 680 1832 4683 3661
f 219 2405 4684 2404
f 1011 4323 4684 2405
f 1173 3659 4684 4323
f 679 2404 4684 3659
f 295 2710 4685 2709
f 1019 4340 4685 2710
f 1173 4323 4685 4340
f 1011 2709 4685 4323
f 223 2420 4686 2421
f 680 3661 4686 2420
f 1173 4340 4686 3661
f 1019 2421 4686 4340
f 39 1683 4687 1685
f 531 3363 4687 1683
f 1174 3367 4687 3363
f 533 1685 4687 3367
f 111 1973 4688 1972
f 795 3892 4688 1973
f 1174 3363 4688 3892
f 531 1972 4688 3363
f 295 2709 4689 2707
f 1011 4324 4689 2709
f 1174 3892 4689 4324
f 795 2707 4689 3892
f 219 2403 4690 2405
f 533 3367 4690 2403
f 1174 4324 4690 3367
f 1011 2405 4690 4324
f 18 1601 4691 1599
f 449 3199 4691 1601
f 1175 3195 4691 3199
f 447 1599 4691 3195
f 161 2173 4692 2171
f 895 4091 4692 2173
f 1175 3199 4692 4091
f 449 2171 4692 3199
f 296 2711 4693 2712
f 891 4083 4693 2711
f 1175 4091 4693 4083
f 895 2712 4693 4091
f 159 2163 4694 2165
f 447 3195 4694 2163
f 1175 4083 4694 3195
f 891 2165 4694 4083
f 69 1805 4695 1803
f 653 3607 4695 1805
f 1176 3603 4695 3607
f 651 1803 4695 3603
f 279 2645 4696 2643
f 1131 4563 4696 2645
f 1176 3607 4696 4563
f 653 2643 4696 3607
f 296 2712 4697 2714
f 895 4092 4697 2712
f 1176 4563 4697 4092
f 1131 2714 4697 4563
f 161 2172 4698 2173
f 651 3603 4698 2172
f 1176 4092 4698 3603
f 895 2173 4698 4092
f 76 1832 4699 1834
f 680 3662 4699 1832
f 1177 3665 4699 3662
f 682 1834 4699 3665
f 223 2422 4700 2420
f 1020 4341 4700 2422
f 1177 3662 4700 4341
f 680 2420 4700 3662
f 296 2714 4701 2713
f 1131 4564 4701 2714
f 1177 4341 4701 4564
f 1020 2713 4701 4341
f 279 2644 4702 2645
f 682 3665 4702 2644
f 1177 4564 4702 3665
f 1131 2645 4702 4564
f 41 1692 4703 1693
f 540 3381 4703 1692
f 1178 3384 4703 3381
f 541 1693 4703 3384
f 159 2165 4704 2164
f 891 4084 4704 2165
f 1178 3381 4704 4084
f 540 2164 4704 3381
f 296 2713 4705 2711
f 1020 4342 4705 2713
f 1178 4084 4705 4342
f 891 2711 4705 4084
f 223 2419 4706 2422
f 541 3384 4706 2419
f 1178 4342 4706 3384
f 1020 2422 4706 4342
f 21 1612 4707 1614
f 460 3222 4707 1612
f 1179 3225 4707 3222
f 462 1614 4707 3225
f 172 2218 4708 2215
f 918 4137 4708 2218
f 1179 3222 4708 4137
f 460 2215 4708 3222
f 297 2716 4709 2715
f 921 4143 4709 2716
f 1179 4137 4709 4143
f 918 2715 4709 4137
f 174 2223 4710 2225
f 462 3225 4710 2223
f 1179 4143 4710 3225
f 921 2225 4710 4143
f 55 1750 4711 1748
f 598 3497 4711 1750
f 1180 3494 4711 3497
f 596 1748 4711 3494
f 252 2537 4712 2535
f 1077 4455 4712 2537
f 1180 3497 4712 4455
f 598 2535 4712 3497
f 297 2715 4713 2717
f 918 4138 4713 2715
f 1180 4455 4713 4138
f 1077 2717 4713 4455
f 172 2216 4714 2218
f 596 3494 4714 2216
f 1180 4138 4714 3494
f 918 2218 4714 4138
f 76 1834 4715 1833
f 682 3666 4715 1834
f 1181 3663 4715 3666
f 681 1833 4715 3663
f 279 2646 4716 2644
f 1132 4565 4716 2646
f 1181 3666 4716 4565
f 682 2644 4716 3666
f 297 2717 4717 2718
f 1077 4456 4717 2717
f 1181 4565 4717 4456
f 1132 2718 4717 4565
f 252 2536 4718 2537
f 681 3663 4718 2536
f 1181 4456 4718 3663
f 1077 2537 4718 4456
f 69 1804 4719 1805
f 652 3605 4719 1804
f 1182 3608 4719 3605
f 653 1805 4719 3608
f 174 2225 4720 2224
f 921 4144 4720 2225
f 1182 3605 4720 4144
f 652 2224 4720 3605
f 297 2718 4721 2716
f 1132 4566 4721 2718
f 1182 4144 4721 4566
f 921 2716 4721 4144
f 279 2643 4722 2646
f 653 3608 4722 2643
f 1182 4566 4722 3608
f 1132 2646 4722 4566
f 11 1572 4723 1573
f 420 3141 4723 1572
f 1183 3144 4723 3141
f 421 1573 4723 3144
f 132 2057 4724 2055
f 837 3975 4724 2057
f 1183 3141 4724 3975
f 420 2055 4724 3141
f 298 2720 4725 2719
f 840 3981 4725 2720
f 1183 3975 4725 3981
f 837 2719 4725 3975
f 133 2059 4726 2062
f 421 3144 4726 2059
f 1183 3981 4726 3144
f 840 2062 4726 3981
f 39 1685 4727 1684
f 533 3368 4727 1685
f 1184 3365 4727 3368
f 532 1684 4727 3365
f 219 2406 4728 2403
f 1012 4325 4728 2406
f 1184 3368 4728 4325
f 533 2403 4728 3368
f 298 2719 4729 2721
f 837 3976 4729 2719
f 1184 4325 4729 3976
f 1012 2721 4729 4325
f 132 2056 4730 2057
f 532 3365 4730 2056
f 1184 3976 4730 3365
f 837 2057 4730 3976
f 76 1833 4731 1831
f 681 3664 4731 1833
f 1185 3660 4731 3664
f 679 1831 4731 3660
f 252 2538 4732 2536
f 1078 4457 4732 2538
f 1185 3664 4732 4457
f 681 2536 4732 3664
f 298 2721 4733 2722
f 1012 4326 4733 2721
f 1185 4457 4733 4326
f 1078 2722 4733 4457
f 219 2404 4734 2406
f 679 3660 4734 2404
f 1185 4326 4734 3660
f 1012 2406 4734 4326
f 55 1747 4735 1750
f 595 3492 4735 1747
f 1186 3498 4735 3492
f 598 1750 4735 3498
f 133 2062 4736 2060
f 840 3982 4736 2062
f 1186 3492 4736 3982
f 595 2060 4736 3492
f 298 2722 4737 2720
f 1078 4458 4737 2722
f 1186 3982 4737 4458
f 840 2720 4737 3982
f 252 2535 4738 2538
f 598 3498 4738 2535
f 1186 4458 4738 3498
f 1078 2538 4738 4458
f 7 1557 4739 1558
f 405 3111 4739 1557
f 1187 3113 4739 3111
f 406 1558 4739 3113
f 117 1997 4740 1995
f 807 3915 4740 1997
f 1187 3111 4740 3915
f 405 1995 4740 3111
f 299 2724 4741 2723
f 809 3919 4741 2724
f 1187 3915 4741 3919
f 807 2723 4741 3915
f 118 1999 4742 2001
f 406 3113 4742 1999
f 1187 3919 4742 3113
f 809 2001 4742 3919
f 45 1709 4743 1707
f 557 3415 4743 1709
f 1188 3411 4743 3415
f 555 1707 4743 3411
f 231 2453 4744 2451
f 1035 4371 4744 2453
f 1188 3415 4744 4371
f 557 2451 4744 3415
f 299 2723 4745 2725
f 807 3916 4745 2723
f 1188 4371 4745 3916
f 1035 2725 4745 4371
f 117 1996 4746 1997
f 555 3411 4746 1996
f 1188 3916 4746 3411
f 807 1997 4746 3916
f 77 1836 4747 1835
f 684 3669 4747 1836
f 1189 3667 4747 3669
f 683 1835 4747 3667
f 233 2461 4748 2460
f 1039 4379 4748 2461
f 1189 3669 4748 4379
f 684 2460 4748 3669
f 299 2725 4749 2726
f 1035 4372 4749 2725
f 1189 4379 4749 4372
f 1039 2726 4749 4379
f 231 2452 4750 2453
f 683 3667 4750 2452
f 1189 4372 4750 3667
f 1035 2453 4750 4372
f 46 1711 4751 1713
f 559 3419 4751 1711
f 1190 3423 4751 3419
f 561 1713 4751 3423
f 118 2001 4752 2000
f 809 3920 4752 2001
f 1190 3419 4752 3920
f 559 2000 4752 3419
f 299 2726 4753 2724
f 1039 4380 4753 2726
f 1190 3920 4753 4380
f 809 2724 4753 3920
f 233 2459 4754 2461
f 561 3423 4754 2459
f 1190 4380 4754 3423
f 1039 2461 4754 4380
f 15 1589 4755 1588
f 437 3175 4755 1589
f 1191 3173 4755 3175
f 436 1588 4755 3173
f 149 2125 4756 2123
f 871 4043 4756 2125
f 1191 3175 4756 4043
f 437 2123 4756 3175
f 300 2727 4757 2728
f 869 4039 4757 2727
f 1191 4043 4757 4039
f 871 2728 4757 4043
f 148 2119 4758 2121
f 436 3173 4758 2119
f 1191 4039 4758 3173
f 869 2121 4758 4039
f 63 1781 4759 1779
f 629 3559 4759 1781
f 1192 3555 4759 3559
f 627 1779 4759 3555
f 267 2597 4760 2595
f 1107 4515 4760 2597
f 1192 3559 4760 4515
f 629 2595 4760 3559
f 300 2728 4761 2730
f 871 4044 4761 2728
f 1192 4515 4761 4044
f 1107 2730 4761 4515
f 149 2124 4762 2125
f 627 3555 4762 2124
f 1192 4044 4762 3555
f 871 2125 4762 4044
f 77 1835 4763 1837
f 683 3668 4763 1835
f 1193 3671 4763 3668
f 685 1837 4763 3671
f 231 2454 4764 2452
f 1036 4373 4764 2454
f 1193 3668 4764 4373
f 683 2452 4764 3668
f 300 2730 4765 2729
f 1107 4516 4765 2730
f 1193 4373 4765 4516
f 1036 2729 4765 4373
f 267 2596 4766 2597
f 685 3671 4766 2596
f 1193 4516 4766 3671
f 1107 2597 4766 4516
f 45 1708 4767 1709
f 556 3413 4767 1708
f 1194 3416 4767 3413
f 557 1709 4767 3416
f 148 2121 4768 2120
f 869 4040 4768 2121
f 1194 3413 4768 4040
f 556 2120 4768 3413
f 300 2729 4769 2727
f 1036 4374 4769 2729
f 1194 4040 4769 4374
f 869 2727 4769 4040
f 231 2451 4770 2454
f 557 3416 4770 2451
f 1194 4374 4770 3416
f 1036 2454 4770 4374
f 21 1614 4771 1613
f 462 3226 4771 1614
f 1195 3223 4771 3226
f 461 1613 4771 3223
f 174 2226 4772 2223
f 922 4145 4772 2226
f 1195 3226 4772 4145
f 462 2223 4772 3226
f 301 2731 4773 2732
f 919 4139 4773 2731
f 1195 4145 4773 4139
f 922 2732 4773 4145
f 173 2219 4774 2221
f 461 3223 4774 2219
f 1195 4139 4774 3223
f 919 2221 4774 4139
f 69 1806 4775 1804
f 654 3609 4775 1806
f 1196 3606 4775 3609
f 652 1804 4775 3606
f 280 2649 4776 2647
f 1133 4567 4776 2649
f 1196 3609 4776 4567
f 654 2647 4776 3609
f 301 2732 4777 2734
f 922 4146 4777 2732
f 1196 4567 4777 4146
f 1133 2734 4777 4567
f 174 2224 4778 2226
f 652 3606 4778 2224
f 1196 4146 4778 3606
f 922 2226 4778 4146
f 77 1837 4779 1838
f 685 3672 4779 1837
f 1197 3673 4779 3672
f 686 1838 4779 3673
f 267 2598 4780 2596
f 1108 4517 4780 2598
f 1197 3672 4780 4517
f 685 2596 4780 3672
f 301 2734 4781 2733
f 1133 4568 4781 2734
f 1197 4517 4781 4568
f 1108 2733 4781 4517
f 280 2648 4782 2649
f 686 3673 4782 2648
f 1197 4568 4782 3673
f 1133 2649 4782 4568
f 63 1780 4783 1781
f 628 3557 4783 1780
f 1198 3560 4783 3557
f 629 1781 4783 3560
f 173 2221 4784 2220
f 919 4140 4784 2221
f 1198 3557 4784 4140
f 628 2220 4784 3557
f 301 2733 4785 2731
f 1108 4518 4785 2733
f 1198 4140 4785 4518
f 919 2731 4785 4140
f 267 2595 4786 2598
f 629 3560 4786 2595
f 1198 4518 4786 3560
f 1108 2598 4786 4518
f 18 1600 4787 1601
f 448 3197 4787 1600
f 1199 3200 4787 3197
f 449 1601 4787 3200
f 160 2169 4788 2167
f 893 4087 4788 2169
f 1199 3197 4788 4087
f 448 2167 4788 3197
f 302 2736 4789 2735
f 896 4093 4789 2736
f 1199 4087 4789 4093
f 893 2735 4789 4087
f 161 2171 4790 2174
f 449 3200 4790 2171
f 1199 4093 4790 3200
f 896 2174 4790 4093
f 46 1713 4791 1712
f 561 3424 4791 1713
f 1200 3421 4791 3424
f 560 1712 4791 3421
f 233 2462 4792 2459
f 1040 4381 4792 2462
f 1200 3424 4792 4381
f 561 2459 4792 3424
f 302 2735 4793 2737
f 893 4088 4793 2735
f 1200 4381 4793 4088
f 1040 2737 4793 4381
f 160 2168 4794 2169
f 560 3421 4794 2168
f 1200 4088 4794 3421
f 893 2169 4794 4088
f 77 1838 4795 1836
f 686 3674 4795 1838
f 1201 3670 4795 3674
f 684 1836 4795 3670
f 280 2650 4796 2648
f 1134 4569 4796 2650
f 1201 3674 4796 4569
f 686 2648 4796 3674
f 302 2737 4797 2738
f 1040 4382 4797 2737
f 1201 4569 4797 4382
f 1134 2738 4797 4569
f 233 2460 4798 2462
f 684 3670 4798 2460
f 1201 4382 4798 3670
f 1040 2462 4798 4382
f 69 1803 4799 1806
f 651 3604 4799 1803
f 1202 3610 4799 3604
f 654 1806 4799 3610
f 161 2174 4800 2172
f 896 4094 4800 2174
f 1202 3604 4800 4094
f 651 2172 4800 3604
f 302 2738 4801 2736
f 1134 4570 4801 2738
f 1202 4094 4801 4570
f 896 2736 4801 4094
f 280 2647 4802 2650
f 654 3610 4802 2647
f 1202 4570 4802 3610
f 1134 2650 4802 4570
f 3 1545 4803 1547
f 393 3087 4803 1545
f 1203 3091 4803 3087
f 395 1547 4803 3091
f 105 1949 4804 1947
f 783 3867 4804 1949
f 1203 3087 4804 3867
f 393 1947 4804 3087
f 303 2740 4805 2739
f 787 3875 4805 2740
f 1203 3867 4805 3875
f 783 2739 4805 3867
f 107 1955 4806 1957
f 395 3091 4806 1955
f 1203 3875 4806 3091
f 787 1957 4806 3875
f 33 1661 4807 1659
f 509 3319 4807 1661
f 1204 3315 4807 3319
f 507 1659 4807 3315
f 207 2357 4808 2355
f 987 4275 4808 2357
f 1204 3319 4808 4275
f 509 2355 4808 3319
f 303 2739 4809 2741
f 783 3868 4809 2739
f 1204 4275 4809 3868
f 987 2741 4809 4275
f 105 1948 4810 1949
f 507 3315 4810 1948
f 1204 3868 4810 3315
f 783 1949 4810 3868
f 78 1840 4811 1839
f 688 3677 4811 1840
f 1205 3675 4811 3677
f 687 1839 4811 3675
f 211 2373 4812 2372
f 995 4291 4812 2373
f 1205 3677 4812 4291
f 688 2372 4812 3677
f 303 2741 4813 2742
f 987 4276 4813 2741
f 1205 4291 4813 4276
f 995 2742 4813 4291
f 207 2356 4814 2357
f 687 3675 4814 2356
f 1205 4276 4814 3675
f 987 2357 4814 4276
f 35 1667 4815 1669
f 515 3331 4815 1667
f 1206 3335 4815 3331
f 517 1669 4815 3335
f 107 1957 4816 1956
f 787 3876 4816 1957
f 1206 3331 4816 3876
f 515 1956 4816 3331
f 303 2742 4817 2740
f 995 4292 4817 2742
f 1206 3876 4817 4292
f 787 2740 4817 3876
f 211 2371 4818 2373
f 517 3335 4818 2371
f 1206 4292 4818 3335
f 995 2373 4818 4292
f 10 1569 4819 1568
f 417 3136 4819 1569
f 1207 3133 4819 3136
f 416 1568 4819 3133
f 129 2046 4820 2043
f 832 3965 4820 2046
f 1207 3136 4820 3965
f 417 2043 4820 3136
f 304 2743 4821 2744
f 829 3959 4821 2743
f 1207 3965 4821 3959
f 832 2744 4821 3965
f 128 2039 4822 2041
f 416 3133 4822 2039
f 1207 3959 4822 3133
f 829 2041 4822 3959
f 53 1742 4823 1739
f 590 3481 4823 1742
f 1208 3476 4823 3481
f 587 1739 4823 3476
f 248 2521 4824 2519
f 1069 4439 4824 2521
f 1208 3481 4824 4439
f 590 2519 4824 3481
f 304 2744 4825 2746
f 832 3966 4825 2744
f 1208 4439 4825 3966
f 1069 2746 4825 4439
f 129 2044 4826 2046
f 587 3476 4826 2044
f 1208 3966 4826 3476
f 832 2046 4826 3966
f 78 1839 4827 1841
f 687 3676 4827 1839
f 1209 3679 4827 3676
f 689 1841 4827 3679
f 207 2358 4828 2356
f 988 4277 4828 2358
f 1209 3676 4828 4277
f 687 2356 4828 3676
f 304 2746 4829 2745
f 1069 4440 4829 2746
f 1209 4277 4829 4440
f 988 2745 4829 4277
f 248 2520 4830 2521
f 689 3679 4830 2520
f 1209 4440 4830 3679
f 1069 2521 4830 4440
f 33 1660 4831 1661
f 508 3317 4831 1660
f 1210 3320 4831 3317
f 509 1661 4831 3320
f 128 2041 4832 2040
f 829 3960 4832 2041
f 1210 3317 4832 3960
f 508 2040 4832 3317
f 304 2745 4833 2743
f 988 4278 4833 2745
f 1210 3960 4833 4278
f 829 2743 4833 3960
f 207 2355 4834 2358
f 509 3320 4834 2355
f 1210 4278 4834 3320
f 988 2358 4834 4278
f 21 1613 4835 1611
f 461 3224 4835 1613
f 1211 3220 4835 3224
f 459 1611 4835 3220
f 173 2222 4836 2219
f 920 4141 4836 2222
f 1211 3224 4836 4141
f 461 2219 4836 3224
f 305 2747 4837 2748
f 916 4133 4837 2747
f 1211 4141 4837 4133
f 920 2748 4837 4141
f 171 2211 4838 2214
f 459 3220 4838 2211
f 1211 4133 4838 3220
f 916 2214 4838 4133
f 63 1782 4839 1780
f 630 3561 4839 1782
f 1212 3558 4839 3561
f 628 1780 4839 3558
f 268 2601 4840 2599
f 1109 4519 4840 2601
f 1212 3561 4840 4519
f 630 2599 4840 3561
f 305 2748 4841 2750
f 920 4142 4841 2748
f 1212 4519 4841 4142
f 1109 2750 4841 4519
f 173 2220 4842 2222
f 628 3558 4842 2220
f 1212 4142 4842 3558
f 920 2222 4842 4142
f 78 1841 4843 1842
f 689 3680 4843 1841
f 1213 3681 4843 3680
f 690 1842 4843 3681
f 248 2522 4844 2520
f 1070 4441 4844 2522
f 1213 3680 4844 4441
f 689 2520 4844 3680
f 305 2750 4845 2749
f 1109 4520 4845 2750
f 1213 4441 4845 4520
f 1070 2749 4845 4441
f 268 2600 4846 2601
f 690 3681 4846 2600
f 1213 4520 4846 3681
f 1109 2601 4846 4520
f 53 1740 4847 1742
f 588 3478 4847 1740
f 1214 3482 4847 3478
f 590 1742 4847 3482
f 171 2214 4848 2212
f 916 4134 4848 2214
f 1214 3478 4848 4134
f 588 2212 4848 3478
f 305 2749 4849 2747
f 1070 4442 4849 2749
f 1214 4134 4849 4442
f 916 2747 4849 4134
f 248 2519 4850 2522
f 590 3482 4850 2519
f 1214 4442 4850 3482
f 1070 2522 4850 4442
f 15 1587 4851 1589
f 435 3171 4851 1587
f 1215 3176 4851 3171
f 437 1589 4851 3176
f 147 2117 4852 2115
f 867 4035 4852 2117
f 1215 3171 4852 4035
f 435 2115 4852 3171
f 306 2752 4853 2751
f 872 4045 4853 2752
f 1215 4035 4853 4045
f 867 2751 4853 4035
f 149 2123 4854 2126
f 437 3176 4854 2123
f 1215 4045 4854 3176
f 872 2126 4854 4045
f 35 1669 4855 1668
f 517 3336 4855 1669
f 1216 3333 4855 3336
f 516 1668 4855 3333
f 211 2374 4856 2371
f 996 4293 4856 2374
f 1216 3336 4856 4293
f 517 2371 4856 3336
f 306 2751 4857 2753
f 867 4036 4857 2751
f 1216 4293 4857 4036
f 996 2753 4857 4293
f 147 2116 4858 2117
f 516 3333 4858 2116
f 1216 4036 4858 3333
f 867 2117 4858 4036
f 78 1842 4859 1840
f 690 3682 4859 1842
f 1217 3678 4859 3682
f 688 1840 4859 3678
f 268 2602 4860 2600
f 1110 4521 4860 2602
f 1217 3682 4860 4521
f 690 2600 4860 3682
f 306 2753 4861 2754
f 996 4294 4861 2753
f 1217 4521 4861 4294
f 1110 2754 4861 4521
f 211 2372 4862 2374
f 688 3678 4862 2372
f 1217 4294 4862 3678
f 996 2374 4862 4294
f 63 1779 4863 1782
f 627 3556 4863 1779
f 1218 3562 4863 3556
f 630 1782 4863 3562
f 149 2126 4864 2124
f 872 4046 4864 2126
f 1218 3556 4864 4046
f 627 2124 4864 3556
f 306 2754 4865 2752
f 1110 4522 4865 2754
f 1218 4046 4865 4522
f 872 2752 4865 4046
f 268 2599 4866 2602
f 630 3562 4866 2599
f 1218 4522 4866 3562
f 1110 2602 4866 4522
f 2 1543 4867 1544
f 391 3083 4867 1543
f 1219 3085 4867 3083
f 392 1544 4867 3085
f 103 1941 4868 1939
f 779 3859 4868 1941
f 1219 3083 4868 3859
f 391 1939 4868 3083
f 307 2756 4869 2755
f 781 3863 4869 2756
f 1219 3859 4869 3863
f 779 2755 4869 3859
f 104 1943 4870 1945
f 392 3085 4870 1943
f 1219 3863 4870 3085
f 781 1945 4870 3863
f 31 1653 4871 1651
f 501 3303 4871 1653
f 1220 3299 4871 3303
f 499 1651 4871 3299
f 203 2341 4872 2339
f 979 4259 4872 2341
f 1220 3303 4872 4259
f 501 2339 4872 3303
f 307 2755 4873 2757
f 779 3860 4873 2755
f 1220 4259 4873 3860
f 979 2757 4873 4259
f 103 1940 4874 1941
f 499 3299 4874 1940
f 1220 3860 4874 3299
f 779 1941 4874 3860
f 79 1844 4875 1843
f 692 3685 4875 1844
f 1221 3683 4875 3685
f 691 1843 4875 3683
f 205 2349 4876 2348
f 983 4267 4876 2349
f 1221 3685 4876 4267
f 692 2348 4876 3685
f 307 2757 4877 2758
f 979 4260 4877 2757
f 1221 4267 4877 4260
f 983 2758 4877 4267
f 203 2340 4878 2341
f 691 3683 4878 2340
f 1221 4260 4878 3683
f 979 2341 4878 4260
f 32 1655 4879 1657
f 503 3307 4879 1655
f 1222 3311 4879 3307
f 505 1657 4879 3311
f 104 1945 4880 1944
f 781 3864 4880 1945
f 1222 3307 4880 3864
f 503 1944 4880 3307
f 307 2758 4881 2756
f 983 4268 4881 2758
f 1222 3864 4881 4268
f 781 2756 4881 3864
f 205 2347 4882 2349
f 505 3311 4882 2347
f 1222 4268 4882 3311
f 983 2349 4882 4268
f 12 1577 4883 1575
f 425 3151 4883 1577
f 1223 3147 4883 3151
f 423 1575 4883 3147
f 137 2077 4884 2075
f 847 3995 4884 2077
f 1223 3151 4884 3995
f 425 2075 4884 3151
f 308 2759 4885 2760
f 843 3987 4885 2759
f 1223 3995 4885 3987
f 847 2760 4885 3995
f 135 2067 4886 2069
f 423 3147 4886 2067
f 1223 3987 4886 3147
f 843 2069 4886 3987
f 57 1757 4887 1755
f 605 3511 4887 1757
f 1224 3507 4887 3511
f 603 1755 4887 3507
f 255 2549 4888 2547
f 1083 4467 4888 2549
f 1224 3511 4888 4467
f 605 2547 4888 3511
f 308 2760 4889 2762
f 847 3996 4889 2760
f 1224 4467 4889 3996
f 1083 2762 4889 4467
f 137 2076 4890 2077
f 603 3507 4890 2076
f 1224 3996 4890 3507
f 847 2077 4890 3996
f 79 1843 4891 1845
f 691 3684 4891 1843
f 1225 3687 4891 3684
f 693 1845 4891 3687
f 203 2342 4892 2340
f 980 4261 4892 2342
f 1225 3684 4892 4261
f 691 2340 4892 3684
f 308 2762 4893 2761
f 1083 4468 4893 2762
f 1225 4261 4893 4468
f 980 2761 4893 4261
f 255 2548 4894 2549
f 693 3687 4894 2548
f 1225 4468 4894 3687
f 1083 2549 4894 4468
f 31 1652 4895 1653
f 500 3301 4895 1652
f 1226 3304 4895 3301
f 501 1653 4895 3304
f 135 2069 4896 2068
f 843 3988 4896 2069
f 1226 3301 4896 3988
f 500 2068 4896 3301
f 308 2761 4897 2759
f 980 4262 4897 2761
f 1226 3988 4897 4262
f 843 2759 4897 3988
f 203 2339 4898 2342
f 501 3304 4898 2339
f 1226 4262 4898 3304
f 980 2342 4898 4262
f 22 1616 4899 1615
f 464 3229 4899 1616
f 1227 3227 4899 3229
f 463 1615 4899 3227
f 176 2233 4900 2231
f 925 4151 4900 2233
f 1227 3229 4900 4151
f 464 2231 4900 3229
f 309 2763 4901 2764
f 923 4147 4901 2763
f 1227 4151 4901 4147
f 925 2764 4901 4151
f 175 2227 4902 2229
f 463 3227 4902 2227
f 1227 4147 4902 3227
f 923 2229 4902 4147
f 59 1765 4903 1764
f 613 3527 4903 1765
f 1228 3525 4903 3527
f 612 1764 4903 3525
f 259 2565 4904 2563
f 1091 4483 4904 2565
f 1228 3527 4904 4483
f 613 2563 4904 3527
f 309 2764 4905 2766
f 925 4152 4905 2764
f 1228 4483 4905 4152
f 1091 2766 4905 4483
f 176 2232 4906 2233
f 612 3525 4906 2232
f 1228 4152 4906 3525
f 925 2233 4906 4152
f 79 1845 4907 1846
f 693 3688 4907 1845
f 1229 3689 4907 3688
f 694 1846 4907 3689
f 255 2550 4908 2548
f 1084 4469 4908 2550
f 1229 3688 4908 4469
f 693 2548 4908 3688
f 309 2766 4909 2765
f 1091 4484 4909 2766
f 1229 4469 4909 4484
f 1084 2765 4909 4469
f 259 2564 4910 2565
f 694 3689 4910 2564
f 1229 4484 4910 3689
f 1091 2565 4910 4484
f 57 1756 4911 1757
f 604 3509 4911 1756
f 1230 3512 4911 3509
f 605 1757 4911 3512
f 175 2229 4912 2228
f 923 4148 4912 2229
f 1230 3509 4912 4148
f 604 2228 4912 3509
f 309 2765 4913 2763
f 1084 4470 4913 2765
f 1230 4148 4913 4470
f 923 2763 4913 4148
f 255 2547 4914 2550
f 605 3512 4914 2547
f 1230 4470 4914 3512
f 1084 2550 4914 4470
f 13 1579 4915 1581
f 427 3155 4915 1579
f 1231 3159 4915 3155
f 429 1581 4915 3159
f 139 2085 4916 2083
f 851 4003 4916 2085
f 1231 3155 4916 4003
f 427 2083 4916 3155
f 310 2768 4917 2767
f 855 4011 4917 2768
f 1231 4003 4917 4011
f 851 2767 4917 4003
f 141 2091 4918 2093
f 429 3159 4918 2091
f 1231 4011 4918 3159
f 855 2093 4918 4011
f 32 1657 4919 1656
f 505 3312 4919 1657
f 1232 3309 4919 3312
f 504 1656 4919 3309
f 205 2350 4920 2347
f 984 4269 4920 2350
f 1232 3312 4920 4269
f 505 2347 4920 3312
f 310 2767 4921 2769
f 851 4004 4921 2767
f 1232 4269 4921 4004
f 984 2769 4921 4269
f 139 2084 4922 2085
f 504 3309 4922 2084
f 1232 4004 4922 3309
f 851 2085 4922 4004
f 79 1846 4923 1844
f 694 3690 4923 1846
f 1233 3686 4923 3690
f 692 1844 4923 3686
f 259 2566 4924 2564
f 1092 4485 4924 2566
f 1233 3690 4924 4485
f 694 2564 4924 3690
f 310 2769 4925 2770
f 984 4270 4925 2769
f 1233 4485 4925 4270
f 1092 2770 4925 4485
f 205 2348 4926 2350
f 692 3686 4926 2348
f 1233 4270 4926 3686
f 984 2350 4926 4270
f 59 1763 4927 1765
f 611 3523 4927 1763
f 1234 3528 4927 3523
f 613 1765 4927 3528
f 141 2093 4928 2092
f 855 4012 4928 2093
f 1234 3523 4928 4012
f 611 2092 4928 3523
f 310 2770 4929 2768
f 1092 4486 4929 2770
f 1234 4012 4929 4486
f 855 2768 4929 4012
f 259 2563 4930 2566
f 613 3528 4930 2563
f 1234 4486 4930 3528
f 1092 2566 4930 4486
f 4 1550 4931 1548
f 398 3097 4931 1550
f 1235 3093 4931 3097
f 396 1548 4931 3093
f 110 1969 4932 1967
f 793 3887 4932 1969
f 1235 3097 4932 3887
f 398 1967 4932 3097
f 311 2771 4933 2772
f 789 3879 4933 2771
f 1235 3887 4933 3879
f 793 2772 4933 3887
f 108 1959 4934 1961
f 396 3093 4934 1959
f 1235 3879 4934 3093
f 789 1961 4934 3879
f 38 1681 4935 1679
f 529 3359 4935 1681
f 1236 3355 4935 3359
f 527 1679 4935 3355
f 217 2397 4936 2395
f 1007 4315 4936 2397
f 1236 3359 4936 4315
f 529 2395 4936 3359
f 311 2772 4937 2774
f 793 3888 4937 2772
f 1236 4315 4937 3888
f 1007 2774 4937 4315
f 110 1968 4938 1969
f 527 3355 4938 1968
f 1236 3888 4938 3355
f 793 1969 4938 3888
f 80 1847 4939 1848
f 695 3691 4939 1847
f 1237 3693 4939 3691
f 696 1848 4939 3693
f 213 2381 4940 2380
f 999 4299 4940 2381
f 1237 3691 4940 4299
f 695 2380 4940 3691
f 311 2774 4941 2773
f 1007 4316 4941 2774
f 1237 4299 4941 4316
f 999 2773 4941 4299
f 217 2396 4942 2397
f 696 3693 4942 2396
f 1237 4316 4942 3693
f 1007 2397 4942 4316
f 36 1671 4943 1673
f 519 3339 4943 1671
f 1238 3343 4943 3339
f 521 1673 4943 3343
f 108 1961 4944 1960
f 789 3880 4944 1961
f 1238 3339 4944 3880
f 519 1960 4944 3339
f 311 2773 4945 2771
f 999 4300 4945 2773
f 1238 3880 4945 4300
f 789 2771 4945 3880
f 213 2379 4946 2381
f 521 3343 4946 2379
f 1238 4300 4946 3343
f 999 2381 4946 4300
f 16 1593 4947 1591
f 441 3183 4947 1593
f 1239 3179 4947 3183
f 439 1591 4947 3179
f 153 2141 4948 2139
f 879 4059 4948 2141
f 1239 3183 4948 4059
f 441 2139 4948 3183
f 312 2775 4949 2776
f 875 4051 4949 2775
f 1239 4059 4949 4051
f 879 2776 4949 4059
f 151 2131 4950 2133
f 439 3179 4950 2131
f 1239 4051 4950 3179
f 875 2133 4950 4051
f 65 1789 4951 1787
f 637 3575 4951 1789
f 1240 3571 4951 3575
f 635 1787 4951 3571
f 271 2613 4952 2611
f 1115 4531 4952 2613
f 1240 3575 4952 4531
f 637 2611 4952 3575
f 312 2776 4953 2778
f 879 4060 4953 2776
f 1240 4531 4953 4060
f 1115 2778 4953 4531
f 153 2140 4954 2141
f 635 3571 4954 2140
f 1240 4060 4954 3571
f 879 2141 4954 4060
f 80 1848 4955 1850
f 696 3694 4955 1848
f 1241 3697 4955 3694
f 698 1850 4955 3697
f 217 2398 4956 2396
f 1008 4317 4956 2398
f 1241 3694 4956 4317
f 696 2396 4956 3694
f 312 2778 4957 2777
f 1115 4532 4957 2778
f 1241 4317 4957 4532
f 1008 2777 4957 4317
f 271 2612 4958 2613
f 698 3697 4958 2612
f 1241 4532 4958 3697
f 1115 2613 4958 4532
f 38 1680 4959 1681
f 528 3357 4959 1680
f 1242 3360 4959 3357
f 529 1681 4959 3360
f 151 2133 4960 2132
f 875 4052 4960 2133
f 1242 3357 4960 4052
f 528 2132 4960 3357
f 312 2777 4961 2775
f 1008 4318 4961 2777
f 1242 4052 4961 4318
f 875 2775 4961 4052
f 217 2395 4962 2398
f 529 3360 4962 2395
f 1242 4318 4962 3360
f 1008 2398 4962 4318
f 22 1615 4963 1617
f 463 3228 4963 1615
f 1243 3231 4963 3228
f 465 1617 4963 3231
f 175 2230 4964 2227
f 924 4149 4964 2230
f 1243 3228 4964 4149
f 463 2227 4964 3228
f 313 2780 4965 2779
f 927 4155 4965 2780
f 1243 4149 4965 4155
f 924 2779 4965 4149
f 177 2235 4966 2237
f 465 3231 4966 2235
f 1243 4155 4966 3231
f 927 2237 4966 4155
f 57 1758 4967 1756
f 606 3513 4967 1758
f 1244 3510 4967 3513
f 604 1756 4967 3510
f 256 2553 4968 2551
f 1085 4471 4968 2553
f 1244 3513 4968 4471
f 606 2551 4968 3513
f 313 2779 4969 2781
f 924 4150 4969 2779
f 1244 4471 4969 4150
f 1085 2781 4969 4471
f 175 2228 4970 2230
f 604 3510 4970 2228
f 1244 4150 4970 3510
f 924 2230 4970 4150
f 80 1850 4971 1849
f 698 3698 4971 1850
f 1245 3695 4971 3698
f 697 1849 4971 3695
f 271 2614 4972 2612
f 1116 4533 4972 2614
f 1245 3698 4972 4533
f 698 2612 4972 3698
f 313 2781 4973 2782
f 1085 4472 4973 2781
f 1245 4533 4973 4472
f 1116 2782 4973 4533
f 256 2552 4974 2553
f 697 3695 4974 2552
f 1245 4472 4974 3695
f 1085 2553 4974 4472
f 65 1788 4975 1789
f 636 3573 4975 1788
f 1246 3576 4975 3573
f 637 1789 4975 3576
f 177 2237 4976 2236
f 927 4156 4976 2237
f 1246 3573 4976 4156
f 636 2236 4976 3573
f 313 2782 4977 2780
f 1116 4534 4977 2782
f 1246 4156 4977 4534
f 927 2780 4977 4156
f 271 2611 4978 2614
f 637 3576 4978 2611
f 1246 4534 4978 3576
f 1116 2614 4978 4534
f 12 1576 4979 1577
f 424 3149 4979 1576
f 1247 3152 4979 3149
f 425 1577 4979 3152
f 136 2073 4980 2071
f 845 3991 4980 2073
f 1247 3149 4980 3991
f 424 2071 4980 3149
f 314 2784 4981 2783
f 848 3997 4981 2784
f 1247 3991 4981 3997
f 845 2783 4981 3991
f 137 2075 4982 2078
f 425 3152 4982 2075
f 1247 3997 4982 3152
f 848 2078 4982 3997
f 36 1673 4983 1672
f 521 3344 4983 1673
f 1248 3341 4983 3344
f 520 1672 4983 3341
f 213 2382 4984 2379
f 1000 4301 4984 2382
f 1248 3344 4984 4301
f 521 2379 4984 3344
f 314 2783 4985 2785
f 845 3992 4985 2783
f 1248 4301 4985 3992
f 1000 2785 4985 4301
f 136 2072 4986 2073
f 520 3341 4986 2072
f 1248 3992 4986 3341
f 845 2073 4986 3992
f 80 1849 4987 1847
f 697 3696 4987 1849
f 1249 3692 4987 3696
f 695 1847 4987 3692
f 256 2554 4988 2552
f 1086 4473 4988 2554
f 1249 3696 4988 4473
f 697 2552 4988 3696
f 314 2785 4989 2786
f 1000 4302 4989 2785
f 1249 4473 4989 4302
f 1086 2786 4989 4473
f 213 2380 4990 2382
f 695 3692 4990 2380
f 1249 4302 4990 3692
f 1000 2382 4990 4302
f 57 1755 4991 1758
f 603 3508 4991 1755
f 1250 3514 4991 3508
f 606 1758 4991 3514
f 137 2078 4992 2076
f 848 3998 4992 2078
f 1250 3508 4992 3998
f 603 2076 4992 3508
f 314 2786 4993 2784
f 1086 4474 4993 2786
f 1250 3998 4993 4474
f 848 2784 4993 3998
f 256 2551 4994 2554
f 606 3514 4994 2551
f 1250 4474 4994 3514
f 1086 2554 4994 4474
f 8 1561 4995 1560
f 409 3119 4995 1561
f 1251 3117 4995 3119
f 408 1560 4995 3117
f 121 2013 4996 2011
f 815 3931 4996 2013
f 1251 3119 4996 3931
f 409 2011 4996 3119
f 315 2787 4997 2788
f 813 3927 4997 2787
f 1251 3931 4997 3927
f 815 2788 4997 3931
f 120 2007 4998 2009
f 408 3117 4998 2007
f 1251 3927 4998 3117
f 813 2009 4998 3927
f 49 1725 4999 1723
f 573 3447 4999 1725
f 1252 3443 4999 3447
f 571 1723 4999 3443
f 239 2485 5000 2483
f 1051 4403 5000 2485
f 1252 3447 5000 4403
f 573 2483 5000 3447
f 315 2788 5001 2790
f 815 3932 5001 2788
f 1252 4403 5001 3932
f 1051 2790 5001 4403
f 121 2012 5002 2013
f 571 3443 5002 2012
f 1252 3932 5002 3443
f 815 2013 5002 3932
f 81 1851 5003 1852
f 699 3699 5003 1851
f 1253 3701 5003 3699
f 700 1852 5003 3701
f 237 2477 5004 2476
f 1047 4395 5004 2477
f 1253 3699 5004 4395
f 699 2476 5004 3699
f 315 2790 5005 2789
f 1051 4404 5005 2790
f 1253 4395 5005 4404
f 1047 2789 5005 4395
f 239 2484 5006 2485
f 700 3701 5006 2484
f 1253 4404 5006 3701
f 1051 2485 5006 4404
f 48 1719 5007 1721
f 567 3435 5007 1719
f 1254 3439 5007 3435
f 569 1721 5007 3439
f 120 2009 5008 2008
f 813 3928 5008 2009
f 1254 3435 5008 3928
f 567 2008 5008 3435
f 315 2789 5009 2787
f 1047 4396 5009 2789
f 1254 3928 5009 4396
f 813 2787 5009 3928
f 237 2475 5010 2477
f 569 3439 5010 2475
f 1254 4396 5010 3439
f 1047 2477 5010 4396
f 19 1605 5011 1604
f 453 3207 5011 1605
f 1255 3205 5011 3207
f 452 1604 5011 3205
f 165 2189 5012 2187
f 903 4107 5012 2189
f 1255 3207 5012 4107
f 453 2187 5012 3207
f 316 2791 5013 2792
f 901 4103 5013 2791
f 1255 4107 5013 4103
f 903 2792 5013 4107
f 164 2183 5014 2185
f 452 3205 5014 2183
f 1255 4103 5014 3205
f 901 2185 5014 4103
f 71 1813 5015 1811
f 661 3623 5015 1813
f 1256 3619 5015 3623
f 659 1811 5015 3619
f 283 2661 5016 2659
f 1139 4579 5016 2661
f 1256 3623 5016 4579
f 661 2659 5016 3623
f 316 2792 5017 2794
f 903 4108 5017 2792
f 1256 4579 5017 4108
f 1139 2794 5017 4579
f 165 2188 5018 2189
f 659 3619 5018 2188
f 1256 4108 5018 3619
f 903 2189 5018 4108
f 81 1852 5019 1854
f 700 3702 5019 1852
f 1257 3705 5019 3702
f 702 1854 5019 3705
f 239 2486 5020 2484
f 1052 4405 5020 2486
f 1257 3702 5020 4405
f 700 2484 5020 3702
f 316 2794 5021 2793
f 1139 4580 5021 2794
f 1257 4405 5021 4580
f 1052 2793 5021 4405
f 283 2660 5022 2661
f 702 3705 5022 2660
f 1257 4580 5022 3705
f 1139 2661 5022 4580
f 49 1724 5023 1725
f 572 3445 5023 1724
f 1258 3448 5023 3445
f 573 1725 5023 3448
f 164 2185 5024 2184
f 901 4104 5024 2185
f 1258 3445 5024 4104
f 572 2184 5024 3445
f 316 2793 5025 2791
f 1052 4406 5025 2793
f 1258 4104 5025 4406
f 901 2791 5025 4104
f 239 2483 5026 2486
f 573 3448 5026 2483
f 1258 4406 5026 3448
f 1052 2486 5026 4406
f 22 1617 5027 1618
f 465 3232 5027 1617
f 1259 3233 5027 3232
f 466 1618 5027 3233
f 177 2238 5028 2235
f 928 4157 5028 2238
f 1259 3232 5028 4157
f 465 2235 5028 3232
f 317 2796 5029 2795
f 929 4159 5029 2796
f 1259 4157 5029 4159
f 928 2795 5029 4157
f 178 2239 5030 2241
f 466 3233 5030 2239
f 1259 4159 5030 3233
f 929 2241 5030 4159
f 65 1790 5031 1788
f 638 3577 5031 1790
f 1260 3574 5031 3577
f 636 1788 5031 3574
f 272 2617 5032 2615
f 1117 4535 5032 2617
f 1260 3577 5032 4535
f 638 2615 5032 3577
f 317 2795 5033 2797
f 928 4158 5033 2795
f 1260 4535 5033 4158
f 1117 2797 5033 4535
f 177 2236 5034 2238
f 636 3574 5034 2236
f 1260 4158 5034 3574
f 928 2238 5034 4158
f 81 1854 5035 1853
f 702 3706 5035 1854
f 1261 3703 5035 3706
f 701 1853 5035 3703
f 283 2662 5036 2660
f 1140 4581 5036 2662
f 1261 3706 5036 4581
f 702 2660 5036 3706
f 317 2797 5037 2798
f 1117 4536 5037 2797
f 1261 4581 5037 4536
f 1140 2798 5037 4581
f 272 2616 5038 2617
f 701 3703 5038 2616
f 1261 4536 5038 3703
f 1117 2617 5038 4536
f 71 1812 5039 1813
f 660 3621 5039 1812
f 1262 3624 5039 3621
f 661 1813 5039 3624
f 178 2241 5040 2240
f 929 4160 5040 2241
f 1262 3621 5040 4160
f 660 2240 5040 3621
f 317 2798 5041 2796
f 1140 4582 5041 2798
f 1262 4160 5041 4582
f 929 2796 5041 4160
f 283 2659 5042 2662
f 661 3624 5042 2659
f 1262 4582 5042 3624
f 1140 2662 5042 4582
f 16 1592 5043 1593
f 440 3181 5043 1592
f 1263 3184 5043 3181
f 441 1593 5043 3184
f 152 2137 5044 2135
f 877 4055 5044 2137
f 1263 3181 5044 4055
f 440 2135 5044 3181
f 318 2800 5045 2799
f 880 4061 5045 2800
f 1263 4055 5045 4061
f 877 2799 5045 4055
f 153 2139 5046 2142
f 441 3184 5046 2139
f 1263 4061 5046 3184
f 880 2142 5046 4061
f 48 1721 5047 1720
f 569 3440 5047 1721
f 1264 3437 5047 3440
f 568 1720 5047 3437
f 237 2478 5048 2475
f 1048 4397 5048 2478
f 1264 3440 5048 4397
f 569 2475 5048 3440
f 318 2799 5049 2801
f 877 4056 5049 2799
f 1264 4397 5049 4056
f 1048 2801 5049 4397
f 152 2136 5050 2137
f 568 3437 5050 2136
f 1264 4056 5050 3437
f 877 2137 5050 4056
f 81 1853 5051 1851
f 701 3704 5051 1853
f 1265 3700 5051 3704
f 699 1851 5051 3700
f 272 2618 5052 2616
f 1118 4537 5052 2618
f 1265 3704 5052 4537
f 701 2616 5052 3704
f 318 2801 5053 2802
f 1048 4398 5053 2801
f 1265 4537 5053 4398
f 1118 2802 5053 4537
f 237 2476 5054 2478
f 699 3700 5054 2476
f 1265 4398 5054 3700
f 1048 2478 5054 4398
f 65 1787 5055 1790
f 635 3572 5055 1787
f 1266 3578 5055 3572
f 638 1790 5055 3578
f 153 2142 5056 2140
f 880 4062 5056 2142
f 1266 3572 5056 4062
f 635 2140 5056 3572
f 318 2802 5057 2800
f 1118 4538 5057 2802
f 1266 4062 5057 4538
f 880 2800 5057 4062
f 272 2615 5058 2618
f 638 3578 5058 2615
f 1266 4538 5058 3578
f 1118 2618 5058 4538
f 6 1554 5059 1556
f 402 3105 5059 1554
f 1267 3109 5059 3105
f 404 1556 5059 3109
f 114 1985 5060 1983
f 801 3903 5060 1985
f 1267 3105 5060 3903
f 402 1983 5060 3105
f 319 2804 5061 2803
f 805 3911 5061 2804
f 1267 3903 5061 3911
f 801 2803 5061 3903
f 116 1991 5062 1993
f 404 3109 5062 1991
f 1267 3911 5062 3109
f 805 1993 5062 3911
f 42 1697 5063 1695
f 545 3391 5063 1697
f 1268 3387 5063 3391
f 543 1695 5063 3387
f 225 2429 5064 2427
f 1023 4347 5064 2429
f 1268 3391 5064 4347
f 545 2427 5064 3391
f 319 2803 5065 2805
f 801 3904 5065 2803
f 1268 4347 5065 3904
f 1023 2805 5065 4347
f 114 1984 5066 1985
f 543 3387 5066 1984
f 1268 3904 5066 3387
f 801 1985 5066 3904
f 82 1856 5067 1855
f 704 3709 5067 1856
f 1269 3707 5067 3709
f 703 1855 5067 3707
f 229 2445 5068 2444
f 1031 4363 5068 2445
f 1269 3709 5068 4363
f 704 2444 5068 3709
f 319 2805 5069 2806
f 1023 4348 5069 2805
f 1269 4363 5069 4348
f 1031 2806 5069 4363
f 225 2428 5070 2429
f 703 3707 5070 2428
f 1269 4348 5070 3707
f 1023 2429 5070 4348
f 44 1703 5071 1705
f 551 3403 5071 1703
f 1270 3407 5071 3403
f 553 1705 5071 3407
f 116 1993 5072 1992
f 805 3912 5072 1993
f 1270 3403 5072 3912
f 551 1992 5072 3403
f 319 2806 5073 2804
f 1031 4364 5073 2806
f 1270 3912 5073 4364
f 805 2804 5073 3912
f 229 2443 5074 2445
f 553 3407 5074 2443
f 1270 4364 5074 3407
f 1031 2445 5074 4364
f 13 1581 5075 1580
f 429 3160 5075 1581
f 1271 3157 5075 3160
f 428 1580 5075 3157
f 141 2094 5076 2091
f 856 4013 5076 2094
f 1271 3160 5076 4013
f 429 2091 5076 3160
f 320 2807 5077 2808
f 853 4007 5077 2807
f 1271 4013 5077 4007
f 856 2808 5077 4013
f 140 2087 5078 2089
f 428 3157 5078 2087
f 1271 4007 5078 3157
f 853 2089 5078 4007
f 59 1766 5079 1763
f 614 3529 5079 1766
f 1272 3524 5079 3529
f 611 1763 5079 3524
f 260 2569 5080 2567
f 1093 4487 5080 2569
f 1272 3529 5080 4487
f 614 2567 5080 3529
f 320 2808 5081 2810
f 856 4014 5081 2808
f 1272 4487 5081 4014
f 1093 2810 5081 4487
f 141 2092 5082 2094
f 611 3524 5082 2092
f 1272 4014 5082 3524
f 856 2094 5082 4014
f 82 1855 5083 1857
f 703 3708 5083 1855
f 1273 3711 5083 3708
f 705 1857 5083 3711
f 225 2430 5084 2428
f 1024 4349 5084 2430
f 1273 3708 5084 4349
f 703 2428 5084 3708
f 320 2810 5085 2809
f 1093 4488 5085 2810
f 1273 4349 5085 4488
f 1024 2809 5085 4349
f 260 2568 5086 2569
f 705 3711 5086 2568
f 1273 4488 5086 3711
f 1093 2569 5086 4488
f 42 1696 5087 1697
f 544 3389 5087 1696
f 1274 3392 5087 3389
f 545 1697 5087 3392
f 140 2089 5088 2088
f 853 4008 5088 2089
f 1274 3389 5088 4008
f 544 2088 5088 3389
f 320 2809 5089 2807
f 1024 4350 5089 2809
f 1274 4008 5089 4350
f 853 2807 5089 4008
f 225 2427 5090 2430
f 545 3392 5090 2427
f 1274 4350 5090 3392
f 1024 2430 5090 4350
f 22 1618 5091 1616
f 466 3234 5091 1618
f 1275 3230 5091 3234
f 464 1616 5091 3230
f 178 2242 5092 2239
f 930 4161 5092 2242
f 1275 3234 5092 4161
f 466 2239 5092 3234
f 321 2811 5093 2812
f 926 4153 5093 2811
f 1275 4161 5093 4153
f 930 2812 5093 4161
f 176 2231 5094 2234
f 464 3230 5094 2231
f 1275 4153 5094 3230
f 926 2234 5094 4153
f 71 1814 5095 1812
f 662 3625 5095 1814
f 1276 3622 5095 3625
f 660 1812 5095 3622
f 284 2665 5096 2663
f 1141 4583 5096 2665
f 1276 3625 5096 4583
f 662 2663 5096 3625
f 321 2812 5097 2814
f 930 4162 5097 2812
f 1276 4583 5097 4162
f 1141 2814 5097 4583
f 178 2240 5098 2242
f 660 3622 5098 2240
f 1276 4162 5098 3622
f 930 2242 5098 4162
f 82 1857 5099 1858
f 705 3712 5099 1857
f 1277 3713 5099 3712
f 706 1858 5099 3713
f 260 2570 5100 2568
f 1094 4489 5100 2570
f 1277 3712 5100 4489
f 705 2568 5100 3712
f 321 2814 5101 2813
f 1141 4584 5101 2814
f 1277 4489 5101 4584
f 1094 2813 5101 4489
f 284 2664 5102 2665
f 706 3713 5102 2664
f 1277 4584 5102 3713
f 1141 2665 5102 4584
f 59 1764 5103 1766
f 612 3526 5103 1764
f 1278 3530 5103 3526
f 614 1766 5103 3530
f 176 2234 5104 2232
f 926 4154 5104 2234
f 1278 3526 5104 4154
f 612 2232 5104 3526
f 321 2813 5105 2811
f 1094 4490 5105 2813
f 1278 4154 5105 4490
f 926 2811 5105 4154
f 260 2567 5106 2570
f 614 3530 5106 2567
f 1278 4490 5106 3530
f 1094 2570 5106 4490
f 19 1603 5107 1605
f 451 3203 5107 1603
f 1279 3208 5107 3203
f 453 1605 5107 3208
f 163 2181 5108 2179
f 899 4099 5108 2181
f 1279 3203 5108 4099
f 451 2179 5108 3203
f 322 2816 5109 2815
f 904 4109 5109 2816
f 1279 4099 5109 4109
f 899 2815 5109 4099
f 165 2187 5110 2190
f 453 3208 5110 2187
f 1279 4109 5110 3208
f 904 2190 5110 4109
f 44 1705 5111 1704
f 553 3408 5111 1705
f 1280 3405 5111 3408
f 552 1704 5111 3405
f 229 2446 5112 2443
f 1032 4365 5112 2446
f 1280 3408 5112 4365
f 553 2443 5112 3408
f 322 2815 5113 2817
f 899 4100 5113 2815
f 1280 4365 5113 4100
f 1032 2817 5113 4365
f 163 2180 5114 2181
f 552 3405 5114 2180
f 1280 4100 5114 3405
f 899 2181 5114 4100
f 82 1858 5115 1856
f 706 3714 5115 1858
f 1281 3710 5115 3714
f 704 1856 5115 3710
f 284 2666 5116 2664
f 1142 4585 5116 2666
f 1281 3714 5116 4585
f 706 2664 5116 3714
f 322 2817 5117 2818
f 1032 4366 5117 2817
f 1281 4585 5117 4366
f 1142 2818 5117 4585
f 229 2444 5118 2446
f 704 3710 5118 2444
f 1281 4366 5118 3710
f 1032 2446 5118 4366
f 71 1811 5119 1814
f 659 3620 5119 1811
f 1282 3626 5119 3620
f 662 1814 5119 3626
f 165 2190 5120 2188
f 904 4110 5120 2190
f 1282 3620 5120 4110
f 659 2188 5120 3620
f 322 2818 5121 2816
f 1142 4586 5121 2818
f 1282 4110 5121 4586
f 904 2816 5121 4110
f 284 2663 5122 2666
f 662 3626 5122 2663
f 1282 4586 5122 3626
f 1142 2666 5122 4586
f 1 1539 5123 1541
f 387 3075 5123 1539
f 1283 3080 5123 3075
f 389 1541 5123 3080
f 99 1925 5124 1923
f 771 3843 5124 1925
f 1283 3075 5124 3843
f 387 1923 5124 3075
f 323 2820 5125 2819
f 776 3853 5125 2820
f 1283 3843 5125 3853
f 771 2819 5125 3843
f 101 1931 5126 1934
f 389 3080 5126 1931
f 1283 3853 5126 3080
f 776 1934 5126 3853
f 27 1637 5127 1635
f 485 3271 5127 1637
f 1284 3267 5127 3271
f 483 1635 5127 3267
f 195 2309 5128 2307
f 963 4227 5128 2309
f 1284 3271 5128 4227
f 485 2307 5128 3271
f 323 2819 5129 2821
f 771 3844 5129 2819
f 1284 4227 5129 3844
f 963 2821 5129 4227
f 99 1924 5130 1925
f 483 3267 5130 1924
f 1284 3844 5130 3267
f 771 1925 5130 3844
f 83 1860 5131 1859
f 708 3717 5131 1860
f 1285 3715 5131 3717
f 707 1859 5131 3715
f 200 2329 5132 2328
f 973 4247 5132 2329
f 1285 3717 5132 4247
f 708 2328 5132 3717
f 323 2821 5133 2822
f 963 4228 5133 2821
f 1285 4247 5133 4228
f 973 2822 5133 4247
f 195 2308 5134 2309
f 707 3715 5134 2308
f 1285 4228 5134 3715
f 963 2309 5134 4228
f 29 1643 5135 1646
f 491 3284 5135 1643
f 1286 3289 5135 3284
f 494 1646 5135 3289
f 101 1934 5136 1932
f 776 3854 5136 1934
f 1286 3284 5136 3854
f 491 1932 5136 3284
f 323 2822 5137 2820
f 973 4248 5137 2822
f 1286 3854 5137 4248
f 776 2820 5137 3854
f 200 2327 5138 2329
f 494 3289 5138 2327
f 1286 4248 5138 3289
f 973 2329 5138 4248
f 9 1565 5139 1563
f 413 3127 5139 1565
f 1287 3123 5139 3127
f 411 1563 5139 3123
f 125 2029 5140 2027
f 823 3947 5140 2029
f 1287 3127 5140 3947
f 413 2027 5140 3127
f 324 2823 5141 2824
f 819 3939 5141 2823
f 1287 3947 5141 3939
f 823 2824 5141 3947
f 123 2019 5142 2021
f 411 3123 5142 2019
f 1287 3939 5142 3123
f 819 2021 5142 3939
f 51 1733 5143 1731
f 581 3463 5143 1733
f 1288 3459 5143 3463
f 579 1731 5143 3459
f 243 2501 5144 2499
f 1059 4419 5144 2501
f 1288 3463 5144 4419
f 581 2499 5144 3463
f 324 2824 5145 2826
f 823 3948 5145 2824
f 1288 4419 5145 3948
f 1059 2826 5145 4419
f 125 2028 5146 2029
f 579 3459 5146 2028
f 1288 3948 5146 3459
f 823 2029 5146 3948
f 83 1859 5147 1861
f 707 3716 5147 1859
f 1289 3719 5147 3716
f 709 1861 5147 3719
f 195 2310 5148 2308
f 964 4229 5148 2310
f 1289 3716 5148 4229
f 707 2308 5148 3716
f 324 2826 5149 2825
f 1059 4420 5149 2826
f 1289 4229 5149 4420
f 964 2825 5149 4229
f 243 2500 5150 2501
f 709 3719 5150 2500
f 1289 4420 5150 3719
f 1059 2501 5150 4420
f 27 1636 5151 1637
f 484 3269 5151 1636
f 1290 3272 5151 3269
f 485 1637 5151 3272
f 123 2021 5152 2020
f 819 3940 5152 2021
f 1290 3269 5152 3940
f 484 2020 5152 3269
f 324 2825 5153 2823
f 964 4230 5153 2825
f 1290 3940 5153 4230
f 819 2823 5153 3940
f 195 2307 5154 2310
f 485 3272 5154 2307
f 1290 4230 5154 3272
f 964 2310 5154 4230
f 23 1620 5155 1619
f 468 3237 5155 1620
f 1291 3235 5155 3237
f 467 1619 5155 3235
f 180 2249 5156 2247
f 933 4167 5156 2249
f 1291 3237 5156 4167
f 468 2247 5156 3237
f 325 2827 5157 2828
f 931 4163 5157 2827
f 1291 4167 5157 4163
f 933 2828 5157 4167
f 179 2243 5158 2245
f 467 3235 5158 2243
f 1291 4163 5158 3235
f 931 2245 5158 4163
f 56 1753 5159 1752
f 601 3503 5159 1753
f 1292 3501 5159 3503
f 600 1752 5159 3501
f 253 2541 5160 2539
f 1079 4459 5160 2541
f 1292 3503 5160 4459
f 601 2539 5160 3503
f 325 2828 5161 2830
f 933 4168 5161 2828
f 1292 4459 5161 4168
f 1079 2830 5161 4459
f 180 2248 5162 2249
f 600 3501 5162 2248
f 1292 4168 5162 3501
f 933 2249 5162 4168
f 83 1861 5163 1862
f 709 3720 5163 1861
f 1293 3721 5163 3720
f 710 1862 5163 3721
f 243 2502 5164 2500
f 1060 4421 5164 2502
f 1293 3720 5164 4421
f 709 2500 5164 3720
f 325 2830 5165 2829
f 1079 4460 5165 2830
f 1293 4421 5165 4460
f 1060 2829 5165 4421
f 253 2540 5166 2541
f 710 3721 5166 2540
f 1293 4460 5166 3721
f 1079 2541 5166 4460
f 51 1732 5167 1733
f 580 3461 5167 1732
f 1294 3464 5167 3461
f 581 1733 5167 3464
f 179 2245 5168 2244
f 931 4164 5168 2245
f 1294 3461 5168 4164
f 580 2244 5168 3461
f 325 2829 5169 2827
f 1060 4422 5169 2829
f 1294 4164 5169 4422
f 931 2827 5169 4164
f 243 2499 5170 2502
f 581 3464 5170 2499
f 1294 4422 5170 3464
f 1060 2502 5170 4422
f 11 1571 5171 1574
f 419 3140 5171 1571
f 1295 3145 5171 3140
f 422 1574 5171 3145
f 131 2054 5172 2051
f 836 3973 5172 2054
f 1295 3140 5172 3973
f 419 2051 5172 3140
f 326 2832 5173 2831
f 841 3983 5173 2832
f 1295 3973 5173 3983
f 836 2831 5173 3973
f 134 2063 5174 2065
f 422 3145 5174 2063
f 1295 3983 5174 3145
f 841 2065 5174 3983
f 29 1646 5175 1644
f 494 3290 5175 1646
f 1296 3286 5175 3290
f 492 1644 5175 3286
f 200 2330 5176 2327
f 974 4249 5176 2330
f 1296 3290 5176 4249
f 494 2327 5176 3290
f 326 2831 5177 2833
f 836 3974 5177 2831
f 1296 4249 5177 3974
f 974 2833 5177 4249
f 131 2052 5178 2054
f 492 3286 5178 2052
f 1296 3974 5178 3286
f 836 2054 5178 3974
f 83 1862 5179 1860
f 710 3722 5179 1862
f 1297 3718 5179 3722
f 708 1860 5179 3718
f 253 2542 5180 2540
f 1080 4461 5180 2542
f 1297 3722 5180 4461
f 710 2540 5180 3722
f 326 2833 5181 2834
f 974 4250 5181 2833
f 1297 4461 5181 4250
f 1080 2834 5181 4461
f 200 2328 5182 2330
f 708 3718 5182 2328
f 1297 4250 5182 3718
f 974 2330 5182 4250
f 56 1751 5183 1753
f 599 3499 5183 1751
f 1298 3504 5183 3499
f 601 1753 5183 3504
f 134 2065 5184 2064
f 841 3984 5184 2065
f 1298 3499 5184 3984
f 599 2064 5184 3499
f 326 2834 5185 2832
f 1080 4462 5185 2834
f 1298 3984 5185 4462
f 841 2832 5185 3984
f 253 2539 5186 2542
f 601 3504 5186 2539
f 1298 4462 5186 3504
f 1080 2542 5186 4462
f 2 1544 5187 1542
f 392 3086 5187 1544
f 1299 3081 5187 3086
f 390 1542 5187 3081
f 104 1946 5188 1943
f 782 3865 5188 1946
f 1299 3086 5188 3865
f 392 1943 5188 3086
f 327 2835 5189 2836
f 777 3855 5189 2835
f 1299 3865 5189 3855
f 782 2836 5189 3865
f 102 1935 5190 1937
f 390 3081 5190 1935
f 1299 3855 5190 3081
f 777 1937 5190 3855
f 32 1658 5191 1655
f 506 3313 5191 1658
f 1300 3308 5191 3313
f 503 1655 5191 3308
f 206 2353 5192 2351
f 985 4271 5192 2353
f 1300 3313 5192 4271
f 506 2351 5192 3313
f 327 2836 5193 2838
f 782 3866 5193 2836
f 1300 4271 5193 3866
f 985 2838 5193 4271
f 104 1944 5194 1946
f 503 3308 5194 1944
f 1300 3866 5194 3308
f 782 1946 5194 3866
f 84 1863 5195 1864
f 711 3723 5195 1863
f 1301 3725 5195 3723
f 712 1864 5195 3725
f 201 2333 5196 2332
f 975 4251 5196 2333
f 1301 3723 5196 4251
f 711 2332 5196 3723
f 327 2838 5197 2837
f 985 4272 5197 2838
f 1301 4251 5197 4272
f 975 2837 5197 4251
f 206 2352 5198 2353
f 712 3725 5198 2352
f 1301 4272 5198 3725
f 985 2353 5198 4272
f 30 1647 5199 1649
f 495 3291 5199 1647
f 1302 3295 5199 3291
f 497 1649 5199 3295
f 102 1937 5200 1936
f 777 3856 5200 1937
f 1302 3291 5200 3856
f 495 1936 5200 3291
f 327 2837 5201 2835
f 975 4252 5201 2837
f 1302 3856 5201 4252
f 777 2835 5201 3856
f 201 2331 5202 2333
f 497 3295 5202 2331
f 1302 4252 5202 3295
f 975 2333 5202 4252
f 13 1582 5203 1579
f 430 3161 5203 1582
f 1303 3156 5203 3161
f 427 1579 5203 3156
f 142 2097 5204 2095
f 857 4015 5204 2097
f 1303 3161 5204 4015
f 430 2095 5204 3161
f 328 2839 5205 2840
f 852 4005 5205 2839
f 1303 4015 5205 4005
f 857 2840 5205 4015
f 139 2083 5206 2086
f 427 3156 5206 2083
f 1303 4005 5206 3156
f 852 2086 5206 4005
f 60 1769 5207 1767
f 617 3535 5207 1769
f 1304 3531 5207 3535
f 615 1767 5207 3531
f 261 2573 5208 2571
f 1095 4491 5208 2573
f 1304 3535 5208 4491
f 617 2571 5208 3535
f 328 2840 5209 2842
f 857 4016 5209 2840
f 1304 4491 5209 4016
f 1095 2842 5209 4491
f 142 2096 5210 2097
f 615 3531 5210 2096
f 1304 4016 5210 3531
f 857 2097 5210 4016
f 84 1864 5211 1866
f 712 3726 5211 1864
f 1305 3729 5211 3726
f 714 1866 5211 3729
f 206 2354 5212 2352
f 986 4273 5212 2354
f 1305 3726 5212 4273
f 712 2352 5212 3726
f 328 2842 5213 2841
f 1095 4492 5213 2842
f 1305 4273 5213 4492
f 986 2841 5213 4273
f 261 2572 5214 2573
f 714 3729 5214 2572
f 1305 4492 5214 3729
f 1095 2573 5214 4492
f 32 1656 5215 1658
f 504 3310 5215 1656
f 1306 3314 5215 3310
f 506 1658 5215 3314
f 139 2086 5216 2084
f 852 4006 5216 2086
f 1306 3310 5216 4006
f 504 2084 5216 3310
f 328 2841 5217 2839
f 986 4274 5217 2841
f 1306 4006 5217 4274
f 852 2839 5217 4006
f 206 2351 5218 2354
f 506 3314 5218 2351
f 1306 4274 5218 3314
f 986 2354 5218 4274
f 23 1619 5219 1621
f 467 3236 5219 1619
f 1307 3239 5219 3236
f 469 1621 5219 3239
f 179 2246 5220 2243
f 932 4165 5220 2246
f 1307 3236 5220 4165
f 467 2243 5220 3236
f 329 2844 5221 2843
f 935 4171 5221 2844
f 1307 4165 5221 4171
f 932 2843 5221 4165
f 181 2251 5222 2253
f 469 3239 5222 2251
f 1307 4171 5222 3239
f 935 2253 5222 4171
f 51 1734 5223 1732
f 582 3465 5223 1734
f 1308 3462 5223 3465
f 580 1732 5223 3462
f 244 2505 5224 2503
f 1061 4423 5224 2505
f 1308 3465 5224 4423
f 582 2503 5224 3465
f 329 2843 5225 2845
f 932 4166 5225 2843
f 1308 4423 5225 4166
f 1061 2845 5225 4423
f 179 2244 5226 2246
f 580 3462 5226 2244
f 1308 4166 5226 3462
f 932 2246 5226 4166
f 84 1866 5227 1865
f 714 3730 5227 1866
f 1309 3727 5227 3730
f 713 1865 5227 3727
f 261 2574 5228 2572
f 1096 4493 5228 2574
f 1309 3730 5228 4493
f 714 2572 5228 3730
f 329 2845 5229 2846
f 1061 4424 5229 2845
f 1309 4493 5229 4424
f 1096 2846 5229 4493
f 244 2504 5230 2505
f 713 3727 5230 2504
f 1309 4424 5230 3727
f 1061 2505 5230 4424
f 60 1768 5231 1769
f 616 3533 5231 1768
f 1310 3536 5231 3533
f 617 1769 5231 3536
f 181 2253 5232 2252
f 935 4172 5232 2253
f 1310 3533 5232 4172
f 616 2252 5232 3533
f 329 2846 5233 2844
f 1096 4494 5233 2846
f 1310 4172 5233 4494
f 935 2844 5233 4172
f 261 2571 5234 2574
f 617 3536 5234 2571
f 1310 4494 5234 3536
f 1096 2574 5234 4494
f 9 1564 5235 1565
f 412 3125 5235 1564
f 1311 3128 5235 3125
f 413 1565 5235 3128
f 124 2025 5236 2023
f 821 3943 5236 2025
f 1311 3125 5236 3943
f 412 2023 5236 3125
f 330 2848 5237 2847
f 824 3949 5237 2848
f 1311 3943 5237 3949
f 821 2847 5237 3943
f 125 2027 5238 2030
f 413 3128 5238 2027
f 1311 3949 5238 3128
f 824 2030 5238 3949
f 30 1649 5239 1648
f 497 3296 5239 1649
f 1312 3293 5239 3296
f 496 1648 5239 3293
f 201 2334 5240 2331
f 976 4253 5240 2334
f 1312 3296 5240 4253
f 497 2331 5240 3296
f 330 2847 5241 2849
f 821 3944 5241 2847
f 1312 4253 5241 3944
f 976 2849 5241 4253
f 124 2024 5242 2025
f 496 3293 5242 2024
f 1312 3944 5242 3293
f 821 2025 5242 3944
f 84 1865 5243 1863
f 713 3728 5243 1865
f 1313 3724 5243 3728
f 711 1863 5243 3724
f 244 2506 5244 2504
f 1062 4425 5244 2506
f 1313 3728 5244 4425
f 713 2504 5244 3728
f 330 2849 5245 2850
f 976 4254 5245 2849
f 1313 4425 5245 4254
f 1062 2850 5245 4425
f 201 2332 5246 2334
f 711 3724 5246 2332
f 1313 4254 5246 3724
f 976 2334 5246 4254
f 51 1731 5247 1734
f 579 3460 5247 1731
f 1314 3466 5247 3460
f 582 1734 5247 3466
f 125 2030 5248 2028
f 824 3950 5248 2030
f 1314 3460 5248 3950
f 579 2028 5248 3460
f 330 2850 5249 2848
f 1062 4426 5249 2850
f 1314 3950 5249 4426
f 824 2848 5249 3950
f 244 2503 5250 2506
f 582 3466 5250 2503
f 1314 4426 5250 3466
f 1062 2506 5250 4426
f 6 1555 5251 1554
f 403 3107 5251 1555
f 1315 3106 5251 3107
f 402 1554 5251 3106
f 115 1989 5252 1987
f 803 3907 5252 1989
f 1315 3107 5252 3907
f 403 1987 5252 3107
f 331 2851 5253 2852
f 802 3905 5253 2851
f 1315 3907 5253 3905
f 803 2852 5253 3907
f 114 1983 5254 1986
f 402 3106 5254 1983
f 1315 3905 5254 3106
f 802 1986 5254 3905
f 43 1701 5255 1699
f 549 3399 5255 1701
f 1316 3395 5255 3399
f 547 1699 5255 3395
f 227 2437 5256 2435
f 1027 4355 5256 2437
f 1316 3399 5256 4355
f 549 2435 5256 3399
f 331 2852 5257 2854
f 803 3908 5257 2852
f 1316 4355 5257 3908
f 1027 2854 5257 4355
f 115 1988 5258 1989
f 547 3395 5258 1988
f 1316 3908 5258 3395
f 803 1989 5258 3908
f 85 1867 5259 1868
f 715 3731 5259 1867
f 1317 3733 5259 3731
f 716 1868 5259 3733
f 226 2433 5260 2432
f 1025 4351 5260 2433
f 1317 3731 5260 4351
f 715 2432 5260 3731
f 331 2854 5261 2853
f 1027 4356 5261 2854
f 1317 4351 5261 4356
f 1025 2853 5261 4351
f 227 2436 5262 2437
f 716 3733 5262 2436
f 1317 4356 5262 3733
f 1027 2437 5262 4356
f 42 1695 5263 1698
f 543 3388 5263 1695
f 1318 3393 5263 3388
f 546 1698 5263 3393
f 114 1986 5264 1984
f 802 3906 5264 1986
f 1318 3388 5264 3906
f 543 1984 5264 3388
f 331 2853 5265 2851
f 1025 4352 5265 2853
f 1318 3906 5265 4352
f 802 2851 5265 3906
f 226 2431 5266 2433
f 546 3393 5266 2431
f 1318 4352 5266 3393
f 1025 2433 5266 4352
f 17 1597 5267 1596
f 445 3191 5267 1597
f 1319 3189 5267 3191
f 444 1596 5267 3189
f 157 2157 5268 2155
f 887 4075 5268 2157
f 1319 3191 5268 4075
f 445 2155 5268 3191
f 332 2855 5269 2856
f 885 4071 5269 2855
f 1319 4075 5269 4071
f 887 2856 5269 4075
f 156 2151 5270 2153
f 444 3189 5270 2151
f 1319 4071 5270 3189
f 885 2153 5270 4071
f 67 1797 5271 1795
f 645 3591 5271 1797
f 1320 3587 5271 3591
f 643 1795 5271 3587
f 275 2629 5272 2627
f 1123 4547 5272 2629
f 1320 3591 5272 4547
f 645 2627 5272 3591
f 332 2856 5273 2858
f 887 4076 5273 2856
f 1320 4547 5273 4076
f 1123 2858 5273 4547
f 157 2156 5274 2157
f 643 3587 5274 2156
f 1320 4076 5274 3587
f 887 2157 5274 4076
f 85 1868 5275 1870
f 716 3734 5275 1868
f 1321 3737 5275 3734
f 718 1870 5275 3737
f 227 2438 5276 2436
f 1028 4357 5276 2438
f 1321 3734 5276 4357
f 716 2436 5276 3734
f 332 2858 5277 2857
f 1123 4548 5277 2858
f 1321 4357 5277 4548
f 1028 2857 5277 4357
f 275 2628 5278 2629
f 718 3737 5278 2628
f 1321 4548 5278 3737
f 1123 2629 5278 4548
f 43 1700 5279 1701
f 548 3397 5279 1700
f 1322 3400 5279 3397
f 549 1701 5279 3400
f 156 2153 5280 2152
f 885 4072 5280 2153
f 1322 3397 5280 4072
f 548 2152 5280 3397
f 332 2857 5281 2855
f 1028 4358 5281 2857
f 1322 4072 5281 4358
f 885 2855 5281 4072
f 227 2435 5282 2438
f 549 3400 5282 2435
f 1322 4358 5282 3400
f 1028 2438 5282 4358
f 23 1621 5283 1622
f 469 3240 5283 1621
f 1323 3241 5283 3240
f 470 1622 5283 3241
f 181 2254 5284 2251
f 936 4173 5284 2254
f 1323 3240 5284 4173
f 469 2251 5284 3240
f 333 2860 5285 2859
f 937 4175 5285 2860
f 1323 4173 5285 4175
f 936 2859 5285 4173
f 182 2255 5286 2257
f 470 3241 5286 2255
f 1323 4175 5286 3241
f 937 2257 5286 4175
f 60 1770 5287 1768
f 618 3537 5287 1770
f 1324 3534 5287 3537
f 616 1768 5287 3534
f 262 2577 5288 2575
f 1097 4495 5288 2577
f 1324 3537 5288 4495
f 618 2575 5288 3537
f 333 2859 5289 2861
f 936 4174 5289 2859
f 1324 4495 5289 4174
f 1097 2861 5289 4495
f 181 2252 5290 2254
f 616 3534 5290 2252
f 1324 4174 5290 3534
f 936 2254 5290 4174
f 85 1870 5291 1869
f 718 3738 5291 1870
f 1325 3735 5291 3738
f 717 1869 5291 3735
f 275 2630 5292 2628
f 1124 4549 5292 2630
f 1325 3738 5292 4549
f 718 2628 5292 3738
f 333 2861 5293 2862
f 1097 4496 5293 2861
f 1325 4549 5293 4496
f 1124 2862 5293 4549
f 262 2576 5294 2577
f 717 3735 5294 2576
f 1325 4496 5294 3735
f 1097 2577 5294 4496
f 67 1796 5295 1797
f 644 3589 5295 1796
f 1326 3592 5295 3589
f 645 1797 5295 3592
f 182 2257 5296 2256
f 937 4176 5296 2257
f 1326 3589 5296 4176
f 644 2256 5296 3589
f 333 2862 5297 2860
f 1124 4550 5297 2862
f 1326 4176 5297 4550
f 937 2860 5297 4176
f 275 2627 5298 2630
f 645 3592 5298 2627
f 1326 4550 5298 3592
f 1124 2630 5298 4550
f 13 1580 5299 1582
f 428 3158 5299 1580
f 1327 3162 5299 3158
f 430 1582 5299 3162
f 140 2090 5300 2087
f 854 4009 5300 2090
f 1327 3158 5300 4009
f 428 2087 5300 3158
f 334 2864 5301 2863
f 858 4017 5301 2864
f 1327 4009 5301 4017
f 854 2863 5301 4009
f 142 2095 5302 2098
f 430 3162 5302 2095
f 1327 4017 5302 3162
f 858 2098 5302 4017
f 42 1698 5303 1696
f 546 3394 5303 1698
f 1328 3390 5303 3394
f 544 1696 5303 3390
f 226 2434 5304 2431
f 1026 4353 5304 2434
f 1328 3394 5304 4353
f 546 2431 5304 3394
f 334 2863 5305 2865
f 854 4010 5305 2863
f 1328 4353 5305 4010
f 1026 2865 5305 4353
f 140 2088 5306 2090
f 544 3390 5306 2088
f 1328 4010 5306 3390
f 854 2090 5306 4010
f 85 1869 5307 1867
f 717 3736 5307 1869
f 1329 3732 5307 3736
f 715 1867 5307 3732
f 262 2578 5308 2576
f 1098 4497 5308 2578
f 1329 3736 5308 4497
f 717 2576 5308 3736
f 334 2865 5309 2866
f 1026 4354 5309 2865
f 1329 4497 5309 4354
f 1098 2866 5309 4497
f 226 2432 5310 2434
f 715 3732 5310 2432
f 1329 4354 5310 3732
f 1026 2434 5310 4354
f 60 1767 5311 1770
f 615 3532 5311 1767
f 1330 3538 5311 3532
f 618 1770 5311 3538
f 142 2098 5312 2096
f 858 4018 5312 2098
f 1330 3532 5312 4018
f 615 2096 5312 3532
f 334 2866 5313 2864
f 1098 4498 5313 2866
f 1330 4018 5313 4498
f 858 2864 5313 4018
f 262 2575 5314 2578
f 618 3538 5314 2575
f 1330 4498 5314 3538
f 1098 2578 5314 4498
f 5 1551 5315 1552
f 399 3100 5315 1551
f 1331 3101 5315 3100
f 400 1552 5315 3101
f 111 1974 5316 1971
f 796 3893 5316 1974
f 1331 3100 5316 3893
f 399 1971 5316 3100
f 335 2868 5317 2867
f 797 3895 5317 2868
f 1331 3893 5317 3895
f 796 2867 5317 3893
f 112 1975 5318 1977
f 400 3101 5318 1975
f 1331 3895 5318 3101
f 797 1977 5318 3895
f 39 1686 5319 1683
f 534 3369 5319 1686
f 1332 3364 5319 3369
f 531 1683 5319 3364
f 220 2409 5320 2407
f 1013 4327 5320 2409
f 1332 3369 5320 4327
f 534 2407 5320 3369
f 335 2867 5321 2869
f 796 3894 5321 2867
f 1332 4327 5321 3894
f 1013 2869 5321 4327
f 111 1972 5322 1974
f 531 3364 5322 1972
f 1332 3894 5322 3364
f 796 1974 5322 3894
f 86 1872 5323 1871
f 720 3741 5323 1872
f 1333 3739 5323 3741
f 719 1871 5323 3739
f 221 2413 5324 2412
f 1015 4331 5324 2413
f 1333 3741 5324 4331
f 720 2412 5324 3741
f 335 2869 5325 2870
f 1013 4328 5325 2869
f 1333 4331 5325 4328
f 1015 2870 5325 4331
f 220 2408 5326 2409
f 719 3739 5326 2408
f 1333 4328 5326 3739
f 1013 2409 5326 4328
f 40 1687 5327 1689
f 535 3371 5327 1687
f 1334 3375 5327 3371
f 537 1689 5327 3375
f 112 1977 5328 1976
f 797 3896 5328 1977
f 1334 3371 5328 3896
f 535 1976 5328 3371
f 335 2870 5329 2868
f 1015 4332 5329 2870
f 1334 3896 5329 4332
f 797 2868 5329 3896
f 221 2411 5330 2413
f 537 3375 5330 2411
f 1334 4332 5330 3375
f 1015 2413 5330 4332
f 11 1574 5331 1572
f 422 3146 5331 1574
f 1335 3142 5331 3146
f 420 1572 5331 3142
f 134 2066 5332 2063
f 842 3985 5332 2066
f 1335 3146 5332 3985
f 422 2063 5332 3146
f 336 2871 5333 2872
f 838 3977 5333 2871
f 1335 3985 5333 3977
f 842 2872 5333 3985
f 132 2055 5334 2058
f 420 3142 5334 2055
f 1335 3977 5334 3142
f 838 2058 5334 3977
f 56 1754 5335 1751
f 602 3505 5335 1754
f 1336 3500 5335 3505
f 599 1751 5335 3500
f 254 2545 5336 2543
f 1081 4463 5336 2545
f 1336 3505 5336 4463
f 602 2543 5336 3505
f 336 2872 5337 2874
f 842 3986 5337 2872
f 1336 4463 5337 3986
f 1081 2874 5337 4463
f 134 2064 5338 2066
f 599 3500 5338 2064
f 1336 3986 5338 3500
f 842 2066 5338 3986
f 86 1871 5339 1873
f 719 3740 5339 1871
f 1337 3743 5339 3740
f 721 1873 5339 3743
f 220 2410 5340 2408
f 1014 4329 5340 2410
f 1337 3740 5340 4329
f 719 2408 5340 3740
f 336 2874 5341 2873
f 1081 4464 5341 2874
f 1337 4329 5341 4464
f 1014 2873 5341 4329
f 254 2544 5342 2545
f 721 3743 5342 2544
f 1337 4464 5342 3743
f 1081 2545 5342 4464
f 39 1684 5343 1686
f 532 3366 5343 1684
f 1338 3370 5343 3366
f 534 1686 5343 3370
f 132 2058 5344 2056
f 838 3978 5344 2058
f 1338 3366 5344 3978
f 532 2056 5344 3366
f 336 2873 5345 2871
f 1014 4330 5345 2873
f 1338 3978 5345 4330
f 838 2871 5345 3978
f 220 2407 5346 2410
f 534 3370 5346 2407
f 1338 4330 5346 3370
f 1014 2410 5346 4330
f 23 1622 5347 1620
f 470 3242 5347 1622
f 1339 3238 5347 3242
f 468 1620 5347 3238
f 182 2258 5348 2255
f 938 4177 5348 2258
f 1339 3242 5348 4177
f 470 2255 5348 3242
f 337 2875 5349 2876
f 934 4169 5349 2875
f 1339 4177 5349 4169
f 938 2876 5349 4177
f 180 2247 5350 2250
f 468 3238 5350 2247
f 1339 4169 5350 3238
f 934 2250 5350 4169
f 67 1798 5351 1796
f 646 3593 5351 1798
f 1340 3590 5351 3593
f 644 1796 5351 3590
f 276 2633 5352 2631
f 1125 4551 5352 2633
f 1340 3593 5352 4551
f 646 2631 5352 3593
f 337 2876 5353 2878
f 938 4178 5353 2876
f 1340 4551 5353 4178
f 1125 2878 5353 4551
f 182 2256 5354 2258
f 644 3590 5354 2256
f 1340 4178 5354 3590
f 938 2258 5354 4178
f 86 1873 5355 1874
f 721 3744 5355 1873
f 1341 3745 5355 3744
f 722 1874 5355 3745
f 254 2546 5356 2544
f 1082 4465 5356 2546
f 1341 3744 5356 4465
f 721 2544 5356 3744
f 337 2878 5357 2877
f 1125 4552 5357 2878
f 1341 4465 5357 4552
f 1082 2877 5357 4465
f 276 2632 5358 2633
f 722 3745 5358 2632
f 1341 4552 5358 3745
f 1125 2633 5358 4552
f 56 1752 5359 1754
f 600 3502 5359 1752
f 1342 3506 5359 3502
f 602 1754 5359 3506
f 180 2250 5360 2248
f 934 4170 5360 2250
f 1342 3502 5360 4170
f 600 2248 5360 3502
f 337 2877 5361 2875
f 1082 4466 5361 2877
f 1342 4170 5361 4466
f 934 2875 5361 4170
f 254 2543 5362 2546
f 602 3506 5362 2543
f 1342 4466 5362 3506
f 1082 2546 5362 4466
f 17 1595 5363 1597
f 443 3187 5363 1595
f 1343 3192 5363 3187
f 445 1597 5363 3192
f 155 2149 5364 2147
f 883 4067 5364 2149
f 1343 3187 5364 4067
f 443 2147 5364 3187
f 338 2880 5365 2879
f 888 4077 5365 2880
f 1343 4067 5365 4077
f 883 2879 5365 4067
f 157 2155 5366 2158
f 445 3192 5366 2155
f 1343 4077 5366 3192
f 888 2158 5366 4077
f 40 1689 5367 1688
f 537 3376 5367 1689
f 1344 3373 5367 3376
f 536 1688 5367 3373
f 221 2414 5368 2411
f 1016 4333 5368 2414
f 1344 3376 5368 4333
f 537 2411 5368 3376
f 338 2879 5369 2881
f 883 4068 5369 2879
f 1344 4333 5369 4068
f 1016 2881 5369 4333
f 155 2148 5370 2149
f 536 3373 5370 2148
f 1344 4068 5370 3373
f 883 2149 5370 4068
f 86 1874 5371 1872
f 722 3746 5371 1874
f 1345 3742 5371 3746
f 720 1872 5371 3742
f 276 2634 5372 2632
f 1126 4553 5372 2634
f 1345 3746 5372 4553
f 722 2632 5372 3746
f 338 2881 5373 2882
f 1016 4334 5373 2881
f 1345 4553 5373 4334
f 1126 2882 5373 4553
f 221 2412 5374 2414
f 720 3742 5374 2412
f 1345 4334 5374 3742
f 1016 2414 5374 4334
f 67 1795 5375 1798
f 643 3588 5375 1795
f 1346 3594 5375 3588
f 646 1798 5375 3594
f 157 2158 5376 2156
f 888 4078 5376 2158
f 1346 3588 5376 4078
f 643 2156 5376 3588
f 338 2882 5377 2880
f 1126 4554 5377 2882
f 1346 4078 5377 4554
f 888 2880 5377 4078
f 276 2631 5378 2634
f 646 3594 5378 2631
f 1346 4554 5378 3594
f 1126 2634 5378 4554
f 3 1547 5379 1546
f 395 3092 5379 1547
f 1347 3089 5379 3092
f 394 1546 5379 3089
f 107 1958 5380 1955
f 788 3877 5380 1958
f 1347 3092 5380 3877
f 395 1955 5380 3092
f 339 2883 5381 2884
f 785 3871 5381 2883
f 1347 3877 5381 3871
f 788 2884 5381 3877
f 106 1951 5382 1953
f 394 3089 5382 1951
f 1347 3871 5382 3089
f 785 1953 5382 3871
f 35 1670 5383 1667
f 518 3337 5383 1670
f 1348 3332 5383 3337
f 515 1667 5383 3332
f 212 2377 5384 2375
f 997 4295 5384 2377
f 1348 3337 5384 4295
f 518 2375 5384 3337
f 339 2884 5385 2886
f 788 3878 5385 2884
f 1348 4295 5385 3878
f 997 2886 5385 4295
f 107 1956 5386 1958
f 515 3332 5386 1956
f 1348 3878 5386 3332
f 788 1958 5386 3878
f 87 1875 5387 1876
f 723 3747 5387 1875
f 1349 3749 5387 3747
f 724 1876 5387 3749
f 209 2365 5388 2364
f 991 4283 5388 2365
f 1349 3747 5388 4283
f 723 2364 5388 3747
f 339 2886 5389 2885
f 997 4296 5389 2886
f 1349 4283 5389 4296
f 991 2885 5389 4283
f 212 2376 5390 2377
f 724 3749 5390 2376
f 1349 4296 5390 3749
f 997 2377 5390 4296
f 34 1663 5391 1665
f 511 3323 5391 1663
f 1350 3327 5391 3323
f 513 1665 5391 3327
f 106 1953 5392 1952
f 785 3872 5392 1953
f 1350 3323 5392 3872
f 511 1952 5392 3323
f 339 2885 5393 2883
f 991 4284 5393 2885
f 1350 3872 5393 4284
f 785 2883 5393 3872
f 209 2363 5394 2365
f 513 3327 5394 2363
f 1350 4284 5394 3327
f 991 2365 5394 4284
f 15 1590 5395 1587
f 438 3177 5395 1590
f 1351 3172 5395 3177
f 435 1587 5395 3172
f 150 2129 5396 2127
f 873 4047 5396 2129
f 1351 3177 5396 4047
f 438 2127 5396 3177
f 340 2887 5397 2888
f 868 4037 5397 2887
f 1351 4047 5397 4037
f 873 2888 5397 4047
f 147 2115 5398 2118
f 435 3172 5398 2115
f 1351 4037 5398 3172
f 868 2118 5398 4037
f 64 1785 5399 1783
f 633 3567 5399 1785
f 1352 3563 5399 3567
f 631 1783 5399 3563
f 269 2605 5400 2603
f 1111 4523 5400 2605
f 1352 3567 5400 4523
f 633 2603 5400 3567
f 340 2888 5401 2890
f 873 4048 5401 2888
f 1352 4523 5401 4048
f 1111 2890 5401 4523
f 150 2128 5402 2129
f 631 3563 5402 2128
f 1352 4048 5402 3563
f 873 2129 5402 4048
f 87 1876 5403 1878
f 724 3750 5403 1876
f 1353 3753 5403 3750
f 726 1878 5403 3753
f 212 2378 5404 2376
f 998 4297 5404 2378
f 1353 3750 5404 4297
f 724 2376 5404 3750
f 340 2890 5405 2889
f 1111 4524 5405 2890
f 1353 4297 5405 4524
f 998 2889 5405 4297
f 269 2604 5406 2605
f 726 3753 5406 2604
f 1353 4524 5406 3753
f 1111 2605 5406 4524
f 35 1668 5407 1670
f 516 3334 5407 1668
f 1354 3338 5407 3334
f 518 1670 5407 3338
f 147 2118 5408 2116
f 868 4038 5408 2118
f 1354 3334 5408 4038
f 516 2116 5408 3334
f 340 2889 5409 2887
f 998 4298 5409 2889
f 1354 4038 5409 4298
f 868 2887 5409 4038
f 212 2375 5410 2378
f 518 3338 5410 2375
f 1354 4298 5410 3338
f 998 2378 5410 4298
f 24 1623 5411 1624
f 471 3243 5411 1623
f 1355 3245 5411 3243
f 472 1624 5411 3245
f 183 2261 5412 2259
f 939 4179 5412 2261
f 1355 3243 5412 4179
f 471 2259 5412 3243
f 341 2892 5413 2891
f 941 4183 5413 2892
f 1355 4179 5413 4183
f 939 2891 5413 4179
f 184 2263 5414 2265
f 472 3245 5414 2263
f 1355 4183 5414 3245
f 941 2265 5414 4183
f 61 1773 5415 1772
f 621 3543 5415 1773
f 1356 3541 5415 3543
f 620 1772 5415 3541
f 263 2581 5416 2579
f 1099 4499 5416 2581
f 1356 3543 5416 4499
f 621 2579 5416 3543
f 341 2891 5417 2893
f 939 4180 5417 2891
f 1356 4499 5417 4180
f 1099 2893 5417 4499
f 183 2260 5418 2261
f 620 3541 5418 2260
f 1356 4180 5418 3541
f 939 2261 5418 4180
f 87 1878 5419 1877
f 726 3754 5419 1878
f 1357 3751 5419 3754
f 725 1877 5419 3751
f 269 2606 5420 2604
f 1112 4525 5420 2606
f 1357 3754 5420 4525
f 726 2604 5420 3754
f 341 2893 5421 2894
f 1099 4500 5421 2893
f 1357 4525 5421 4500
f 1112 2894 5421 4525
f 263 2580 5422 2581
f 725 3751 5422 2580
f 1357 4500 5422 3751
f 1099 2581 5422 4500
f 64 1784 5423 1785
f 632 3565 5423 1784
f 1358 3568 5423 3565
f 633 1785 5423 3568
f 184 2265 5424 2264
f 941 4184 5424 2265
f 1358 3565 5424 4184
f 632 2264 5424 3565
f 341 2894 5425 2892
f 1112 4526 5425 2894
f 1358 4184 5425 4526
f 941 2892 5425 4184
f 269 2603 5426 2606
f 633 3568 5426 2603
f 1358 4526 5426 3568
f 1112 2606 5426 4526
f 14 1583 5427 1585
f 431 3163 5427 1583
f 1359 3167 5427 3163
f 433 1585 5427 3167
f 143 2101 5428 2099
f 859 4019 5428 2101
f 1359 3163 5428 4019
f 431 2099 5428 3163
f 342 2896 5429 2895
f 863 4027 5429 2896
f 1359 4019 5429 4027
f 859 2895 5429 4019
f 145 2107 5430 2109
f 433 3167 5430 2107
f 1359 4027 5430 3167
f 863 2109 5430 4027
f 34 1665 5431 1664
f 513 3328 5431 1665
f 1360 3325 5431 3328
f 512 1664 5431 3325
f 209 2366 5432 2363
f 992 4285 5432 2366
f 1360 3328 5432 4285
f 513 2363 5432 3328
f 342 2895 5433 2897
f 859 4020 5433 2895
f 1360 4285 5433 4020
f 992 2897 5433 4285
f 143 2100 5434 2101
f 512 3325 5434 2100
f 1360 4020 5434 3325
f 859 2101 5434 4020
f 87 1877 5435 1875
f 725 3752 5435 1877
f 1361 3748 5435 3752
f 723 1875 5435 3748
f 263 2582 5436 2580
f 1100 4501 5436 2582
f 1361 3752 5436 4501
f 725 2580 5436 3752
f 342 2897 5437 2898
f 992 4286 5437 2897
f 1361 4501 5437 4286
f 1100 2898 5437 4501
f 209 2364 5438 2366
f 723 3748 5438 2364
f 1361 4286 5438 3748
f 992 2366 5438 4286
f 61 1771 5439 1773
f 619 3539 5439 1771
f 1362 3544 5439 3539
f 621 1773 5439 3544
f 145 2109 5440 2108
f 863 4028 5440 2109
f 1362 3539 5440 4028
f 619 2108 5440 3539
f 342 2898 5441 2896
f 1100 4502 5441 2898
f 1362 4028 5441 4502
f 863 2896 5441 4028
f 263 2579 5442 2582
f 621 3544 5442 2579
f 1362 4502 5442 3544
f 1100 2582 5442 4502
f 7 1559 5443 1557
f 407 3115 5443 1559
f 1363 3112 5443 3115
f 405 1557 5443 3112
f 119 2005 5444 2003
f 811 3923 5444 2005
f 1363 3115 5444 3923
f 407 2003 5444 3115
f 343 2899 5445 2900
f 808 3917 5445 2899
f 1363 3923 5445 3917
f 811 2900 5445 3923
f 117 1995 5446 1998
f 405 3112 5446 1995
f 1363 3917 5446 3112
f 808 1998 5446 3917
f 47 1717 5447 1715
f 565 3431 5447 1717
f 1364 3427 5447 3431
f 563 1715 5447 3427
f 235 2469 5448 2467
f 1043 4387 5448 2469
f 1364 3431 5448 4387
f 565 2467 5448 3431
f 343 2900 5449 2902
f 811 3924 5449 2900
f 1364 4387 5449 3924
f 1043 2902 5449 4387
f 119 2004 5450 2005
f 563 3427 5450 2004
f 1364 3924 5450 3427
f 811 2005 5450 3924
f 88 1879 5451 1880
f 727 3755 5451 1879
f 1365 3757 5451 3755
f 728 1880 5451 3757
f 232 2457 5452 2456
f 1037 4375 5452 2457
f 1365 3755 5452 4375
f 727 2456 5452 3755
f 343 2902 5453 2901
f 1043 4388 5453 2902
f 1365 4375 5453 4388
f 1037 2901 5453 4375
f 235 2468 5454 2469
f 728 3757 5454 2468
f 1365 4388 5454 3757
f 1043 2469 5454 4388
f 45 1707 5455 1710
f 555 3412 5455 1707
f 1366 3417 5455 3412
f 558 1710 5455 3417
f 117 1998 5456 1996
f 808 3918 5456 1998
f 1366 3412 5456 3918
f 555 1996 5456 3412
f 343 2901 5457 2899
f 1037 4376 5457 2901
f 1366 3918 5457 4376
f 808 2899 5457 3918
f 232 2455 5458 2457
f 558 3417 5458 2455
f 1366 4376 5458 3417
f 1037 2457 5458 4376
f 20 1609 5459 1607
f 457 3215 5459 1609
f 1367 3211 5459 3215
f 455 1607 5459 3211
f 169 2205 5460 2203
f 911 4123 5460 2205
f 1367 3215 5460 4123
f 457 2203 5460 3215
f 344 2903 5461 2904
f 907 4115 5461 2903
f 1367 4123 5461 4115
f 911 2904 5461 4123
f 167 2195 5462 2197
f 455 3211 5462 2195
f 1367 4115 5462 3211
f 907 2197 5462 4115
f 73 1821 5463 1819
f 669 3639 5463 1821
f 1368 3635 5463 3639
f 667 1819 5463 3635
f 287 2677 5464 2675
f 1147 4595 5464 2677
f 1368 3639 5464 4595
f 669 2675 5464 3639
f 344 2904 5465 2906
f 911 4124 5465 2904
f 1368 4595 5465 4124
f 1147 2906 5465 4595
f 169 2204 5466 2205
f 667 3635 5466 2204
f 1368 4124 5466 3635
f 911 2205 5466 4124
f 88 1880 5467 1882
f 728 3758 5467 1880
f 1369 3761 5467 3758
f 730 1882 5467 3761
f 235 2470 5468 2468
f 1044 4389 5468 2470
f 1369 3758 5468 4389
f 728 2468 5468 3758
f 344 2906 5469 2905
f 1147 4596 5469 2906
f 1369 4389 5469 4596
f 1044 2905 5469 4389
f 287 2676 5470 2677
f 730 3761 5470 2676
f 1369 4596 5470 3761
f 1147 2677 5470 4596
f 47 1716 5471 1717
f 564 3429 5471 1716
f 1370 3432 5471 3429
f 565 1717 5471 3432
f 167 2197 5472 2196
f 907 4116 5472 2197
f 1370 3429 5472 4116
f 564 2196 5472 3429
f 344 2905 5473 2903
f 1044 4390 5473 2905
f 1370 4116 5473 4390
f 907 2903 5473 4116
f 235 2467 5474 2470
f 565 3432 5474 2467
f 1370 4390 5474 3432
f 1044 2470 5474 4390
f 24 1624 5475 1626
f 472 3246 5475 1624
f 1371 3249 5475 3246
f 474 1626 5475 3249
f 184 2266 5476 2263
f 942 4185 5476 2266
f 1371 3246 5476 4185
f 472 2263 5476 3246
f 345 2908 5477 2907
f 945 4191 5477 2908
f 1371 4185 5477 4191
f 942 2907 5477 4185
f 186 2271 5478 2273
f 474 3249 5478 2271
f 1371 4191 5478 3249
f 945 2273 5478 4191
f 64 1786 5479 1784
f 634 3569 5479 1786
f 1372 3566 5479 3569
f 632 1784 5479 3566
f 270 2609 5480 2607
f 1113 4527 5480 2609
f 1372 3569 5480 4527
f 634 2607 5480 3569
f 345 2907 5481 2909
f 942 4186 5481 2907
f 1372 4527 5481 4186
f 1113 2909 5481 4527
f 184 2264 5482 2266
f 632 3566 5482 2264
f 1372 4186 5482 3566
f 942 2266 5482 4186
f 88 1882 5483 1881
f 730 3762 5483 1882
f 1373 3759 5483 3762
f 729 1881 5483 3759
f 287 2678 5484 2676
f 1148 4597 5484 2678
f 1373 3762 5484 4597
f 730 2676 5484 3762
f 345 2909 5485 2910
f 1113 4528 5485 2909
f 1373 4597 5485 4528
f 1148 2910 5485 4597
f 270 2608 5486 2609
f 729 3759 5486 2608
f 1373 4528 5486 3759
f 1113 2609 5486 4528
f 73 1820 5487 1821
f 668 3637 5487 1820
f 1374 3640 5487 3637
f 669 1821 5487 3640
f 186 2273 5488 2272
f 945 4192 5488 2273
f 1374 3637 5488 4192
f 668 2272 5488 3637
f 345 2910 5489 2908
f 1148 4598 5489 2910
f 1374 4192 5489 4598
f 945 2908 5489 4192
f 287 2675 5490 2678
f 669 3640 5490 2675
f 1374 4598 5490 3640
f 1148 2678 5490 4598
f 15 1588 5491 1590
f 436 3174 5491 1588
f 1375 3178 5491 3174
f 438 1590 5491 3178
f 148 2122 5492 2119
f 870 4041 5492 2122
f 1375 3174 5492 4041
f 436 2119 5492 3174
f 346 2912 5493 2911
f 874 4049 5493 2912
f 1375 4041 5493 4049
f 870 2911 5493 4041
f 150 2127 5494 2130
f 438 3178 5494 2127
f 1375 4049 5494 3178
f 874 2130 5494 4049
f 45 1710 5495 1708
f 558 3418 5495 1710
f 1376 3414 5495 3418
f 556 1708 5495 3414
f 232 2458 5496 2455
f 1038 4377 5496 2458
f 1376 3418 5496 4377
f 558 2455 5496 3418
f 346 2911 5497 2913
f 870 4042 5497 2911
f 1376 4377 5497 4042
f 1038 2913 5497 4377
f 148 2120 5498 2122
f 556 3414 5498 2120
f 1376 4042 5498 3414
f 870 2122 5498 4042
f 88 1881 5499 1879
f 729 3760 5499 1881
f 1377 3756 5499 3760
f 727 1879 5499 3756
f 270 2610 5500 2608
f 1114 4529 5500 2610
f 1377 3760 5500 4529
f 729 2608 5500 3760
f 346 2913 5501 2914
f 1038 4378 5501 2913
f 1377 4529 5501 4378
f 1114 2914 5501 4529
f 232 2456 5502 2458
f 727 3756 5502 2456
f 1377 4378 5502 3756
f 1038 2458 5502 4378
f 64 1783 5503 1786
f 631 3564 5503 1783
f 1378 3570 5503 3564
f 634 1786 5503 3570
f 150 2130 5504 2128
f 874 4050 5504 2130
f 1378 3564 5504 4050
f 631 2128 5504 3564
f 346 2914 5505 2912
f 1114 4530 5505 2914
f 1378 4050 5505 4530
f 874 2912 5505 4050
f 270 2607 5506 2610
f 634 3570 5506 2607
f 1378 4530 5506 3570
f 1114 2610 5506 4530
f 8 1560 5507 1562
f 408 3118 5507 1560
f 1379 3121 5507 3118
f 410 1562 5507 3121
f 120 2010 5508 2007
f 814 3929 5508 2010
f 1379 3118 5508 3929
f 408 2007 5508 3118
f 347 2916 5509 2915
f 817 3935 5509 2916
f 1379 3929 5509 3935
f 814 2915 5509 3929
f 122 2015 5510 2017
f 410 3121 5510 2015
f 1379 3935 5510 3121
f 817 2017 5510 3935
f 48 1722 5511 1719
f 570 3441 5511 1722
f 1380 3436 5511 3441
f 567 1719 5511 3436
f 238 2481 5512 2479
f 1049 4399 5512 2481
f 1380 3441 5512 4399
f 570 2479 5512 3441
f 347 2915 5513 2917
f 814 3930 5513 2915
f 1380 4399 5513 3930
f 1049 2917 5513 4399
f 120 2008 5514 2010
f 567 3436 5514 2008
f 1380 3930 5514 3436
f 814 2010 5514 3930
f 89 1884 5515 1883
f 732 3765 5515 1884
f 1381 3763 5515 3765
f 731 1883 5515 3763
f 241 2493 5516 2492
f 1055 4411 5516 2493
f 1381 3765 5516 4411
f 732 2492 5516 3765
f 347 2917 5517 2918
f 1049 4400 5517 2917
f 1381 4411 5517 4400
f 1055 2918 5517 4411
f 238 2480 5518 2481
f 731 3763 5518 2480
f 1381 4400 5518 3763
f 1049 2481 5518 4400
f 50 1727 5519 1729
f 575 3451 5519 1727
f 1382 3455 5519 3451
f 577 1729 5519 3455
f 122 2017 5520 2016
f 817 3936 5520 2017
f 1382 3451 5520 3936
f 575 2016 5520 3451
f 347 2918 5521 2916
f 1055 4412 5521 2918
f 1382 3936 5521 4412
f 817 2916 5521 3936
f 241 2491 5522 2493
f 577 3455 5522 2491
f 1382 4412 5522 3455
f 1055 2493 5522 4412
f 16 1594 5523 1592
f 442 3185 5523 1594
f 1383 3182 5523 3185
f 440 1592 5523 3182
f 154 2145 5524 2143
f 881 4063 5524 2145
f 1383 3185 5524 4063
f 442 2143 5524 3185
f 348 2919 5525 2920
f 878 4057 5525 2919
f 1383 4063 5525 4057
f 881 2920 5525 4063
f 152 2135 5526 2138
f 440 3182 5526 2135
f 1383 4057 5526 3182
f 878 2138 5526 4057
f 66 1793 5527 1791
f 641 3583 5527 1793
f 1384 3579 5527 3583
f 639 1791 5527 3579
f 273 2621 5528 2619
f 1119 4539 5528 2621
f 1384 3583 5528 4539
f 641 2619 5528 3583
f 348 2920 5529 2922
f 881 4064 5529 2920
f 1384 4539 5529 4064
f 1119 2922 5529 4539
f 154 2144 5530 2145
f 639 3579 5530 2144
f 1384 4064 5530 3579
f 881 2145 5530 4064
f 89 1883 5531 1885
f 731 3764 5531 1883
f 1385 3767 5531 3764
f 733 1885 5531 3767
f 238 2482 5532 2480
f 1050 4401 5532 2482
f 1385 3764 5532 4401
f 731 2480 5532 3764
f 348 2922 5533 2921
f 1119 4540 5533 2922
f 1385 4401 5533 4540
f 1050 2921 5533 4401
f 273 2620 5534 2621
f 733 3767 5534 2620
f 1385 4540 5534 3767
f 1119 2621 5534 4540
f 48 1720 5535 1722
f 568 3438 5535 1720
f 1386 3442 5535 3438
f 570 1722 5535 3442
f 152 2138 5536 2136
f 878 4058 5536 2138
f 1386 3438 5536 4058
f 568 2136 5536 3438
f 348 2921 5537 2919
f 1050 4402 5537 2921
f 1386 4058 5537 4402
f 878 2919 5537 4058
f 238 2479 5538 2482
f 570 3442 5538 2479
f 1386 4402 5538 3442
f 1050 2482 5538 4402
f 24 1626 5539 1625
f 474 3250 5539 1626
f 1387 3247 5539 3250
f 473 1625 5539 3247
f 186 2274 5540 2271
f 946 4193 5540 2274
f 1387 3250 5540 4193
f 474 2271 5540 3250
f 349 2923 5541 2924
f 943 4187 5541 2923
f 1387 4193 5541 4187
f 946 2924 5541 4193
f 185 2267 5542 2269
f 473 3247 5542 2267
f 1387 4187 5542 3247
f 943 2269 5542 4187
f 73 1822 5543 1820
f 670 3641 5543 1822
f 1388 3638 5543 3641
f 668 1820 5543 3638
f 288 2681 5544 2679
f 1149 4599 5544 2681
f 1388 3641 5544 4599
f 670 2679 5544 3641
f 349 2924 5545 2926
f 946 4194 5545 2924
f 1388 4599 5545 4194
f 1149 2926 5545 4599
f 186 2272 5546 2274
f 668 3638 5546 2272
f 1388 4194 5546 3638
f 946 2274 5546 4194
f 89 1885 5547 1886
f 733 3768 5547 1885
f 1389 3769 5547 3768
f 734 1886 5547 3769
f 273 2622 5548 2620
f 1120 4541 5548 2622
f 1389 3768 5548 4541
f 733 2620 5548 3768
f 349 2926 5549 2925
f 1149 4600 5549 2926
f 1389 4541 5549 4600
f 1120 2925 5549 4541
f 288 2680 5550 2681
f 734 3769 5550 2680
f 1389 4600 5550 3769
f 1149 2681 5550 4600
f 66 1792 5551 1793
f 640 3581 5551 1792
f 1390 3584 5551 3581
f 641 1793 5551 3584
f 185 2269 5552 2268
f 943 4188 5552 2269
f 1390 3581 5552 4188
f 640 2268 5552 3581
f 349 2925 5553 2923
f 1120 4542 5553 2925
f 1390 4188 5553 4542
f 943 2923 5553 4188
f 273 2619 5554 2622
f 641 3584 5554 2619
f 1390 4542 5554 3584
f 1120 2622 5554 4542
f 20 1608 5555 1609
f 456 3213 5555 1608
f 1391 3216 5555 3213
f 457 1609 5555 3216
f 168 2201 5556 2199
f 909 4119 5556 2201
f 1391 3213 5556 4119
f 456 2199 5556 3213
f 350 2928 5557 2927
f 912 4125 5557 2928
f 1391 4119 5557 4125
f 909 2927 5557 4119
f 169 2203 5558 2206
f 457 3216 5558 2203
f 1391 4125 5558 3216
f 912 2206 5558 4125
f 50 1729 5559 1728
f 577 3456 5559 1729
f 1392 3453 5559 3456
f 576 1728 5559 3453
f 241 2494 5560 2491
f 1056 4413 5560 2494
f 1392 3456 5560 4413
f 577 2491 5560 3456
f 350 2927 5561 2929
f 909 4120 5561 2927
f 1392 4413 5561 4120
f 1056 2929 5561 4413
f 168 2200 5562 2201
f 576 3453 5562 2200
f 1392 4120 5562 3453
f 909 2201 5562 4120
f 89 1886 5563 1884
f 734 3770 5563 1886
f 1393 3766 5563 3770
f 732 1884 5563 3766
f 288 2682 5564 2680
f 1150 4601 5564 2682
f 1393 3770 5564 4601
f 734 2680 5564 3770
f 350 2929 5565 2930
f 1056 4414 5565 2929
f 1393 4601 5565 4414
f 1150 2930 5565 4601
f 241 2492 5566 2494
f 732 3766 5566 2492
f 1393 4414 5566 3766
f 1056 2494 5566 4414
f 73 1819 5567 1822
f 667 3636 5567 1819
f 1394 3642 5567 3636
f 670 1822 5567 3642
f 169 2206 5568 2204
f 912 4126 5568 2206
f 1394 3636 5568 4126
f 667 2204 5568 3636
f 350 2930 5569 2928
f 1150 4602 5569 2930
f 1394 4126 5569 4602
f 912 2928 5569 4126
f 288 2679 5570 2682
f 670 3642 5570 2679
f 1394 4602 5570 3642
f 1150 2682 5570 4602
f 4 1549 5571 1550
f 397 3095 5571 1549
f 1395 3098 5571 3095
f 398 1550 5571 3098
f 109 1965 5572 1963
f 791 3883 5572 1965
f 1395 3095 5572 3883
f 397 1963 5572 3095
f 351 2932 5573 2931
f 794 3889 5573 2932
f 1395 3883 5573 3889
f 791 2931 5573 3883
f 110 1967 5574 1970
f 398 3098 5574 1967
f 1395 3889 5574 3098
f 794 1970 5574 3889
f 37 1677 5575 1675
f 525 3351 5575 1677
f 1396 3347 5575 3351
f 523 1675 5575 3347
f 215 2389 5576 2387
f 1003 4307 5576 2389
f 1396 3351 5576 4307
f 525 2387 5576 3351
f 351 2931 5577 2933
f 791 3884 5577 2931
f 1396 4307 5577 3884
f 1003 2933 5577 4307
f 109 1964 5578 1965
f 523 3347 5578 1964
f 1396 3884 5578 3347
f 791 1965 5578 3884
f 90 1888 5579 1887
f 736 3773 5579 1888
f 1397 3771 5579 3773
f 735 1887 5579 3771
f 218 2401 5580 2400
f 1009 4319 5580 2401
f 1397 3773 5580 4319
f 736 2400 5580 3773
f 351 2933 5581 2934
f 1003 4308 5581 2933
f 1397 4319 5581 4308
f 1009 2934 5581 4319
f 215 2388 5582 2389
f 735 3771 5582 2388
f 1397 4308 5582 3771
f 1003 2389 5582 4308
f 38 1679 5583 1682
f 527 3356 5583 1679
f 1398 3361 5583 3356
f 530 1682 5583 3361
f 110 1970 5584 1968
f 794 3890 5584 1970
f 1398 3356 5584 3890
f 527 1968 5584 3356
f 351 2934 5585 2932
f 1009 4320 5585 2934
f 1398 3890 5585 4320
f 794 2932 5585 3890
f 218 2399 5586 2401
f 530 3361 5586 2399
f 1398 4320 5586 3361
f 1009 2401 5586 4320
f 14 1585 5587 1584
f 433 3168 5587 1585
f 1399 3165 5587 3168
f 432 1584 5587 3165
f 145 2110 5588 2107
f 864 4029 5588 2110
f 1399 3168 5588 4029
f 433 2107 5588 3168
f 352 2935 5589 2936
f 861 4023 5589 2935
f 1399 4029 5589 4023
f 864 2936 5589 4029
f 144 2103 5590 2105
f 432 3165 5590 2103
f 1399 4023 5590 3165
f 861 2105 5590 4023
f 61 1774 5591 1771
f 622 3545 5591 1774
f 1400 3540 5591 3545
f 619 1771 5591 3540
f 264 2585 5592 2583
f 1101 4503 5592 2585
f 1400 3545 5592 4503
f 622 2583 5592 3545
f 352 2936 5593 2938
f 864 4030 5593 2936
f 1400 4503 5593 4030
f 1101 2938 5593 4503
f 145 2108 5594 2110
f 619 3540 5594 2108
f 1400 4030 5594 3540
f 864 2110 5594 4030
f 90 1887 5595 1889
f 735 3772 5595 1887
f 1401 3775 5595 3772
f 737 1889 5595 3775
f 215 2390 5596 2388
f 1004 4309 5596 2390
f 1401 3772 5596 4309
f 735 2388 5596 3772
f 352 2938 5597 2937
f 1101 4504 5597 2938
f 1401 4309 5597 4504
f 1004 2937 5597 4309
f 264 2584 5598 2585
f 737 3775 5598 2584
f 1401 4504 5598 3775
f 1101 2585 5598 4504
f 37 1676 5599 1677
f 524 3349 5599 1676
f 1402 3352 5599 3349
f 525 1677 5599 3352
f 144 2105 5600 2104
f 861 4024 5600 2105
f 1402 3349 5600 4024
f 524 2104 5600 3349
f 352 2937 5601 2935
f 1004 4310 5601 2937
f 1402 4024 5601 4310
f 861 2935 5601 4024
f 215 2387 5602 2390
f 525 3352 5602 2387
f 1402 4310 5602 3352
f 1004 2390 5602 4310
f 24 1625 5603 1623
f 473 3248 5603 1625
f 1403 3244 5603 3248
f 471 1623 5603 3244
f 185 2270 5604 2267
f 944 4189 5604 2270
f 1403 3248 5604 4189
f 473 2267 5604 3248
f 353 2939 5605 2940
f 940 4181 5605 2939
f 1403 4189 5605 4181
f 944 2940 5605 4189
f 183 2259 5606 2262
f 471 3244 5606 2259
f 1403 4181 5606 3244
f 940 2262 5606 4181
f 66 1794 5607 1792
f 642 3585 5607 1794
f 1404 3582 5607 3585
f 640 1792 5607 3582
f 274 2625 5608 2623
f 1121 4543 5608 2625
f 1404 3585 5608 4543
f 642 2623 5608 3585
f 353 2940 5609 2942
f 944 4190 5609 2940
f 1404 4543 5609 4190
f 1121 2942 5609 4543
f 185 2268 5610 2270
f 640 3582 5610 2268
f 1404 4190 5610 3582
f 944 2270 5610 4190
f 90 1889 5611 1890
f 737 3776 5611 1889
f 1405 3777 5611 3776
f 738 1890 5611 3777
f 264 2586 5612 2584
f 1102 4505 5612 2586
f 1405 3776 5612 4505
f 737 2584 5612 3776
f 353 2942 5613 2941
f 1121 4544 5613 2942
f 1405 4505 5613 4544
f 1102 2941 5613 4505
f 274 2624 5614 2625
f 738 3777 5614 2624
f 1405 4544 5614 3777
f 1121 2625 5614 4544
f 61 1772 5615 1774
f 620 3542 5615 1772
f 1406 3546 5615 3542
f 622 1774 5615 3546
f 183 2262 5616 2260
f 940 4182 5616 2262
f 1406 3542 5616 4182
f 620 2260 5616 3542
f 353 2941 5617 2939
f 1102 4506 5617 2941
f 1406 4182 5617 4506
f 940 2939 5617 4182
f 264 2583 5618 2586
f 622 3546 5618 2583
f 1406 4506 5618 3546
f 1102 2586 5618 4506
f 16 1591 5619 1594
f 439 3180 5619 1591
f 1407 3186 5619 3180
f 442 1594 5619 3186
f 151 2134 5620 2131
f 876 4053 5620 2134
f 1407 3180 5620 4053
f 439 2131 5620 3180
f 354 2944 5621 2943
f 882 4065 5621 2944
f 1407 4053 5621 4065
f 876 2943 5621 4053
f 154 2143 5622 2146
f 442 3186 5622 2143
f 1407 4065 5622 3186
f 882 2146 5622 4065
f 38 1682 5623 1680
f 530 3362 5623 1682
f 1408 3358 5623 3362
f 528 1680 5623 3358
f 218 2402 5624 2399
f 1010 4321 5624 2402
f 1408 3362 5624 4321
f 530 2399 5624 3362
f 354 2943 5625 2945
f 876 4054 5625 2943
f 1408 4321 5625 4054
f 1010 2945 5625 4321
f 151 2132 5626 2134
f 528 3358 5626 2132
f 1408 4054 5626 3358
f 876 2134 5626 4054
f 90 1890 5627 1888
f 738 3778 5627 1890
f 1409 3774 5627 3778
f 736 1888 5627 3774
f 274 2626 5628 2624
f 1122 4545 5628 2626
f 1409 3778 5628 4545
f 738 2624 5628 3778
f 354 2945 5629 2946
f 1010 4322 5629 2945
f 1409 4545 5629 4322
f 1122 2946 5629 4545
f 218 2400 5630 2402
f 736 3774 5630 2400
f 1409 4322 5630 3774
f 1010 2402 5630 4322
f 66 1791 5631 1794
f 639 3580 5631 1791
f 1410 3586 5631 3580
f 642 1794 5631 3586
f 154 2146 5632 2144
f 882 4066 5632 2146
f 1410 3580 5632 4066
f 639 2144 5632 3580
f 354 2946 5633 2944
f 1122 4546 5633 2946
f 1410 4066 5633 4546
f 882 2944 5633 4066
f 274 2623 5634 2626
f 642 3586 5634 2623
f 1410 4546 5634 3586
f 1122 2626 5634 4546
f 1 1540 5635 1539
f 388 3078 5635 1540
f 1411 3076 5635 3078
f 387 1539 5635 3076
f 100 1930 5636 1927
f 774 3849 5636 1930
f 1411 3078 5636 3849
f 388 1927 5636 3078
f 355 2947 5637 2948
f 772 3845 5637 2947
f 1411 3849 5637 3845
f 774 2948 5637 3849
f 99 1923 5638 1926
f 387 3076 5638 1923
f 1411 3845 5638 3076
f 772 1926 5638 3845
f 28 1642 5639 1639
f 490 3281 5639 1642
f 1412 3276 5639 3281
f 487 1639 5639 3276
f 198 2321 5640 2319
f 969 4239 5640 2321
f 1412 3281 5640 4239
f 490 2319 5640 3281
f 355 2948 5641 2950
f 774 3850 5641 2948
f 1412 4239 5641 3850
f 969 2950 5641 4239
f 100 1928 5642 1930
f 487 3276 5642 1928
f 1412 3850 5642 3276
f 774 1930 5642 3850
f 91 1891 5643 1892
f 739 3779 5643 1891
f 1413 3781 5643 3779
f 740 1892 5643 3781
f 196 2313 5644 2312
f 965 4231 5644 2313
f 1413 3779 5644 4231
f 739 2312 5644 3779
f 355 2950 5645 2949
f 969 4240 5645 2950
f 1413 4231 5645 4240
f 965 2949 5645 4231
f 198 2320 5646 2321
f 740 3781 5646 2320
f 1413 4240 5646 3781
f 969 2321 5646 4240
f 27 1635 5647 1638
f 483 3268 5647 1635
f 1414 3273 5647 3268
f 486 1638 5647 3273
f 99 1926 5648 1924
f 772 3846 5648 1926
f 1414 3268 5648 3846
f 483 1924 5648 3268
f 355 2949 5649 2947
f 965 4232 5649 2949
f 1414 3846 5649 4232
f 772 2947 5649 3846
f 196 2311 5650 2313
f 486 3273 5650 2311
f 1414 4232 5650 3273
f 965 2313 5650 4232
f 10 1570 5651 1567
f 418 3137 5651 1570
f 1415 3132 5651 3137
f 415 1567 5651 3132
f 130 2049 5652 2047
f 833 3967 5652 2049
f 1415 3137 5652 3967
f 418 2047 5652 3137
f 356 2951 5653 2952
f 828 3957 5653 2951
f 1415 3967 5653 3957
f 833 2952 5653 3967
f 127 2035 5654 2038
f 415 3132 5654 2035
f 1415 3957 5654 3132
f 828 2038 5654 3957
f 54 1745 5655 1743
f 593 3487 5655 1745
f 1416 3483 5655 3487
f 591 1743 5655 3483
f 249 2525 5656 2523
f 1071 4443 5656 2525
f 1416 3487 5656 4443
f 593 2523 5656 3487
f 356 2952 5657 2954
f 833 3968 5657 2952
f 1416 4443 5657 3968
f 1071 2954 5657 4443
f 130 2048 5658 2049
f 591 3483 5658 2048
f 1416 3968 5658 3483
f 833 2049 5658 3968
f 91 1892 5659 1894
f 740 3782 5659 1892
f 1417 3785 5659 3782
f 742 1894 5659 3785
f 198 2322 5660 2320
f 970 4241 5660 2322
f 1417 3782 5660 4241
f 740 2320 5660 3782
f 356 2954 5661 2953
f 1071 4444 5661 2954
f 1417 4241 5661 4444
f 970 2953 5661 4241
f 249 2524 5662 2525
f 742 3785 5662 2524
f 1417 4444 5662 3785
f 1071 2525 5662 4444
f 28 1640 5663 1642
f 488 3278 5663 1640
f 1418 3282 5663 3278
f 490 1642 5663 3282
f 127 2038 5664 2036
f 828 3958 5664 2038
f 1418 3278 5664 3958
f 488 2036 5664 3278
f 356 2953 5665 2951
f 970 4242 5665 2953
f 1418 3958 5665 4242
f 828 2951 5665 3958
f 198 2319 5666 2322
f 490 3282 5666 2319
f 1418 4242 5666 3282
f 970 2322 5666 4242
f 25 1627 5667 1628
f 475 3251 5667 1627
f 1419 3253 5667 3251
f 476 1628 5667 3253
f 187 2277 5668 2275
f 947 4195 5668 2277
f 1419 3251 5668 4195
f 475 2275 5668 3251
f 357 2956 5669 2955
f 949 4199 5669 2956
f 1419 4195 5669 4199
f 947 2955 5669 4195
f 188 2279 5670 2281
f 476 3253 5670 2279
f 1419 4199 5670 3253
f 949 2281 5670 4199
f 52 1737 5671 1736
f 585 3471 5671 1737
f 1420 3469 5671 3471
f 584 1736 5671 3469
f 245 2509 5672 2507
f 1063 4427 5672 2509
f 1420 3471 5672 4427
f 585 2507 5672 3471
f 357 2955 5673 2957
f 947 4196 5673 2955
f 1420 4427 5673 4196
f 1063 2957 5673 4427
f 187 2276 5674 2277
f 584 3469 5674 2276
f 1420 4196 5674 3469
f 947 2277 5674 4196
f 91 1894 5675 1893
f 742 3786 5675 1894
f 1421 3783 5675 3786
f 741 1893 5675 3783
f 249 2526 5676 2524
f 1072 4445 5676 2526
f 1421 3786 5676 4445
f 742 2524 5676 3786
f 357 2957 5677 2958
f 1063 4428 5677 2957
f 1421 4445 5677 4428
f 1072 2958 5677 4445
f 245 2508 5678 2509
f 741 3783 5678 2508
f 1421 4428 5678 3783
f 1063 2509 5678 4428
f 54 1744 5679 1745
f 592 3485 5679 1744
f 1422 3488 5679 3485
f 593 1745 5679 3488
f 188 2281 5680 2280
f 949 4200 5680 2281
f 1422 3485 5680 4200
f 592 2280 5680 3485
f 357 2958 5681 2956
f 1072 4446 5681 2958
f 1422 4200 5681 4446
f 949 2956 5681 4200
f 249 2523 5682 2526
f 593 3488 5682 2523
f 1422 4446 5682 3488
f 1072 2526 5682 4446
f 9 1563 5683 1566
f 411 3124 5683 1563
f 1423 3129 5683 3124
f 414 1566 5683 3129
f 123 2022 5684 2019
f 820 3941 5684 2022
f 1423 3124 5684 3941
f 411 2019 5684 3124
f 358 2960 5685 2959
f 825 3951 5685 2960
f 1423 3941 5685 3951
f 820 2959 5685 3941
f 126 2031 5686 2033
f 414 3129 5686 2031
f 1423 3951 5686 3129
f 825 2033 5686 3951
f 27 1638 5687 1636
f 486 3274 5687 1638
f 1424 3270 5687 3274
f 484 1636 5687 3270
f 196 2314 5688 2311
f 966 4233 5688 2314
f 1424 3274 5688 4233
f 486 2311 5688 3274
f 358 2959 5689 2961
f 820 3942 5689 2959
f 1424 4233 5689 3942
f 966 2961 5689 4233
f 123 2020 5690 2022
f 484 3270 5690 2020
f 1424 3942 5690 3270
f 820 2022 5690 3942
f 91 1893 5691 1891
f 741 3784 5691 1893
f 1425 3780 5691 3784
f 739 1891 5691 3780
f 245 2510 5692 2508
f 1064 4429 5692 2510
f 1425 3784 5692 4429
f 741 2508 5692 3784
f 358 2961 5693 2962
f 966 4234 5693 2961
f 1425 4429 5693 4234
f 1064 2962 5693 4429
f 196 2312 5694 2314
f 739 3780 5694 2312
f 1425 4234 5694 3780
f 966 2314 5694 4234
f 52 1735 5695 1737
f 583 3467 5695 1735
f 1426 3472 5695 3467
f 585 1737 5695 3472
f 126 2033 5696 2032
f 825 3952 5696 2033
f 1426 3467 5696 3952
f 583 2032 5696 3467
f 358 2962 5697 2960
f 1064 4430 5697 2962
f 1426 3952 5697 4430
f 825 2960 5697 3952
f 245 2507 5698 2510
f 585 3472 5698 2507
f 1426 4430 5698 3472
f 1064 2510 5698 4430
f 3 1546 5699 1545
f 394 3090 5699 1546
f 1427 3088 5699 3090
f 393 1545 5699 3088
f 106 1954 5700 1951
f 786 3873 5700 1954
f 1427 3090 5700 3873
f 394 1951 5700 3090
f 359 2963 5701 2964
f 784 3869 5701 2963
f 1427 3873 5701 3869
f 786 2964 5701 3873
f 105 1947 5702 1950
f 393 3088 5702 1947
f 1427 3869 5702 3088
f 784 1950 5702 3869
f 34 1666 5703 1663
f 514 3329 5703 1666
f 1428 3324 5703 3329
f 511 1663 5703 3324
f 210 2369 5704 2367
f 993 4287 5704 2369
f 1428 3329 5704 4287
f 514 2367 5704 3329
f 359 2964 5705 2966
f 786 3874 5705 2964
f 1428 4287 5705 3874
f 993 2966 5705 4287
f 106 1952 5706 1954
f 511 3324 5706 1952
f 1428 3874 5706 3324
f 786 1954 5706 3874
f 92 1895 5707 1896
f 743 3787 5707 1895
f 1429 3789 5707 3787
f 744 1896 5707 3789
f 208 2361 5708 2360
f 989 4279 5708 2361
f 1429 3787 5708 4279
f 743 2360 5708 3787
f 359 2966 5709 2965
f 993 4288 5709 2966
f 1429 4279 5709 4288
f 989 2965 5709 4279
f 210 2368 5710 2369
f 744 3789 5710 2368
f 1429 4288 5710 3789
f 993 2369 5710 4288
f 33 1659 5711 1662
f 507 3316 5711 1659
f 1430 3321 5711 3316
f 510 1662 5711 3321
f 105 1950 5712 1948
f 784 3870 5712 1950
f 1430 3316 5712 3870
f 507 1948 5712 3316
f 359 2965 5713 2963
f 989 4280 5713 2965
f 1430 3870 5713 4280
f 784 2963 5713 3870
f 208 2359 5714 2361
f 510 3321 5714 2359
f 1430 4280 5714 3321
f 989 2361 5714 4280
f 14 1586 5715 1583
f 434 3169 5715 1586
f 1431 3164 5715 3169
f 431 1583 5715 3164
f 146 2113 5716 2111
f 865 4031 5716 2113
f 1431 3169 5716 4031
f 434 2111 5716 3169
f 360 2967 5717 2968
f 860 4021 5717 2967
f 1431 4031 5717 4021
f 865 2968 5717 4031
f 143 2099 5718 2102
f 431 3164 5718 2099
f 1431 4021 5718 3164
f 860 2102 5718 4021
f 62 1777 5719 1775
f 625 3551 5719 1777
f 1432 3547 5719 3551
f 623 1775 5719 3547
f 265 2589 5720 2587
f 1103 4507 5720 2589
f 1432 3551 5720 4507
f 625 2587 5720 3551
f 360 2968 5721 2970
f 865 4032 5721 2968
f 1432 4507 5721 4032
f 1103 2970 5721 4507
f 146 2112 5722 2113
f 623 3547 5722 2112
f 1432 4032 5722 3547
f 865 2113 5722 4032
f 92 1896 5723 1898
f 744 3790 5723 1896
f 1433 3793 5723 3790
f 746 1898 5723 3793
f 210 2370 5724 2368
f 994 4289 5724 2370
f 1433 3790 5724 4289
f 744 2368 5724 3790
f 360 2970 5725 2969
f 1103 4508 5725 2970
f 1433 4289 5725 4508
f 994 2969 5725 4289
f 265 2588 5726 2589
f 746 3793 5726 2588
f 1433 4508 5726 3793
f 1103 2589 5726 4508
f 34 1664 5727 1666
f 512 3326 5727 1664
f 1434 3330 5727 3326
f 514 1666 5727 3330
f 143 2102 5728 2100
f 860 4022 5728 2102
f 1434 3326 5728 4022
f 512 2100 5728 3326
f 360 2969 5729 2967
f 994 4290 5729 2969
f 1434 4022 5729 4290
f 860 2967 5729 4022
f 210 2367 5730 2370
f 514 3330 5730 2367
f 1434 4290 5730 3330
f 994 2370 5730 4290
f 25 1628 5731 1630
f 476 3254 5731 1628
f 1435 3257 5731 3254
f 478 1630 5731 3257
f 188 2282 5732 2279
f 950 4201 5732 2282
f 1435 3254 5732 4201
f 476 2279 5732 3254
f 361 2972 5733 2971
f 953 4207 5733 2972
f 1435 4201 5733 4207
f 950 2971 5733 4201
f 190 2287 5734 2289
f 478 3257 5734 2287
f 1435 4207 5734 3257
f 953 2289 5734 4207
f 54 1746 5735 1744
f 594 3489 5735 1746
f 1436 3486 5735 3489
f 592 1744 5735 3486
f 250 2529 5736 2527
f 1073 4447 5736 2529
f 1436 3489 5736 4447
f 594 2527 5736 3489
f 361 2971 5737 2973
f 950 4202 5737 2971
f 1436 4447 5737 4202
f 1073 2973 5737 4447
f 188 2280 5738 2282
f 592 3486 5738 2280
f 1436 4202 5738 3486
f 950 2282 5738 4202
f 92 1898 5739 1897
f 746 3794 5739 1898
f 1437 3791 5739 3794
f 745 1897 5739 3791
f 265 2590 5740 2588
f 1104 4509 5740 2590
f 1437 3794 5740 4509
f 746 2588 5740 3794
f 361 2973 5741 2974
f 1073 4448 5741 2973
f 1437 4509 5741 4448
f 1104 2974 5741 4509
f 250 2528 5742 2529
f 745 3791 5742 2528
f 1437 4448 5742 3791
f 1073 2529 5742 4448
f 62 1776 5743 1777
f 624 3549 5743 1776
f 1438 3552 5743 3549
f 625 1777 5743 3552
f 190 2289 5744 2288
f 953 4208 5744 2289
f 1438 3549 5744 4208
f 624 2288 5744 3549
f 361 2974 5745 2972
f 1104 4510 5745 2974
f 1438 4208 5745 4510
f 953 2972 5745 4208
f 265 2587 5746 2590
f 625 3552 5746 2587
f 1438 4510 5746 3552
f 1104 2590 5746 4510
f 10 1568 5747 1570
f 416 3134 5747 1568
f 1439 3138 5747 3134
f 418 1570 5747 3138
f 128 2042 5748 2039
f 830 3961 5748 2042
f 1439 3134 5748 3961
f 416 2039 5748 3134
f 362 2976 5749 2975
f 834 3969 5749 2976
f 1439 3961 5749 3969
f 830 2975 5749 3961
f 130 2047 5750 2050
f 418 3138 5750 2047
f 1439 3969 5750 3138
f 834 2050 5750 3969
f 33 1662 5751 1660
f 510 3322 5751 1662
f 1440 3318 5751 3322
f 508 1660 5751 3318
f 208 2362 5752 2359
f 990 4281 5752 2362
f 1440 3322 5752 4281
f 510 2359 5752 3322
f 362 2975 5753 2977
f 830 3962 5753 2975
f 1440 4281 5753 3962
f 990 2977 5753 4281
f 128 2040 5754 2042
f 508 3318 5754 2040
f 1440 3962 5754 3318
f 830 2042 5754 3962
f 92 1897 5755 1895
f 745 3792 5755 1897
f 1441 3788 5755 3792
f 743 1895 5755 3788
f 250 2530 5756 2528
f 1074 4449 5756 2530
f 1441 3792 5756 4449
f 745 2528 5756 3792
f 362 2977 5757 2978
f 990 4282 5757 2977
f 1441 4449 5757 4282
f 1074 2978 5757 4449
f 208 2360 5758 2362
f 743 3788 5758 2360
f 1441 4282 5758 3788
f 990 2362 5758 4282
f 54 1743 5759 1746
f 591 3484 5759 1743
f 1442 3490 5759 3484
f 594 1746 5759 3490
f 130 2050 5760 2048
f 834 3970 5760 2050
f 1442 3484 5760 3970
f 591 2048 5760 3484
f 362 2978 5761 2976
f 1074 4450 5761 2978
f 1442 3970 5761 4450
f 834 2976 5761 3970
f 250 2527 5762 2530
f 594 3490 5762 2527
f 1442 4450 5762 3490
f 1074 2530 5762 4450
f 4 1548 5763 1549
f 396 3094 5763 1548
f 1443 3096 5763 3094
f 397 1549 5763 3096
f 108 1962 5764 1959
f 790 3881 5764 1962
f 1443 3094 5764 3881
f 396 1959 5764 3094
f 363 2980 5765 2979
f 792 3885 5765 2980
f 1443 3881 5765 3885
f 790 2979 5765 3881
f 109 1963 5766 1966
f 397 3096 5766 1963
f 1443 3885 5766 3096
f 792 1966 5766 3885
f 36 1674 5767 1671
f 522 3345 5767 1674
f 1444 3340 5767 3345
f 519 1671 5767 3340
f 214 2385 5768 2383
f 1001 4303 5768 2385
f 1444 3345 5768 4303
f 522 2383 5768 3345
f 363 2979 5769 2981
f 790 3882 5769 2979
f 1444 4303 5769 3882
f 1001 2981 5769 4303
f 108 1960 5770 1962
f 519 3340 5770 1960
f 1444 3882 5770 3340
f 790 1962 5770 3882
f 93 1900 5771 1899
f 748 3797 5771 1900
f 1445 3795 5771 3797
f 747 1899 5771 3795
f 216 2393 5772 2392
f 1005 4311 5772 2393
f 1445 3797 5772 4311
f 748 2392 5772 3797
f 363 2981 5773 2982
f 1001 4304 5773 2981
f 1445 4311 5773 4304
f 1005 2982 5773 4311
f 214 2384 5774 2385
f 747 3795 5774 2384
f 1445 4304 5774 3795
f 1001 2385 5774 4304
f 37 1675 5775 1678
f 523 3348 5775 1675
f 1446 3353 5775 3348
f 526 1678 5775 3353
f 109 1966 5776 1964
f 792 3886 5776 1966
f 1446 3348 5776 3886
f 523 1964 5776 3348
f 363 2982 5777 2980
f 1005 4312 5777 2982
f 1446 3886 5777 4312
f 792 2980 5777 3886
f 216 2391 5778 2393
f 526 3353 5778 2391
f 1446 4312 5778 3353
f 1005 2393 5778 4312
f 12 1578 5779 1576
f 426 3153 5779 1578
f 1447 3150 5779 3153
f 424 1576 5779 3150
f 138 2081 5780 2079
f 849 3999 5780 2081
f 1447 3153 5780 3999
f 426 2079 5780 3153
f 364 2983 5781 2984
f 846 3993 5781 2983
f 1447 3999 5781 3993
f 849 2984 5781 3999
f 136 2071 5782 2074
f 424 3150 5782 2071
f 1447 3993 5782 3150
f 846 2074 5782 3993
f 58 1761 5783 1759
f 609 3519 5783 1761
f 1448 3515 5783 3519
f 607 1759 5783 3515
f 257 2557 5784 2555
f 1087 4475 5784 2557
f 1448 3519 5784 4475
f 609 2555 5784 3519
f 364 2984 5785 2986
f 849 4000 5785 2984
f 1448 4475 5785 4000
f 1087 2986 5785 4475
f 138 2080 5786 2081
f 607 3515 5786 2080
f 1448 4000 5786 3515
f 849 2081 5786 4000
f 93 1899 5787 1901
f 747 3796 5787 1899
f 1449 3799 5787 3796
f 749 1901 5787 3799
f 214 2386 5788 2384
f 1002 4305 5788 2386
f 1449 3796 5788 4305
f 747 2384 5788 3796
f 364 2986 5789 2985
f 1087 4476 5789 2986
f 1449 4305 5789 4476
f 1002 2985 5789 4305
f 257 2556 5790 2557
f 749 3799 5790 2556
f 1449 4476 5790 3799
f 1087 2557 5790 4476
f 36 1672 5791 1674
f 520 3342 5791 1672
f 1450 3346 5791 3342
f 522 1674 5791 3346
f 136 2074 5792 2072
f 846 3994 5792 2074
f 1450 3342 5792 3994
f 520 2072 5792 3342
f 364 2985 5793 2983
f 1002 4306 5793 2985
f 1450 3994 5793 4306
f 846 2983 5793 3994
f 214 2383 5794 2386
f 522 3346 5794 2383
f 1450 4306 5794 3346
f 1002 2386 5794 4306
f 25 1630 5795 1629
f 478 3258 5795 1630
f 1451 3255 5795 3258
f 477 1629 5795 3255
f 190 2290 5796 2287
f 954 4209 5796 2290
f 1451 3258 5796 4209
f 478 2287 5796 3258
f 365 2987 5797 2988
f 951 4203 5797 2987
f 1451 4209 5797 4203
f 954 2988 5797 4209
f 189 2283 5798 2285
f 477 3255 5798 2283
f 1451 4203 5798 3255
f 951 2285 5798 4203
f 62 1778 5799 1776
f 626 3553 5799 1778
f 1452 3550 5799 3553
f 624 1776 5799 3550
f 266 2593 5800 2591
f 1105 4511 5800 2593
f 1452 3553 5800 4511
f 626 2591 5800 3553
f 365 2988 5801 2990
f 954 4210 5801 2988
f 1452 4511 5801 4210
f 1105 2990 5801 4511
f 190 2288 5802 2290
f 624 3550 5802 2288
f 1452 4210 5802 3550
f 954 2290 5802 4210
f 93 1901 5803 1902
f 749 3800 5803 1901
f 1453 3801 5803 3800
f 750 1902 5803 3801
f 257 2558 5804 2556
f 1088 4477 5804 2558
f 1453 3800 5804 4477
f 749 2556 5804 3800
f 365 2990 5805 2989
f 1105 4512 5805 2990
f 1453 4477 5805 4512
f 1088 2989 5805 4477
f 266 2592 5806 2593
f 750 3801 5806 2592
f 1453 4512 5806 3801
f 1105 2593 5806 4512
f 58 1760 5807 1761
f 608 3517 5807 1760
f 1454 3520 5807 3517
f 609 1761 5807 3520
f 189 2285 5808 2284
f 951 4204 5808 2285
f 1454 3517 5808 4204
f 608 2284 5808 3517
f 365 2989 5809 2987
f 1088 4478 5809 2989
f 1454 4204 5809 4478
f 951 2987 5809 4204
f 257 2555 5810 2558
f 609 3520 5810 2555
f 1454 4478 5810 3520
f 1088 2558 5810 4478
f 14 1584 5811 1586
f 432 3166 5811 1584
f 1455 3170 5811 3166
f 434 1586 5811 3170
f 144 2106 5812 2103
f 862 4025 5812 2106
f 1455 3166 5812 4025
f 432 2103 5812 3166
f 366 2992 5813 2991
f 866 4033 5813 2992
f 1455 4025 5813 4033
f 862 2991 5813 4025
f 146 2111 5814 2114
f 434 3170 5814 2111
f 1455 4033 5814 3170
f 866 2114 5814 4033
f 37 1678 5815 1676
f 526 3354 5815 1678
f 1456 3350 5815 3354
f 524 1676 5815 3350
f 216 2394 5816 2391
f 1006 4313 5816 2394
f 1456 3354 5816 4313
f 526 2391 5816 3354
f 366 2991 5817 2993
f 862 4026 5817 2991
f 1456 4313 5817 4026
f 1006 2993 5817 4313
f 144 2104 5818 2106
f 524 3350 5818 2104
f 1456 4026 5818 3350
f 862 2106 5818 4026
f 93 1902 5819 1900
f 750 3802 5819 1902
f 1457 3798 5819 3802
f 748 1900 5819 3798
f 266 2594 5820 2592
f 1106 4513 5820 2594
f 1457 3802 5820 4513
f 750 2592 5820 3802
f 366 2993 5821 2994
f 1006 4314 5821 2993
f 1457 4513 5821 4314
f 1106 2994 5821 4513
f 216 2392 5822 2394
f 748 3798 5822 2392
f 1457 4314 5822 3798
f 1006 2394 5822 4314
f 62 1775 5823 1778
f 623 3548 5823 1775
f 1458 3554 5823 3548
f 626 1778 5823 3554
f 146 2114 5824 2112
f 866 4034 5824 2114
f 1458 3548 5824 4034
f 623 2112 5824 3548
f 366 2994 5825 2992
f 1106 4514 5825 2994
f 1458 4034 5825 4514
f 866 2992 5825 4034
f 266 2591 5826 2594
f 626 3554 5826 2591
f 1458 4514 5826 3554
f 1106 2594 5826 4514
f 2 1542 5827 1543
f 390 3082 5827 1542
f 1459 3084 5827 3082
f 391 1543 5827 3084
f 102 1938 5828 1935
f 778 3857 5828 1938
f 1459 3082 5828 3857
f 390 1935 5828 3082
f 367 2996 5829 2995
f 780 3861 5829 2996
f 1459 3857 5829 3861
f 778 2995 5829 3857
f 103 1939 5830 1942
f 391 3084 5830 1939
f 1459 3861 5830 3084
f 780 1942 5830 3861
f 30 1650 5831 1647
f 498 3297 5831 1650
f 1460 3292 5831 3297
f 495 1647 5831 3292
f 202 2337 5832 2335
f 977 4255 5832 2337
f 1460 3297 5832 4255
f 498 2335 5832 3297
f 367 2995 5833 2997
f 778 3858 5833 2995
f 1460 4255 5833 3858
f 977 2997 5833 4255
f 102 1936 5834 1938
f 495 3292 5834 1936
f 1460 3858 5834 3292
f 778 1938 5834 3858
f 94 1904 5835 1903
f 752 3805 5835 1904
f 1461 3803 5835 3805
f 751 1903 5835 3803
f 204 2345 5836 2344
f 981 4263 5836 2345
f 1461 3805 5836 4263
f 752 2344 5836 3805
f 367 2997 5837 2998
f 977 4256 5837 2997
f 1461 4263 5837 4256
f 981 2998 5837 4263
f 202 2336 5838 2337
f 751 3803 5838 2336
f 1461 4256 5838 3803
f 977 2337 5838 4256
f 31 1651 5839 1654
f 499 3300 5839 1651
f 1462 3305 5839 3300
f 502 1654 5839 3305
f 103 1942 5840 1940
f 780 3862 5840 1942
f 1462 3300 5840 3862
f 499 1940 5840 3300
f 367 2998 5841 2996
f 981 4264 5841 2998
f 1462 3862 5841 4264
f 780 2996 5841 3862
f 204 2343 5842 2345
f 502 3305 5842 2343
f 1462 4264 5842 3305
f 981 2345 5842 4264
f 9 1566 5843 1564
f 414 3130 5843 1566
f 1463 3126 5843 3130
f 412 1564 5843 3126
f 126 2034 5844 2031
f 826 3953 5844 2034
f 1463 3130 5844 3953
f 414 2031 5844 3130
f 368 2999 5845 3000
f 822 3945 5845 2999
f 1463 3953 5845 3945
f 826 3000 5845 3953
f 124 2023 5846 2026
f 412 3126 5846 2023
f 1463 3945 5846 3126
f 822 2026 5846 3945
f 52 1738 5847 1735
f 586 3473 5847 1738
f 1464 3468 5847 3473
f 583 1735 5847 3468
f 246 2513 5848 2511
f 1065 4431 5848 2513
f 1464 3473 5848 4431
f 586 2511 5848 3473
f 368 3000 5849 3002
f 826 3954 5849 3000
f 1464 4431 5849 3954
f 1065 3002 5849 4431
f 126 2032 5850 2034
f 583 3468 5850 2032
f 1464 3954 5850 3468
f 826 2034 5850 3954
f 94 1903 5851 1905
f 751 3804 5851 1903
f 1465 3807 5851 3804
f 753 1905 5851 3807
f 202 2338 5852 2336
f 978 4257 5852 2338
f 1465 3804 5852 4257
f 751 2336 5852 3804
f 368 3002 5853 3001
f 1065 4432 5853 3002
f 1465 4257 5853 4432
f 978 3001 5853 4257
f 246 2512 5854 2513
f 753 3807 5854 2512
f 1465 4432 5854 3807
f 1065 2513 5854 4432
f 30 1648 5855 1650
f 496 3294 5855 1648
f 1466 3298 5855 3294
f 498 1650 5855 3298
f 124 2026 5856 2024
f 822 3946 5856 2026
f 1466 3294 5856 3946
f 496 2024 5856 3294
f 368 3001 5857 2999
f 978 4258 5857 3001
f 1466 3946 5857 4258
f 822 2999 5857 3946
f 202 2335 5858 2338
f 498 3298 5858 2335
f 1466 4258 5858 3298
f 978 2338 5858 4258
f 25 1629 5859 1627
f 477 3256 5859 1629
f 1467 3252 5859 3256
f 475 1627 5859 3252
f 189 2286 5860 2283
f 952 4205 5860 2286
f 1467 3256 5860 4205
f 477 2283 5860 3256
f 369 3003 5861 3004
f 948 4197 5861 3003
f 1467 4205 5861 4197
f 952 3004 5861 4205
f 187 2275 5862 2278
f 475 3252 5862 2275
f 1467 4197 5862 3252
f 948 2278 5862 4197
f 58 1762 5863 1760
f 610 3521 5863 1762
f 1468 3518 5863 3521
f 608 1760 5863 3518
f 258 2561 5864 2559
f 1089 4479 5864 2561
f 1468 3521 5864 4479
f 610 2559 5864 3521
f 369 3004 5865 3006
f 952 4206 5865 3004
f 1468 4479 5865 4206
f 1089 3006 5865 4479
f 189 2284 5866 2286
f 608 3518 5866 2284
f 1468 4206 5866 3518
f 952 2286 5866 4206
f 94 1905 5867 1906
f 753 3808 5867 1905
f 1469 3809 5867 3808
f 754 1906 5867 3809
f 246 2514 5868 2512
f 1066 4433 5868 2514
f 1469 3808 5868 4433
f 753 2512 5868 3808
f 369 3006 5869 3005
f 1089 4480 5869 3006
f 1469 4433 5869 4480
f 1066 3005 5869 4433
f 258 2560 5870 2561
f 754 3809 5870 2560
f 1469 4480 5870 3809
f 1089 2561 5870 4480
f 52 1736 5871 1738
f 584 3470 5871 1736
f 1470 3474 5871 3470
f 586 1738 5871 3474
f 187 2278 5872 2276
f 948 4198 5872 2278
f 1470 3470 5872 4198
f 584 2276 5872 3470
f 369 3005 5873 3003
f 1066 4434 5873 3005
f 1470 4198 5873 4434
f 948 3003 5873 4198
f 246 2511 5874 2514
f 586 3474 5874 2511
f 1470 4434 5874 3474
f 1066 2514 5874 4434
f 12 1575 5875 1578
f 423 3148 5875 1575
f 1471 3154 5875 3148
f 426 1578 5875 3154
f 135 2070 5876 2067
f 844 3989 5876 2070
f 1471 3148 5876 3989
f 423 2067 5876 3148
f 370 3008 5877 3007
f 850 4001 5877 3008
f 1471 3989 5877 4001
f 844 3007 5877 3989
f 138 2079 5878 2082
f 426 3154 5878 2079
f 1471 4001 5878 3154
f 850 2082 5878 4001
f 31 1654 5879 1652
f 502 3306 5879 1654
f 1472 3302 5879 3306
f 500 1652 5879 3302
f 204 2346 5880 2343
f 982 4265 5880 2346
f 1472 3306 5880 4265
f 502 2343 5880 3306
f 370 3007 5881 3009
f 844 3990 5881 3007
f 1472 4265 5881 3990
f 982 3009 5881 4265
f 135 2068 5882 2070
f 500 3302 5882 2068
f 1472 3990 5882 3302
f 844 2070 5882 3990
f 94 1906 5883 1904
f 754 3810 5883 1906
f 1473 3806 5883 3810
f 752 1904 5883 3806
f 258 2562 5884 2560
f 1090 4481 5884 2562
f 1473 3810 5884 4481
f 754 2560 5884 3810
f 370 3009 5885 3010
f 982 4266 5885 3009
f 1473 4481 5885 4266
f 1090 3010 5885 4481
f 204 2344 5886 2346
f 752 3806 5886 2344
f 1473 4266 5886 3806
f 982 2346 5886 4266
f 58 1759 5887 1762
f 607 3516 5887 1759
f 1474 3522 5887 3516
f 610 1762 5887 3522
f 138 2082 5888 2080
f 850 4002 5888 2082
f 1474 3516 5888 4002
f 607 2080 5888 3516
f 370 3010 5889 3008
f 1090 4482 5889 3010
f 1474 4002 5889 4482
f 850 3008 5889 4002
f 258 2559 5890 2562
f 610 3522 5890 2559
f 1474 4482 5890 3522
f 1090 2562 5890 4482
f 5 1552 5891 1553
f 400 3102 5891 1552
f 1475 3104 5891 3102
f 401 1553 5891 3104
f 112 1978 5892 1975
f 798 3897 5892 1978
f 1475 3102 5892 3897
f 400 1975 5892 3102
f 371 3012 5893 3011
f 800 3901 5893 3012
f 1475 3897 5893 3901
f 798 3011 5893 3897
f 113 1979 5894 1982
f 401 3104 5894 1979
f 1475 3901 5894 3104
f 800 1982 5894 3901
f 40 1690 5895 1687
f 538 3377 5895 1690
f 1476 3372 5895 3377
f 535 1687 5895 3372
f 222 2417 5896 2415
f 1017 4335 5896 2417
f 1476 3377 5896 4335
f 538 2415 5896 3377
f 371 3011 5897 3013
f 798 3898 5897 3011
f 1476 4335 5897 3898
f 1017 3013 5897 4335
f 112 1976 5898 1978
f 535 3372 5898 1976
f 1476 3898 5898 3372
f 798 1978 5898 3898
f 95 1908 5899 1907
f 756 3813 5899 1908
f 1477 3811 5899 3813
f 755 1907 5899 3811
f 224 2425 5900 2424
f 1021 4343 5900 2425
f 1477 3813 5900 4343
f 756 2424 5900 3813
f 371 3013 5901 3014
f 1017 4336 5901 3013
f 1477 4343 5901 4336
f 1021 3014 5901 4343
f 222 2416 5902 2417
f 755 3811 5902 2416
f 1477 4336 5902 3811
f 1017 2417 5902 4336
f 41 1691 5903 1694
f 539 3380 5903 1691
f 1478 3385 5903 3380
f 542 1694 5903 3385
f 113 1982 5904 1980
f 800 3902 5904 1982
f 1478 3380 5904 3902
f 539 1980 5904 3380
f 371 3014 5905 3012
f 1021 4344 5905 3014
f 1478 3902 5905 4344
f 800 3012 5905 3902
f 224 2423 5906 2425
f 542 3385 5906 2423
f 1478 4344 5906 3385
f 1021 2425 5906 4344
f 17 1598 5907 1595
f 446 3193 5907 1598
f 1479 3188 5907 3193
f 443 1595 5907 3188
f 158 2161 5908 2159
f 889 4079 5908 2161
f 1479 3193 5908 4079
f 446 2159 5908 3193
f 372 3015 5909 3016
f 884 4069 5909 3015
f 1479 4079 5909 4069
f 889 3016 5909 4079
f 155 2147 5910 2150
f 443 3188 5910 2147
f 1479 4069 5910 3188
f 884 2150 5910 4069
f 68 1801 5911 1799
f 649 3599 5911 1801
f 1480 3595 5911 3599
f 647 1799 5911 3595
f 277 2637 5912 2635
f 1127 4555 5912 2637
f 1480 3599 5912 4555
f 649 2635 5912 3599
f 372 3016 5913 3018
f 889 4080 5913 3016
f 1480 4555 5913 4080
f 1127 3018 5913 4555
f 158 2160 5914 2161
f 647 3595 5914 2160
f 1480 4080 5914 3595
f 889 2161 5914 4080
f 95 1907 5915 1909
f 755 3812 5915 1907
f 1481 3815 5915 3812
f 757 1909 5915 3815
f 222 2418 5916 2416
f 1018 4337 5916 2418
f 1481 3812 5916 4337
f 755 2416 5916 3812
f 372 3018 5917 3017
f 1127 4556 5917 3018
f 1481 4337 5917 4556
f 1018 3017 5917 4337
f 277 2636 5918 2637
f 757 3815 5918 2636
f 1481 4556 5918 3815
f 1127 2637 5918 4556
f 40 1688 5919 1690
f 536 3374 5919 1688
f 1482 3378 5919 3374
f 538 1690 5919 3378
f 155 2150 5920 2148
f 884 4070 5920 2150
f 1482 3374 5920 4070
f 536 2148 5920 3374
f 372 3017 5921 3015
f 1018 4338 5921 3017
f 1482 4070 5921 4338
f 884 3015 5921 4070
f 222 2415 5922 2418
f 538 3378 5922 2415
f 1482 4338 5922 3378
f 1018 2418 5922 4338
f 26 1632 5923 1631
f 480 3261 5923 1632
f 1483 3259 5923 3261
f 479 1631 5923 3259
f 192 2297 5924 2295
f 957 4215 5924 2297
f 1483 3261 5924 4215
f 480 2295 5924 3261
f 373 3019 5925 3020
f 955 4211 5925 3019
f 1483 4215 5925 4211
f 957 3020 5925 4215
f 191 2291 5926 2293
f 479 3259 5926 2291
f 1483 4211 5926 3259
f 955 2293 5926 4211
f 70 1809 5927 1808
f 657 3615 5927 1809
f 1484 3613 5927 3615
f 656 1808 5927 3613
f 281 2653 5928 2651
f 1135 4571 5928 2653
f 1484 3615 5928 4571
f 657 2651 5928 3615
f 373 3020 5929 3022
f 957 4216 5929 3020
f 1484 4571 5929 4216
f 1135 3022 5929 4571
f 192 2296 5930 2297
f 656 3613 5930 2296
f 1484 4216 5930 3613
f 957 2297 5930 4216
f 95 1909 5931 1910
f 757 3816 5931 1909
f 1485 3817 5931 3816
f 758 1910 5931 3817
f 277 2638 5932 2636
f 1128 4557 5932 2638
f 1485 3816 5932 4557
f 757 2636 5932 3816
f 373 3022 5933 3021
f 1135 4572 5933 3022
f 1485 4557 5933 4572
f 1128 3021 5933 4557
f 281 2652 5934 2653
f 758 3817 5934 2652
f 1485 4572 5934 3817
f 1135 2653 5934 4572
f 68 1800 5935 1801
f 648 3597 5935 1800
f 1486 3600 5935 3597
f 649 1801 5935 3600
f 191 2293 5936 2292
f 955 4212 5936 2293
f 1486 3597 5936 4212
f 648 2292 5936 3597
f 373 3021 5937 3019
f 1128 4558 5937 3021
f 1486 4212 5937 4558
f 955 3019 5937 4212
f 277 2635 5938 2638
f 649 3600 5938 2635
f 1486 4558 5938 3600
f 1128 2638 5938 4558
f 18 1599 5939 1602
f 447 3196 5939 1599
f 1487 3201 5939 3196
f 450 1602 5939 3201
f 159 2166 5940 2163
f 892 4085 5940 2166
f 1487 3196 5940 4085
f 447 2163 5940 3196
f 374 3024 5941 3023
f 897 4095 5941 3024
f 1487 4085 5941 4095
f 892 3023 5941 4085
f 162 2175 5942 2177
f 450 3201 5942 2175
f 1487 4095 5942 3201
f 897 2177 5942 4095
f 41 1694 5943 1692
f 542 3386 5943 1694
f 1488 3382 5943 3386
f 540 1692 5943 3382
f 224 2426 5944 2423
f 1022 4345 5944 2426
f 1488 3386 5944 4345
f 542 2423 5944 3386
f 374 3023 5945 3025
f 892 4086 5945 3023
f 1488 4345 5945 4086
f 1022 3025 5945 4345
f 159 2164 5946 2166
f 540 3382 5946 2164
f 1488 4086 5946 3382
f 892 2166 5946 4086
f 95 1910 5947 1908
f 758 3818 5947 1910
f 1489 3814 5947 3818
f 756 1908 5947 3814
f 281 2654 5948 2652
f 1136 4573 5948 2654
f 1489 3818 5948 4573
f 758 2652 5948 3818
f 374 3025 5949 3026
f 1022 4346 5949 3025
f 1489 4573 5949 4346
f 1136 3026 5949 4573
f 224 2424 5950 2426
f 756 3814 5950 2424
f 1489 4346 5950 3814
f 1022 2426 5950 4346
f 70 1807 5951 1809
f 655 3611 5951 1807
f 1490 3616 5951 3611
f 657 1809 5951 3616
f 162 2177 5952 2176
f 897 4096 5952 2177
f 1490 3611 5952 4096
f 655 2176 5952 3611
f 374 3026 5953 3024
f 1136 4574 5953 3026
f 1490 4096 5953 4574
f 897 3024 5953 4096
f 281 2651 5954 2654
f 657 3616 5954 2651
f 1490 4574 5954 3616
f 1136 2654 5954 4574
f 6 1556 5955 1555
f 404 3110 5955 1556
f 1491 3108 5955 3110
f 403 1555 5955 3108
f 116 1994 5956 1991
f 806 3913 5956 1994
f 1491 3110 5956 3913
f 404 1991 5956 3110
f 375 3027 5957 3028
f 804 3909 5957 3027
f 1491 3913 5957 3909
f 806 3028 5957 3913
f 115 1987 5958 1990
f 403 3108 5958 1987
f 1491 3909 5958 3108
f 804 1990 5958 3909
f 44 1706 5959 1703
f 554 3409 5959 1706
f 1492 3404 5959 3409
f 551 1703 5959 3404
f 230 2449 5960 2447
f 1033 4367 5960 2449
f 1492 3409 5960 4367
f 554 2447 5960 3409
f 375 3028 5961 3030
f 806 3914 5961 3028
f 1492 4367 5961 3914
f 1033 3030 5961 4367
f 116 1992 5962 1994
f 551 3404 5962 1992
f 1492 3914 5962 3404
f 806 1994 5962 3914
f 96 1911 5963 1912
f 759 3819 5963 1911
f 1493 3821 5963 3819
f 760 1912 5963 3821
f 228 2441 5964 2440
f 1029 4359 5964 2441
f 1493 3819 5964 4359
f 759 2440 5964 3819
f 375 3030 5965 3029
f 1033 4368 5965 3030
f 1493 4359 5965 4368
f 1029 3029 5965 4359
f 230 2448 5966 2449
f 760 3821 5966 2448
f 1493 4368 5966 3821
f 1033 2449 5966 4368
f 43 1699 5967 1702
f 547 3396 5967 1699
f 1494 3401 5967 3396
f 550 1702 5967 3401
f 115 1990 5968 1988
f 804 3910 5968 1990
f 1494 3396 5968 3910
f 547 1988 5968 3396
f 375 3029 5969 3027
f 1029 4360 5969 3029
f 1494 3910 5969 4360
f 804 3027 5969 3910
f 228 2439 5970 2441
f 550 3401 5970 2439
f 1494 4360 5970 3401
f 1029 2441 5970 4360
f 19 1606 5971 1603
f 454 3209 5971 1606
f 1495 3204 5971 3209
f 451 1603 5971 3204
f 166 2193 5972 2191
f 905 4111 5972 2193
f 1495 3209 5972 4111
f 454 2191 5972 3209
f 376 3031 5973 3032
f 900 4101 5973 3031
f 1495 4111 5973 4101
f 905 3032 5973 4111
f 163 2179 5974 2182
f 451 3204 5974 2179
f 1495 4101 5974 3204
f 900 2182 5974 4101
f 72 1817 5975 1815
f 665 3631 5975 1817
f 1496 3627 5975 3631
f 663 1815 5975 3627
f 285 2669 5976 2667
f 1143 4587 5976 2669
f 1496 3631 5976 4587
f 665 2667 5976 3631
f 376 3032 5977 3034
f 905 4112 5977 3032
f 1496 4587 5977 4112
f 1143 3034 5977 4587
f 166 2192 5978 2193
f 663 3627 5978 2192
f 1496 4112 5978 3627
f 905 2193 5978 4112
f 96 1912 5979 1914
f 760 3822 5979 1912
f 1497 3825 5979 3822
f 762 1914 5979 3825
f 230 2450 5980 2448
f 1034 4369 5980 2450
f 1497 3822 5980 4369
f 760 2448 5980 3822
f 376 3034 5981 3033
f 1143 4588 5981 3034
f 1497 4369 5981 4588
f 1034 3033 5981 4369
f 285 2668 5982 2669
f 762 3825 5982 2668
f 1497 4588 5982 3825
f 1143 2669 5982 4588
f 44 1704 5983 1706
f 552 3406 5983 1704
f 1498 3410 5983 3406
f 554 1706 5983 3410
f 163 2182 5984 2180
f 900 4102 5984 2182
f 1498 3406 5984 4102
f 552 2180 5984 3406
f 376 3033 5985 3031
f 1034 4370 5985 3033
f 1498 4102 5985 4370
f 900 3031 5985 4102
f 230 2447 5986 2450
f 554 3410 5986 2447
f 1498 4370 5986 3410
f 1034 2450 5986 4370
f 26 1631 5987 1633
f 479 3260 5987 1631
f 1499 3263 5987 3260
f 481 1633 5987 3263
f 191 2294 5988 2291
f 956 4213 5988 2294
f 1499 3260 5988 4213
f 479 2291 5988 3260
f 377 3036 5989 3035
f 959 4219 5989 3036
f 1499 4213 5989 4219
f 956 3035 5989 4213
f 193 2299 5990 2301
f 481 3263 5990 2299
f 1499 4219 5990 3263
f 959 2301 5990 4219
f 68 1802 5991 1800
f 650 3601 5991 1802
f 1500 3598 5991 3601
f 648 1800 5991 3598
f 278 2641 5992 2639
f 1129 4559 5992 2641
f 1500 3601 5992 4559
f 650 2639 5992 3601
f 377 3035 5993 3037
f 956 4214 5993 3035
f 1500 4559 5993 4214
f 1129 3037 5993 4559
f 191 2292 5994 2294
f 648 3598 5994 2292
f 1500 4214 5994 3598
f 956 2294 5994 4214
f 96 1914 5995 1913
f 762 3826 5995 1914
f 1501 3823 5995 3826
f 761 1913 5995 3823
f 285 2670 5996 2668
f 1144 4589 5996 2670
f 1501 3826 5996 4589
f 762 2668 5996 3826
f 377 3037 5997 3038
f 1129 4560 5997 3037
f 1501 4589 5997 4560
f 1144 3038 5997 4589
f 278 2640 5998 2641
f 761 3823 5998 2640
f 1501 4560 5998 3823
f 1129 2641 5998 4560
f 72 1816 5999 1817
f 664 3629 5999 1816
f 1502 3632 5999 3629
f 665 1817 5999 3632
f 193 2301 6000 2300
f 959 4220 6000 2301
f 1502 3629 6000 4220
f 664 2300 6000 3629
f 377 3038 6001 3036
f 1144 4590 6001 3038
f 1502 4220 6001 4590
f 959 3036 6001 4220
f 285 2667 6002 2670
f 665 3632 6002 2667
f 1502 4590 6002 3632
f 1144 2670 6002 4590
f 17 1596 6003 1598
f 444 3190 6003 1596
f 1503 3194 6003 3190
f 446 1598 6003 3194
f 156 2154 6004 2151
f 886 4073 6004 2154
f 1503 3190 6004 4073
f 444 2151 6004 3190
f 378 3040 6005 3039
f 890 4081 6005 3040
f 1503 4073 6005 4081
f 886 3039 6005 4073
f 158 2159 6006 2162
f 446 3194 6006 2159
f 1503 4081 6006 3194
f 890 2162 6006 4081
f 43 1702 6007 1700
f 550 3402 6007 1702
f 1504 3398 6007 3402
f 548 1700 6007 3398
f 228 2442 6008 2439
f 1030 4361 6008 2442
f 1504 3402 6008 4361
f 550 2439 6008 3402
f 378 3039 6009 3041
f 886 4074 6009 3039
f 1504 4361 6009 4074
f 1030 3041 6009 4361
f 156 2152 6010 2154
f 548 3398 6010 2152
f 1504 4074 6010 3398
f 886 2154 6010 4074
f 96 1913 6011 1911
f 761 3824 6011 1913
f 1505 3820 6011 3824
f 759 1911 6011 3820
f 278 2642 6012 2640
f 1130 4561 6012 2642
f 1505 3824 6012 4561
f 761 2640 6012 3824
f 378 3041 6013 3042
f 1030 4362 6013 3041
f 1505 4561 6013 4362
f 1130 3042 6013 4561
f 228 2440 6014 2442
f 759 3820 6014 2440
f 1505 4362 6014 3820
f 1030 2442 6014 4362
f 68 1799 6015 1802
f 647 3596 6015 1799
f 1506 3602 6015 3596
f 650 1802 6015 3602
f 158 2162 6016 2160
f 890 4082 6016 2162
f 1506 3596 6016 4082
f 647 2160 6016 3596
f 378 3042 6017 3040
f 1130 4562 6017 3042
f 1506 4082 6017 4562
f 890 3040 6017 4082
f 278 2639 6018 2642
f 650 3602 6018 2639
f 1506 4562 6018 3602
f 1130 2642 6018 4562
f 8 1562 6019 1561
f 410 3122 6019 1562
f 1507 3120 6019 3122
f 409 1561 6019 3120
f 122 2018 6020 2015
f 818 3937 6020 2018
f 1507 3122 6020 3937
f 410 2015 6020 3122
f 379 3043 6021 3044
f 816 3933 6021 3043
f 1507 3937 6021 3933
f 818 3044 6021 3937
f 121 2011 6022 2014
f 409 3120 6022 2011
f 1507 3933 6022 3120
f 816 2014 6022 3933
f 50 1730 6023 1727
f 578 3457 6023 1730
f 1508 3452 6023 3457
f 575 1727 6023 3452
f 242 2497 6024 2495
f 1057 4415 6024 2497
f 1508 3457 6024 4415
f 578 2495 6024 3457
f 379 3044 6025 3046
f 818 3938 6025 3044
f 1508 4415 6025 3938
f 1057 3046 6025 4415
f 122 2016 6026 2018
f 575 3452 6026 2016
f 1508 3938 6026 3452
f 818 2018 6026 3938
f 97 1915 6027 1916
f 763 3827 6027 1915
f 1509 3829 6027 3827
f 764 1916 6027 3829
f 240 2489 6028 2488
f 1053 4407 6028 2489
f 1509 3827 6028 4407
f 763 2488 6028 3827
f 379 3046 6029 3045
f 1057 4416 6029 3046
f 1509 4407 6029 4416
f 1053 3045 6029 4407
f 242 2496 6030 2497
f 764 3829 6030 2496
f 1509 4416 6030 3829
f 1057 2497 6030 4416
f 49 1723 6031 1726
f 571 3444 6031 1723
f 1510 3449 6031 3444
f 574 1726 6031 3449
f 121 2014 6032 2012
f 816 3934 6032 2014
f 1510 3444 6032 3934
f 571 2012 6032 3444
f 379 3045 6033 3043
f 1053 4408 6033 3045
f 1510 3934 6033 4408
f 816 3043 6033 3934
f 240 2487 6034 2489
f 574 3449 6034 2487
f 1510 4408 6034 3449
f 1053 2489 6034 4408
f 20 1610 6035 1608
f 458 3217 6035 1610
f 1511 3214 6035 3217
f 456 1608 6035 3214
f 170 2209 6036 2207
f 913 4127 6036 2209
f 1511 3217 6036 4127
f 458 2207 6036 3217
f 380 3047 6037 3048
f 910 4121 6037 3047
f 1511 4127 6037 4121
f 913 3048 6037 4127
f 168 2199 6038 2202
f 456 3214 6038 2199
f 1511 4121 6038 3214
f 910 2202 6038 4121
f 74 1825 6039 1823
f 673 3647 6039 1825
f 1512 3643 6039 3647
f 671 1823 6039 3643
f 289 2685 6040 2683
f 1151 4603 6040 2685
f 1512 3647 6040 4603
f 673 2683 6040 3647
f 380 3048 6041 3050
f 913 4128 6041 3048
f 1512 4603 6041 4128
f 1151 3050 6041 4603
f 170 2208 6042 2209
f 671 3643 6042 2208
f 1512 4128 6042 3643
f 913 2209 6042 4128
f 97 1916 6043 1918
f 764 3830 6043 1916
f 1513 3833 6043 3830
f 766 1918 6043 3833
f 242 2498 6044 2496
f 1058 4417 6044 2498
f 1513 3830 6044 4417
f 764 2496 6044 3830
f 380 3050 6045 3049
f 1151 4604 6045 3050
f 1513 4417 6045 4604
f 1058 3049 6045 4417
f 289 2684 6046 2685
f 766 3833 6046 2684
f 1513 4604 6046 3833
f 1151 2685 6046 4604
f 50 1728 6047 1730
f 576 3454 6047 1728
f 1514 3458 6047 3454
f 578 1730 6047 3458
f 168 2202 6048 2200
f 910 4122 6048 2202
f 1514 3454 6048 4122
f 576 2200 6048 3454
f 380 3049 6049 3047
f 1058 4418 6049 3049
f 1514 4122 6049 4418
f 910 3047 6049 4122
f 242 2495 6050 2498
f 578 3458 6050 2495
f 1514 4418 6050 3458
f 1058 2498 6050 4418
f 26 1633 6051 1634
f 481 3264 6051 1633
f 1515 3265 6051 3264
f 482 1634 6051 3265
f 193 2302 6052 2299
f 960 4221 6052 2302
f 1515 3264 6052 4221
f 481 2299 6052 3264
f 381 3052 6053 3051
f 961 4223 6053 3052
f 1515 4221 6053 4223
f 960 3051 6053 4221
f 194 2303 6054 2305
f 482 3265 6054 2303
f 1515 4223 6054 3265
f 961 2305 6054 4223
f 72 1818 6055 1816
f 666 3633 6055 1818
f 1516 3630 6055 3633
f 664 1816 6055 3630
f 286 2673 6056 2671
f 1145 4591 6056 2673
f 1516 3633 6056 4591
f 666 2671 6056 3633
f 381 3051 6057 3053
f 960 4222 6057 3051
f 1516 4591 6057 4222
f 1145 3053 6057 4591
f 193 2300 6058 2302
f 664 3630 6058 2300
f 1516 4222 6058 3630
f 960 2302 6058 4222
f 97 1918 6059 1917
f 766 3834 6059 1918
f 1517 3831 6059 3834
f 765 1917 6059 3831
f 289 2686 6060 2684
f 1152 4605 6060 2686
f 1517 3834 6060 4605
f 766 2684 6060 3834
f 381 3053 6061 3054
f 1145 4592 6061 3053
f 1517 4605 6061 4592
f 1152 3054 6061 4605
f 286 2672 6062 2673
f 765 3831 6062 2672
f 1517 4592 6062 3831
f 1145 2673 6062 4592
f 74 1824 6063 1825
f 672 3645 6063 1824
f 1518 3648 6063 3645
f 673 1825 6063 3648
f 194 2305 6064 2304
f 961 4224 6064 2305
f 1518 3645 6064 4224
f 672 2304 6064 3645
f 381 3054 6065 3052
f 1152 4606 6065 3054
f 1518 4224 6065 4606
f 961 3052 6065 4224
f 289 2683 6066 2686
f 673 3648 6066 2683
f 1518 4606 6066 3648
f 1152 2686 6066 4606
f 19 1604 6067 1606
f 452 3206 6067 1604
f 1519 3210 6067 3206
f 454 1606 6067 3210
f 164 2186 6068 2183
f 902 4105 6068 2186
f 1519 3206 6068 4105
f 452 2183 6068 3206
f 382 3056 6069 3055
f 906 4113 6069 3056
f 1519 4105 6069 4113
f 902 3055 6069 4105
f 166 2191 6070 2194
f 454 3210 6070 2191
f 1519 4113 6070 3210
f 906 2194 6070 4113
f 49 1726 6071 1724
f 574 3450 6071 1726
f 1520 3446 6071 3450
f 572 1724 6071 3446
f 240 2490 6072 2487
f 1054 4409 6072 2490
f 1520 3450 6072 4409
f 574 2487 6072 3450
f 382 3055 6073 3057
f 902 4106 6073 3055
f 1520 4409 6073 4106
f 1054 3057 6073 4409
f 164 2184 6074 2186
f 572 3446 6074 2184
f 1520 4106 6074 3446
f 902 2186 6074 4106
f 97 1917 6075 1915
f 765 3832 6075 1917
f 1521 3828 6075 3832
f 763 1915 6075 3828
f 286 2674 6076 2672
f 1146 4593 6076 2674
f 1521 3832 6076 4593
f 765 2672 6076 3832
f 382 3057 6077 3058
f 1054 4410 6077 3057
f 1521 4593 6077 4410
f 1146 3058 6077 4593
f 240 2488 6078 2490
f 763 3828 6078 2488
f 1521 4410 6078 3828
f 1054 2490 6078 4410
f 72 1815 6079 1818
f 663 3628 6079 1815
f 1522 3634 6079 3628
f 666 1818 6079 3634
f 166 2194 6080 2192
f 906 4114 6080 2194
f 1522 3628 6080 4114
f 663 2192 6080 3628
f 382 3058 6081 3056
f 1146 4594 6081 3058
f 1522 4114 6081 4594
f 906 3056 6081 4114
f 286 2671 6082 2674
f 666 3634 6082 2671
f 1522 4594 6082 3634
f 1146 2674 6082 4594
f 7 1558 6083 1559
f 406 3114 6083 1558
f 1523 3116 6083 3114
f 407 1559 6083 3116
f 118 2002 6084 1999
f 810 3921 6084 2002
f 1523 3114 6084 3921
f 406 1999 6084 3114
f 383 3060 6085 3059
f 812 3925 6085 3060
f 1523 3921 6085 3925
f 810 3059 6085 3921
f 119 2003 6086 2006
f 407 3116 6086 2003
f 1523 3925 6086 3116
f 812 2006 6086 3925
f 46 1714 6087 1711
f 562 3425 6087 1714
f 1524 3420 6087 3425
f 559 1711 6087 3420
f 234 2465 6088 2463
f 1041 4383 6088 2465
f 1524 3425 6088 4383
f 562 2463 6088 3425
f 383 3059 6089 3061
f 810 3922 6089 3059
f 1524 4383 6089 3922
f 1041 3061 6089 4383
f 118 2000 6090 2002
f 559 3420 6090 2000
f 1524 3922 6090 3420
f 810 2002 6090 3922
f 98 1920 6091 1919
f 768 3837 6091 1920
f 1525 3835 6091 3837
f 767 1919 6091 3835
f 236 2473 6092 2472
f 1045 4391 6092 2473
f 1525 3837 6092 4391
f 768 2472 6092 3837
f 383 3061 6093 3062
f 1041 4384 6093 3061
f 1525 4391 6093 4384
f 1045 3062 6093 4391
f 234 2464 6094 2465
f 767 3835 6094 2464
f 1525 4384 6094 3835
f 1041 2465 6094 4384
f 47 1715 6095 1718
f 563 3428 6095 1715
f 1526 3433 6095 3428
f 566 1718 6095 3433
f 119 2006 6096 2004
f 812 3926 6096 2006
f 1526 3428 6096 3926
f 563 2004 6096 3428
f 383 3062 6097 3060
f 1045 4392 6097 3062
f 1526 3926 6097 4392
f 812 3060 6097 3926
f 236 2471 6098 2473
f 566 3433 6098 2471
f 1526 4392 6098 3433
f 1045 2473 6098 4392
f 18 1602 6099 1600
f 450 3202 6099 1602
f 1527 3198 6099 3202
f 448 1600 6099 3198
f 162 2178 6100 2175
f 898 4097 6100 2178
f 1527 3202 6100 4097
f 450 2175 6100 3202
f 384 3063 6101 3064
f 894 4089 6101 3063
f 1527 4097 6101 4089
f 898 3064 6101 4097
f 160 2167 6102 2170
f 448 3198 6102 2167
f 1527 4089 6102 3198
f 894 2170 6102 4089
f 70 1810 6103 1807
f 658 3617 6103 1810
f 1528 3612 6103 3617
f 655 1807 6103 3612
f 282 2657 6104 2655
f 1137 4575 6104 2657
f 1528 3617 6104 4575
f 658 2655 6104 3617
f 384 3064 6105 3066
f 898 4098 6105 3064
f 1528 4575 6105 4098
f 1137 3066 6105 4575
f 162 2176 6106 2178
f 655 3612 6106 2176
f 1528 4098 6106 3612
f 898 2178 6106 4098
f 98 1919 6107 1921
f 767 3836 6107 1919
f 1529 3839 6107 3836
f 769 1921 6107 3839
f 234 2466 6108 2464
f 1042 4385 6108 2466
f 1529 3836 6108 4385
f 767 2464 6108 3836
f 384 3066 6109 3065
f 1137 4576 6109 3066
f 1529 4385 6109 4576
f 1042 3065 6109 4385
f 282 2656 6110 2657
f 769 3839 6110 2656
f 1529 4576 6110 3839
f 1137 2657 6110 4576
f 46 1712 6111 1714
f 560 3422 6111 1712
f 1530 3426 6111 3422
f 562 1714 6111 3426
f 160 2170 6112 2168
f 894 4090 6112 2170
f 1530 3422 6112 4090
f 560 2168 6112 3422
f 384 3065 6113 3063
f 1042 4386 6113 3065
f 1530 4090 6113 4386
f 894 3063 6113 4090
f 234 2463 6114 2466
f 562 3426 6114 2463
f 1530 4386 6114 3426
f 1042 2466 6114 4386
f 26 1634 6115 1632
f 482 3266 6115 1634
f 1531 3262 6115 3266
f 480 1632 6115 3262
f 194 2306 6116 2303
f 962 4225 6116 2306
f 1531 3266 6116 4225
f 482 2303 6116 3266
f 385 3067 6117 3068
f 958 4217 6117 3067
f 1531 4225 6117 4217
f 962 3068 6117 4225
f 192 2295 6118 2298
f 480 3262 6118 2295
f 1531 4217 6118 3262
f 958 2298 6118 4217
f 74 1826 6119 1824
f 674 3649 6119 1826
f 1532 3646 6119 3649
f 672 1824 6119 3646
f 290 2689 6120 2687
f 1153 4607 6120 2689
f 1532 3649 6120 4607
f 674 2687 6120 3649
f 385 3068 6121 3070
f 962 4226 6121 3068
f 1532 4607 6121 4226
f 1153 3070 6121 4607
f 194 2304 6122 2306
f 672 3646 6122 2304
f 1532 4226 6122 3646
f 962 2306 6122 4226
f 98 1921 6123 1922
f 769 3840 6123 1921
f 1533 3841 6123 3840
f 770 1922 6123 3841
f 282 2658 6124 2656
f 1138 4577 6124 2658
f 1533 3840 6124 4577
f 769 2656 6124 3840
f 385 3070 6125 3069
f 1153 4608 6125 3070
f 1533 4577 6125 4608
f 1138 3069 6125 4577
f 290 2688 6126 2689
f 770 3841 6126 2688
f 1533 4608 6126 3841
f 1153 2689 6126 4608
f 70 1808 6127 1810
f 656 3614 6127 1808
f 1534 3618 6127 3614
f 658 1810 6127 3618
f 192 2298 6128 2296
f 958 4218 6128 2298
f 1534 3614 6128 4218
f 656 2296 6128 3614
f 385 3069 6129 3067
f 1138 4578 6129 3069
f 1534 4218 6129 4578
f 958 3067 6129 4218
f 282 2655 6130 2658
f 658 3618 6130 2655
f 1534 4578 6130 3618
f 1138 2658 6130 4578
f 20 1607 6131 1610
f 455 3212 6131 1607
f 1535 3218 6131 3212
f 458 1610 6131 3218
f 167 2198 6132 2195
f 908 4117 6132 2198
f 1535 3212 6132 4117
f 455 2195 6132 3212
f 386 3072 6133 3071
f 914 4129 6133 3072
f 1535 4117 6133 4129
f 908 3071 6133 4117
f 170 2207 6134 2210
f 458 3218 6134 2207
f 1535 4129 6134 3218
f 914 2210 6134 4129
f 47 1718 6135 1716
f 566 3434 6135 1718
f 1536 3430 6135 3434
f 564 1716 6135 3430
f 236 2474 6136 2471
f 1046 4393 6136 2474
f 1536 3434 6136 4393
f 566 2471 6136 3434
f 386 3071 6137 3073
f 908 4118 6137 3071
f 1536 4393 6137 4118
f 1046 3073 6137 4393
f 167 2196 6138 2198
f 564 3430 6138 2196
f 1536 4118 6138 3430
f 908 2198 6138 4118
f 98 1922 6139 1920
f 770 3842 6139 1922
f 1537 3838 6139 3842
f 768 1920 6139 3838
f 290 2690 6140 2688
f 1154 4609 6140 2690
f 1537 3842 6140 4609
f 770 2688 6140 3842
f 386 3073 6141 3074
f 1046 4394 6141 3073
f 1537 4609 6141 4394
f 1154 3074 6141 4609
f 236 2472 6142 2474
f 768 3838 6142 2472
f 1537 4394 6142 3838
f 1046 2474 6142 4394
f 74 1823 6143 1826
f 671 3644 6143 1823
f 1538 3650 6143 3644
f 674 1826 6143 3650
f 170 2210 6144 2208
f 914 4130 6144 2210
f 1538 3644 6144 4130
f 671 2208 6144 3644
f 386 3074 6145 3072
f 1154 4610 6145 3074
f 1538 4130 6145 4610
f 914 3072 6145 4130
f 290 2687 6146 2690
f 674 3650 6146 2687
f 1538 4610 6146 3650
f 1154 2690 6146 4610
'''.strip()

def make_obj (verts, name):
    return OBJ_TEMPLATE.format(name=name, vertices='\n'.join([
        'v %s %s %s'%(verts[i], verts[i+1], verts[i+2])
        for i in range(0, len(verts) - 2, 3)
    ]))
print(make_obj(x_train[12], sw_keys[12]))

# nobuyuki-data/e333b5c405fbbb98457cfef9186c20c7.json
v -0.625 0.095 -1.77223
v 0.622197 0.095 -1.775
v -0.635 0.472287 -1.795
v 0.635 0.472255 -1.795
v -0.512952 0.165 1.945
v 0.641926 0.225 1.875
v -0.58457 0.875 1.825
v 0.584568 0.875 1.825
v -0.00140141 0.0817718 -1.78684
v -0.631677 0.283644 -1.78529
v -0.599517 0.0994585 0.0863853
v 0.588037 0.283627 -1.74444
v 0.678034 0.114027 0.05
v 1.75731e-18 0.555732 -1.87846
v -0.706754 0.770612 0.015
v 0.701522 0.765365 0.015
v 0.064487 0.249123 1.85588
v -0.598932 0.52 1.93517
v 0.666463 0.55 1.90322
v -1.20699e-06 0.914477 1.86448
v -0.73557 0.401822 0.0506927
v 0.737945 0.416814 0.0325
v 0.0315428 0.184153 0.0681927
v -6.03497e-07 1.2853 0.015
v -0.000700705 0.283635 -1.87993
v 0.0322429 0.535 1.9867
v -0.313201 0.0817873 -1.78613
v -0.586695 0.189322 -1.73712
v -0.619929 0.0895594 -0.842922
v 0.310398 0.0817758 -1.78753
v 0.584251 0.189314 -1.73885
v 0.663591 0.0910384 -0.8625
v -0.626606 0.377965 -1.78341
v -0.3175 0.535993 -1.85871


In [15]:
import multiprocessing as mp
pool = mp.Pool(12)

def write_obj (args):
    verts, path = args
    print("writing %s"%path)
    with open(path, 'w') as f:
        f.write(make_obj(verts, path))
        
def build_dirs (paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def export_model_objs (epoch, num_samples=10, output_dir='altencoder-objs', parallel=False):
    model = load_model_from_epoch(epoch)
    print("predicting for epoch %s"%epoch)
    y_train, y_test = map(model.predict, (x_train, x_test))
    tasks = []
    print("generating write-obj jobs for epoch %s"%epoch)
    for i in range(num_samples):
        train_name = sw_keys[i].split('/')[1].split('.')[0]
        test_name  = sw_keys[i+500].split('/')[1].split('.')[0]
        
        build_dirs([
            os.path.join(output_dir, str(epoch), 'input'),
            os.path.join(output_dir, str(epoch), 'output'),
        ])
        tasks.append((x_train[i], os.path.join(output_dir, str(epoch), 'input', 'train-%s-input.obj'%train_name)))
        tasks.append((y_train[i], os.path.join(output_dir, str(epoch), 'output', 'train-%s-output.obj'%train_name)))
        tasks.append((x_test[i], os.path.join(output_dir, str(epoch), 'input', 'test-%s-input.obj'%test_name)))
        tasks.append((y_test[i], os.path.join(output_dir, str(epoch), 'output', 'test-%s-output.obj'%test_name)))
    
    print("dispatching write-obj jobs for epoch %s"%epoch)
    if parallel:
        pool.map(write_obj, tasks)
    else:
        list(map(write_obj, tasks))
    print("done processing models for epoch %s"%epoch)

# export_model_objs(2)
# export_model_objs(300)
# export_model_objs(200)
# export_model_objs(100)
# export_model_objs(50)
export_model_objs(40)
export_model_objs(30)

Loading altmodel/altmodel-epoch-40.h5
predicting for epoch 40
generating write-obj jobs for epoch 40
dispatching write-obj jobs for epoch 40
writing altencoder-objs/40/input/train-b8c8758f8ee058182a9eb0f146e94477-input.obj
writing altencoder-objs/40/output/train-b8c8758f8ee058182a9eb0f146e94477-output.obj
writing altencoder-objs/40/input/test-a52f704da0d3c947ca92a4cdad802b45-input.obj
writing altencoder-objs/40/output/test-a52f704da0d3c947ca92a4cdad802b45-output.obj
writing altencoder-objs/40/input/train-9b0d7aac4500546911718baa6d4afc0b-input.obj
writing altencoder-objs/40/output/train-9b0d7aac4500546911718baa6d4afc0b-output.obj
writing altencoder-objs/40/input/test-f61888bca4392106bc790478b4a84bf2-input.obj
writing altencoder-objs/40/output/test-f61888bca4392106bc790478b4a84bf2-output.obj
writing altencoder-objs/40/input/train-86c31c04c436f5932e877e82c90c24d-input.obj
writing altencoder-objs/40/output/train-86c31c04c436f5932e877e82c90c24d-output.obj
writing altencoder-objs/40/input/te

Process ForkPoolWorker-105:
Process ForkPoolWorker-101:
Process ForkPoolWorker-98:
Process ForkPoolWorker-99:
Process ForkPoolWorker-104:
Process ForkPoolWorker-97:
Process ForkPoolWorker-108:
Process ForkPoolWorker-106:
Process ForkPoolWorker-107:
Process ForkPoolWorker-102:
Process ForkPoolWorker-103:
Process ForkPoolWorker-100:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/p

  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/synchronize.py", line

In [48]:
y_train, y_test = model.predict(x_train), model.predict(x_test)

In [49]:
x_train[0]

array([-0.188041,  0.071757, -0.041727, -0.191469,  0.008376,  0.29926 ,
       -0.186761,  0.076401, -0.039019, -0.191926,  0.010706, -0.317276,
       -0.176773, -0.078195, -0.001716, -0.1874  , -0.107019,  0.266355,
       -0.19355 ,  0.027168,  0.255872, -0.193178,  0.020274, -0.28742 ,
       -0.1874  , -0.118797, -0.285778,  0.002345, -0.018081, -0.446181,
       -0.139854, -0.006102, -0.403974,  0.001501,  0.021966, -0.43595 ,
        0.144545, -0.006102, -0.403974,  0.002345, -0.066678, -0.445537,
       -0.129798, -0.059359, -0.40573 , -0.140946,  0.028435, -0.388389,
        0.145285,  0.028404, -0.388583,  0.132084, -0.059637, -0.406831,
        0.192732,  0.071757, -0.041727,  0.196616,  0.010706, -0.317276,
        0.191451,  0.076401, -0.039019,  0.19616 ,  0.008376,  0.29926 ,
        0.181463, -0.078195, -0.001712,  0.19209 , -0.118797, -0.286042,
        0.197869,  0.020274, -0.28742 ,  0.198237,  0.027142,  0.256007,
        0.19209 , -0.107019,  0.26609 ,  0.001707, 

In [50]:
y_train[0]

array([-1.90569937e-01,  2.16046330e-02, -2.42814086e-02, -1.83027416e-01,
       -5.75248152e-04,  3.04515243e-01, -1.83217198e-01,  6.07237145e-02,
       -3.82540114e-02, -1.83499277e-01, -1.13823311e-02, -3.08607608e-01,
       -1.74083784e-01, -6.93430454e-02, -3.85557255e-03, -1.81659818e-01,
       -9.26676616e-02,  2.62888491e-01, -1.77214965e-01,  5.37756905e-02,
        2.89658010e-01, -1.78229228e-01,  3.21672484e-02, -2.69728482e-01,
       -1.81090280e-01, -9.46186632e-02, -2.68231452e-01,  3.77157331e-03,
       -2.22427957e-02, -4.29328889e-01, -1.35522544e-01, -2.63939966e-02,
       -4.06195939e-01, -5.12617640e-04,  4.54471121e-03, -4.19039786e-01,
        1.32203668e-01, -2.92866174e-02, -4.10046130e-01,  1.58825703e-03,
       -6.69832081e-02, -4.21616077e-01, -1.27337530e-01, -6.67602643e-02,
       -4.04059321e-01, -1.36281550e-01, -8.13290011e-04, -3.94701898e-01,
        1.35797545e-01,  4.93134093e-03, -3.95223409e-01,  1.30829513e-01,
       -6.47926629e-02, -

In [59]:
delta = x_train[0] - y_train[0]
delta / x_train[0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


array([-1.34488608e-02,  6.98919507e-01,  4.18088800e-01,  4.40885132e-02,
        1.06867815e+00, -1.75607919e-02,  1.89750662e-02,  2.05197386e-01,
        1.96055403e-02,  4.39061063e-02,  2.06317309e+00,  2.73212973e-02,
        1.52128197e-02,  1.13203589e-01, -1.24683715e+00,  3.06306420e-02,
        1.34100846e-01,  1.30146190e-02,  8.43969764e-02, -9.79376122e-01,
       -1.32042623e-01,  7.73834098e-02, -5.86625650e-01,  6.15528433e-02,
        3.36697950e-02,  2.03526493e-01,  6.13992277e-02, -6.08346825e-01,
       -2.30175083e-01,  3.77696750e-02,  3.09712673e-02, -3.32546649e+00,
       -5.50020196e-03,  1.34151742e+00,  7.93102467e-01,  3.87893431e-02,
        8.53805503e-02, -3.79951121e+00, -1.50309931e-02,  3.22704891e-01,
       -4.57734276e-03,  5.36900921e-02,  1.89561453e-02, -1.24686473e-01,
        4.11771205e-03,  3.30938804e-02,  1.02860172e+00, -1.62540600e-02,
        6.53023690e-02,  8.26385688e-01, -1.70887788e-02,  9.49764489e-03,
       -8.64507413e-02, -